In [1]:
import datetime
from sklearn.feature_selection import chi2, SelectPercentile
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import lightgbm as lgb
import warnings
import time
import pandas as pd
import numpy as np
import os
import re
import gc
import pandas as pd
import urllib.parse
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import math

c:\users\hasee\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
warnings.filterwarnings("ignore")
path = './data1/train'
# round2 train
train1 = pd.read_csv(path + '/operation_train_new.csv')
train2 = pd.read_csv(path + '/transaction_train_new.csv')
train3 = pd.read_csv(path + '/tag_train_new.csv')

# round1 test
test1_round1 = pd.read_csv(path + '/operation_round1_new.csv')
test2_round1 = pd.read_csv(path + '/transaction_round1_new.csv')
test1_round1['day'] = test1_round1['day'].apply(lambda x: x + 30)
test2_round1['day'] = test2_round1['day'].apply(lambda x: x + 30)
result_round1 = pd.read_csv(path + '/test_round1_tag.csv')

operation_data = pd.concat([train1, test1_round1], axis=0, ignore_index=True)
transaction_data = pd.concat([train2, test2_round1], axis=0, ignore_index=True)
data = pd.concat([train3, result_round1], axis=0, ignore_index=True)


# merchant nunique10-1000 and bal > 300
temp = transaction_data[transaction_data.bal > 300][['UID', 'merchant']]
temp1 = temp.copy()
temp['merchant_nunique'] = temp['UID']
temp = temp.groupby('merchant')['merchant_nunique'].agg('nunique').reset_index()
merchant1 = list(temp[(temp.merchant_nunique > 10)&(temp.merchant_nunique < 1000)]['merchant'])
temp1 = list(temp1[temp1.merchant.isin(merchant1)]['UID'].drop_duplicates())

data = data[~data.UID.isin(temp1)]
transaction_data = transaction_data[~transaction_data.UID.isin(temp1)]
operation_data = operation_data[~operation_data.UID.isin(temp1)]


# round2 test
path1 = './data1/test'
test1 = pd.read_csv(path1 + '/test_operation_round2.csv')
test2 = pd.read_csv(path1 + '/test_transaction_round2.csv')
test1['day'] = test1['day'].apply(lambda x: x + 64)
test2['day'] = test2['day'].apply(lambda x: x + 64)
result = pd.read_csv(path1 + '/submit_example.csv')

operation_data = pd.concat([operation_data, test1], axis=0, ignore_index=True)
transaction_data = pd.concat([transaction_data, test2], axis=0, ignore_index=True)
data = pd.concat([data, result[['UID']]], axis=0, ignore_index=True)

operation_data['hour'] = operation_data['time'].apply(lambda x:int(x.split(':')[0]))
transaction_data['hour'] = transaction_data['time'].apply(lambda x:int(x.split(':')[0]))

In [14]:
# 去掉channel118
data = data[~data.UID.isin(list(set(transaction_data[transaction_data.channel.isin([118])]['UID'])))]
operation_data = operation_data[~operation_data.UID.isin(list(set(transaction_data[transaction_data.channel.isin([118])]['UID'])))]
transaction_data = transaction_data[~transaction_data.UID.isin(list(set(transaction_data[transaction_data.channel.isin([118])]['UID'])))]

In [16]:
# 去掉amt_src1 fd4d2d1006a95637
data = data[~data.UID.isin(list(set(transaction_data[transaction_data['amt_src1'].isin(['fd4d2d1006a95637'])]['UID'])))]
operation_data = operation_data[~operation_data.UID.isin(list(set(transaction_data[transaction_data['amt_src1'].isin(['fd4d2d1006a95637'])]['UID'])))]
transaction_data = transaction_data[~transaction_data.UID.isin(list(set(transaction_data[transaction_data['amt_src1'].isin(['fd4d2d1006a95637'])]['UID'])))]

In [17]:
# 去掉code1 f1fa4af14fd5b68f
data = data[~data.UID.isin(list(set(transaction_data[transaction_data['code1'].isin(['f1fa4af14fd5b68f'])]['UID'])))]
operation_data = operation_data[~operation_data.UID.isin(list(set(transaction_data[transaction_data['code1'].isin(['f1fa4af14fd5b68f'])]['UID'])))]
transaction_data = transaction_data[~transaction_data.UID.isin(list(set(transaction_data[transaction_data['code1'].isin(['f1fa4af14fd5b68f'])]['UID'])))]

In [18]:
"""
操作表零碎特征：
"""
# 每一个用户操作次数
operation_count = operation_data[['UID']]
operation_count['operation_count'] = 1
operation_count = operation_count.groupby('UID').agg('count').reset_index()
data = pd.merge(data, operation_count, on='UID', how='left')

# 每一个用户交易次数
transaction_count = transaction_data[['UID']]
transaction_count['transaction_count'] = 1
transaction_count = transaction_count.groupby('UID').agg('count').reset_index()
data = pd.merge(data, transaction_count, on='UID', how='left')

# 01编码：是否有操作或者交易
data['has_operation'] = data['operation_count'].apply(lambda x: 1 if x > 0 else 0)
data['has_transaction'] = data['transaction_count'].apply(lambda x: 1 if x > 0 else 0)

# 交易次数 - 操作次数
data['operation_transaction_count_gap'] = data['transaction_count'] - data['operation_count']
data = data.drop(['transaction_count', 'operation_count'], axis=1)

# 成功次数、成功率
success = operation_data[['UID', 'success']]
success = success.groupby('UID')['success'].agg('sum').reset_index()
all = operation_data[['UID']]
all['all'] = 1
all = all.groupby('UID').agg('count').reset_index()
all = pd.merge(all, success, on='UID', how='left')
all['success_rate'] = all['success'] / all['all']
all = all[['UID', 'success_rate']]
data = pd.merge(data, all, on='UID', how='left')

# 每一种操作类型的成功率
mode_success = operation_data[['UID', 'mode', 'success']]
mode_success = mode_success.groupby(['UID', 'mode'])['success'].agg('sum').reset_index()
mode_all = operation_data[['UID', 'mode']]
mode_all['mode_all'] = 1
mode_all = mode_all.groupby(['UID', 'mode']).agg('count').reset_index()
mode_all = pd.merge(mode_all, mode_success, on=['UID', 'mode'], how='left')
mode_all['success_rate'] = mode_all['success'] / mode_all['mode_all']
mode_all = mode_all[['UID', 'success_rate']]
mode_all = mode_all.groupby('UID')['success_rate'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
mode_all.columns = ['UID', 'mode_success_max', 'mode_success_std', 'mode_success_sum', 'mode_success_min', 'mode_success_mean']
data = pd.merge(data, mode_all, on='UID', how='left')

In [19]:
def get_nunique(data, table, x, t):
    """
    每一个UID字段的x种类数
    table: 交易表或者操作表
    x：字段名
    """
    temp = table[['UID', x]]
    temp[x + '_nunique_of_' + t] = temp[x]
    temp = temp.groupby('UID')[x + '_nunique_of_' + t].agg('nunique').reset_index()
    data = pd.merge(data, temp, on='UID', how='left')
    return data

operation_label = ['day', 'mode', 'os', 'version', 'device1', 'device2', 'mac1', 'ip1', 'ip2', 'device_code3',
                   'mac2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub'] 
transaction_label = ['day', 'channel', 'amt_src1', 'merchant', 'code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
                     'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1', 'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2', 'market_code', 'market_type', 'ip1_sub'] 
for x in operation_label: 
    data = get_nunique(data, operation_data, x, 'operation') 
for x in transaction_label: 
    data = get_nunique(data, transaction_data, x, 'transaction')

In [20]:
def get_time_frequence(data, table):  
    """
    用户级：用户时间频率
    """
    # 每一个用户每一天每一小时操作数的最大值，最小值，均值，标准差
    frequence_one_hour = table[['UID', 'day', 'hour']]
    frequence_one_hour['everyday_everyhour'] = 1
    frequence_one_hour = frequence_one_hour.groupby(['UID', 'day', 'hour']).agg('count').reset_index()
    frequence_one_hour = frequence_one_hour.drop(['day', 'hour'],axis = 1)
    frequence_one_hour = frequence_one_hour.groupby('UID')['everyday_everyhour'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_one_hour.columns = ['UID', 'frequence_one_hour_mean', 'frequence_one_hour_max', 'frequence_one_hour_min', 'frequence_one_hour_std']
    data = pd.merge(data, frequence_one_hour, on='UID', how='left')

    # 每一个用户每一天操作数的最大值，最小值，均值，标准差
    frequence_one_day = table[['UID', 'day']]
    frequence_one_day['everyday'] = 1
    frequence_one_day = frequence_one_day.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_one_day = frequence_one_day.drop('day', axis=1)
    frequence_one_day = frequence_one_day.groupby('UID')['everyday'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_one_day.columns = ['UID', 'frequence_one_day_mean', 'frequence_one_day_max', 'frequence_one_day_min', 'frequence_one_day_std']
    data = pd.merge(data, frequence_one_day, on='UID', how='left')

    # 每一个用户每一天半夜、早上、下午、晚上的最大值，最小值，均值，标准差
    frequence_one_hour = table[['UID', 'day', 'hour']]
    frequence_one_hour['everyday_everyhour'] = 1
    frequence_one_hour = frequence_one_hour.groupby(['UID', 'day', 'hour']).agg('count').reset_index()

    frequence_morning = frequence_one_hour[frequence_one_hour.hour <= 5][['UID', 'day', 'everyday_everyhour']]
    frequence_morning['everyday_morning'] = frequence_morning['everyday_everyhour']
    frequence_morning = frequence_morning.groupby(['UID', 'day'])['everyday_morning'].agg('sum').reset_index()
    frequence_morning = frequence_morning[['UID', 'everyday_morning']]
    frequence_morning = frequence_morning.groupby('UID')['everyday_morning'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_morning.columns = ['UID', 'frequence_morning_mean', 'frequence_morning_max', 'frequence_morning_min', 'frequence_morning_std']
    data = pd.merge(data, frequence_morning, on='UID', how='left')

    frequence_work_time = frequence_one_hour[(frequence_one_hour.hour <= 11)&(frequence_one_hour.hour >= 6)][['UID', 'day', 'everyday_everyhour']]
    frequence_work_time['everyday_work_time'] = frequence_work_time['everyday_everyhour']
    frequence_work_time = frequence_work_time.groupby(['UID', 'day'])['everyday_work_time'].agg('sum').reset_index()
    frequence_work_time = frequence_work_time[['UID', 'everyday_work_time']]
    frequence_work_time = frequence_work_time.groupby('UID')['everyday_work_time'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_work_time.columns = ['UID', 'frequence_work_time_mean', 'frequence_work_time_max', 'frequence_work_time_min', 'frequence_work_time_std']
    data = pd.merge(data, frequence_work_time, on='UID', how='left')

    frequence_afternoon = frequence_one_hour[(frequence_one_hour.hour <= 17)&(frequence_one_hour.hour >= 12)][['UID', 'day', 'everyday_everyhour']]
    frequence_afternoon['everyday_afternoon'] = frequence_afternoon['everyday_everyhour']
    frequence_afternoon = frequence_afternoon.groupby(['UID', 'day'])['everyday_afternoon'].agg('sum').reset_index()
    frequence_afternoon = frequence_afternoon[['UID', 'everyday_afternoon']]
    frequence_afternoon = frequence_afternoon.groupby('UID')['everyday_afternoon'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_afternoon.columns = ['UID', 'frequence_afternoon_mean', 'frequence_afternoon_max', 'frequence_afternoon_min', 'frequence_afternoon_std']
    data = pd.merge(data, frequence_afternoon, on='UID', how='left')

    frequence_night = frequence_one_hour[(frequence_one_hour.hour <= 23)&(frequence_one_hour.hour >= 18)][['UID', 'day', 'everyday_everyhour']]
    frequence_night['everyday_night'] = frequence_night['everyday_everyhour']
    frequence_night = frequence_night.groupby(['UID', 'day'])['everyday_night'].agg('sum').reset_index()
    frequence_night = frequence_night[['UID', 'everyday_night']]
    frequence_night = frequence_night.groupby('UID')['everyday_night'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    frequence_night.columns = ['UID', 'frequence_night_mean', 'frequence_night_max', 'frequence_night_min', 'frequence_night_std']
    data = pd.merge(data, frequence_night, on='UID', how='left')

    # 每一个用户这一天距离前面一段时间操作数的最大值，最小值，均值，标准差
    frequence_one_day_gap = table[['UID', 'day']]
    frequence_one_day_gap['everyday'] = 1
    frequence_one_day_gap = frequence_one_day_gap.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_one_day_gap['everyday_before'] = frequence_one_day_gap.groupby('UID')['everyday'].shift(1)
    frequence_one_day_gap['everyday_before_gap'] = frequence_one_day_gap['everyday'] - frequence_one_day_gap['everyday_before']
    frequence_one_day_gap = frequence_one_day_gap[['UID', 'everyday_before_gap']].groupby('UID')['everyday_before_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    frequence_one_day_gap.columns = ['UID', 'frequence_one_day_gap_sum', 'frequence_one_day_gap_mean', 'frequence_one_day_gap_max', 'frequence_one_day_gap_min', 'frequence_one_day_gap_std']
    data = pd.merge(data, frequence_one_day_gap, on='UID', how='left')

    # 每一个用户这一小时距离前面一段时间操作数的最大值，最小值，均值，标准差
    frequence_one_hour_gap = table[['UID', 'day', 'hour']]
    frequence_one_hour_gap['everyhour'] = 1
    frequence_one_hour_gap = frequence_one_hour_gap.groupby(['UID', 'day', 'hour']).agg('count').reset_index()
    frequence_one_hour_gap['everyhour_before'] = frequence_one_hour_gap.groupby('UID')['everyhour'].shift(1)
    frequence_one_hour_gap['everyhour_before_gap'] = frequence_one_hour_gap['everyhour'] - frequence_one_hour_gap['everyhour_before']
    frequence_one_hour_gap = frequence_one_hour_gap[['UID', 'everyhour_before_gap']].groupby('UID')['everyhour_before_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    frequence_one_hour_gap.columns = ['UID','frequence_one_hour_gap_sum' ,'frequence_one_hour_gap_mean', 'frequence_one_hour_gap_max', 'frequence_one_hour_gap_min', 'frequence_one_hour_gap_std']
    data = pd.merge(data, frequence_one_hour_gap, on='UID', how='left')

    # 每一个用户这一天距离前面一段时间操作数的比率和
    frequence_one_day_rate = table[['UID', 'day']]
    frequence_one_day_rate['everyday'] = 1
    frequence_one_day_rate = frequence_one_day_rate.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_one_day_rate['everyday_before'] = frequence_one_day_rate.groupby('UID')['everyday'].shift(1)
    frequence_one_day_rate['everyday_before_rate'] = frequence_one_day_rate['everyday'] / frequence_one_day_rate['everyday_before']
    frequence_one_day_rate = frequence_one_day_rate[['UID', 'everyday_before_rate']].groupby('UID')['everyday_before_rate'].agg('sum').reset_index()
    data = pd.merge(data, frequence_one_day_rate, on='UID', how='left')

    # 每一个用户这一小时距离前面一段时间操作数的比率和
    frequence_one_hour_rate = table[['UID', 'day', 'hour']]
    frequence_one_hour_rate['everyhour'] = 1
    frequence_one_hour_rate = frequence_one_hour_rate.groupby(['UID', 'day', 'hour']).agg('count').reset_index()
    frequence_one_hour_rate['everyhour_before'] = frequence_one_hour_rate.groupby('UID')['everyhour'].shift(1)
    frequence_one_hour_rate['everyhour_before_rate'] = frequence_one_hour_rate['everyhour'] - frequence_one_hour_rate['everyhour_before']
    frequence_one_hour_rate = frequence_one_hour_rate[['UID', 'everyhour_before_rate']].groupby('UID')['everyhour_before_rate'].agg('sum').reset_index()
    data = pd.merge(data, frequence_one_hour_rate, on='UID', how='left')

    # 每一个用户每三天的操作量的最大值、最小值、均值、标准差
    frequence_three_day = table[['UID', 'day']]
    frequence_three_day['three_day'] = 1
    frequence_three_day = frequence_three_day.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_three_day['before_three_day'] = frequence_three_day.groupby('UID')['three_day'].shift(3)
    frequence_three_day['three_day_gap'] = frequence_three_day['before_three_day'] - frequence_three_day['three_day']
    frequence_three_day = frequence_three_day[['UID', 'three_day_gap']]
    frequence_three_day = frequence_three_day.groupby('UID')['three_day_gap'].agg({'sum', 'max', 'min', 'mean', 'std'}).reset_index()
    frequence_three_day.columns = ['UID', 'frequence_three_day_sum', 'frequence_three_day_mean', 'frequence_three_day_max', 'frequence_three_day_min', 'frequence_three_day_std']
    data = pd.merge(data, frequence_three_day, on='UID', how='left')

    # 每一个用户每七天的操作量的最大值、最小值、均值、标准差
    frequence_sixteen_day = table[['UID', 'day']]
    frequence_sixteen_day['sixteen_day'] = 1
    frequence_sixteen_day = frequence_sixteen_day.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_sixteen_day['before_sixteen_day'] = frequence_sixteen_day.groupby('UID')['sixteen_day'].shift(7)
    frequence_sixteen_day['sixteen_day_gap'] = frequence_sixteen_day['before_sixteen_day'] - frequence_sixteen_day['sixteen_day']
    frequence_sixteen_day = frequence_sixteen_day[['UID', 'sixteen_day_gap']]
    frequence_sixteen_day = frequence_sixteen_day.groupby('UID')['sixteen_day_gap'].agg({'sum', 'max', 'min', 'mean', 'std'}).reset_index()
    frequence_sixteen_day.columns = ['UID', 'frequence_sixteen_day_sum', 'frequence_sixteen_day_mean', 'frequence_sixteen_day_max', 'frequence_sixteen_day_min', 'frequence_sixteen_day_std']
    data = pd.merge(data, frequence_sixteen_day, on='UID', how='left')

    # 每一个用户每三天的操作量的最大值、最小值、均值、标准差比率和
    frequence_three_day = table[['UID', 'day']]
    frequence_three_day['three_day'] = 1
    frequence_three_day = frequence_three_day.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_three_day['before_three_day'] = frequence_three_day.groupby('UID')['three_day'].shift(3)
    frequence_three_day['three_day_rate'] = frequence_three_day['three_day'] / frequence_three_day['before_three_day']
    frequence_three_day = frequence_three_day[['UID', 'three_day_rate']]
    frequence_three_day = frequence_three_day.groupby('UID')['three_day_rate'].agg('sum').reset_index()
    data = pd.merge(data, frequence_three_day, on='UID', how='left')

    # 每一个用户每七天的操作量的最大值、最小值、均值、标准差比率和
    frequence_sixteen_day = table[['UID', 'day']]
    frequence_sixteen_day['sixteen_day'] = 1
    frequence_sixteen_day = frequence_sixteen_day.groupby(['UID', 'day']).agg('count').reset_index()
    frequence_sixteen_day['before_sixteen_day'] = frequence_sixteen_day.groupby('UID')['sixteen_day'].shift(7)
    frequence_sixteen_day['sixteen_day_rate'] = frequence_sixteen_day['sixteen_day'] / frequence_sixteen_day['before_sixteen_day']
    frequence_sixteen_day = frequence_sixteen_day[['UID', 'sixteen_day_rate']]
    frequence_sixteen_day = frequence_sixteen_day.groupby('UID')['sixteen_day_rate'].agg('sum').reset_index()
    data = pd.merge(data, frequence_sixteen_day, on='UID', how='left')

    # 每秒用户的操作最大值、最小值、均值、方差
    every_second = table[['UID', 'time']]
    every_second['operation_second_caozuo'] = 1
    every_second = every_second.groupby(['UID', 'time']).agg('count').reset_index()
    every_second = every_second[['UID', 'operation_second_caozuo']]
    every_second = every_second.groupby('UID')['operation_second_caozuo'].agg({'max', 'min', 'mean', 'std'}).reset_index()
    every_second.columns = ['UID', 'every_second_mean', 'every_second_max', 'every_second_min', 'every_second_std']
    data = pd.merge(data, every_second, on='UID', how='left')

    # 每分钟用户操作的最大值、最小值、均值、方差
    every_minute = table[['UID', 'time']]
    every_minute['every_minute'] = every_minute['time'].apply(lambda x: str(int(x.split(':')[0])) + str(int(x.split(':')[1])))
    every_minute = every_minute[['UID', 'every_minute']]
    every_minute['operation_minute_caozuo'] = 1
    every_minute = every_minute.groupby(['UID', 'every_minute']).agg('count').reset_index()
    every_minute = every_minute[['UID', 'operation_minute_caozuo']]
    every_minute = every_minute.groupby('UID')['operation_minute_caozuo'].agg({'max', 'min', 'mean', 'std'}).reset_index()
    every_minute.columns = ['UID', 'every_minute_mean', 'every_minute_max', 'every_minute_min', 'every_minute_std']
    data = pd.merge(data, every_minute, on='UID', how='left')

    return data

data = get_time_frequence(data, operation_data)
data = get_time_frequence(data, transaction_data)

In [21]:
"""
用户级：用户交易金额时间频率
"""
# 用户每天的每小时交易金额的和、均值、最大值、最小值、标准差
money_frequence_one_hour = transaction_data[['UID', 'day', 'hour', 'trans_amt']]
money_frequence_one_hour = money_frequence_one_hour.groupby(['UID', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
money_frequence_one_hour = money_frequence_one_hour[['UID', 'trans_amt']]
money_frequence_one_hour['money_frequence_one_hour'] = money_frequence_one_hour['trans_amt']
money_frequence_one_hour = money_frequence_one_hour.groupby('UID')['money_frequence_one_hour'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_one_hour.columns = ['UID', 'money_frequence_one_hour_max', 'money_frequence_one_hour_std', 'money_frequence_one_hour_sum', 'money_frequence_one_hour_min', 'money_frequence_one_hour_mean']
data = pd.merge(data, money_frequence_one_hour, on='UID', how='left')

# 用户每天的每小时交易金额的上下两段时间的增量
money_frequence_one_hour_gap = transaction_data[['UID', 'day', 'hour', 'trans_amt']]
money_frequence_one_hour_gap = money_frequence_one_hour_gap.groupby(['UID', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
money_frequence_one_hour_gap['before_hour_amt'] = money_frequence_one_hour_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_one_hour_gap['money_frequence_one_hour_gap'] = money_frequence_one_hour_gap['trans_amt'] - money_frequence_one_hour_gap['before_hour_amt']
money_frequence_one_hour_gap = money_frequence_one_hour_gap[['UID', 'money_frequence_one_hour_gap']]
money_frequence_one_hour_gap = money_frequence_one_hour_gap.groupby('UID')['money_frequence_one_hour_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_one_hour_gap.columns = ['UID', 'money_frequence_one_hour_sum', 'money_frequence_one_hour_max', 'money_frequence_one_hour_std', 'money_frequence_one_hour_min', 'money_frequence_one_hour_mean']
data = pd.merge(data, money_frequence_one_hour_gap, on='UID', how='left')

# 用户每天的每小时交易金额的上下两段时间的比率和
money_frequence_one_hour_rate = transaction_data[['UID', 'day', 'hour', 'trans_amt']]
money_frequence_one_hour_rate = money_frequence_one_hour_rate.groupby(['UID', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
money_frequence_one_hour_rate['before_hour_amt'] = money_frequence_one_hour_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_one_hour_rate['money_frequence_one_hour_rate'] = money_frequence_one_hour_rate['trans_amt'] / money_frequence_one_hour_rate['before_hour_amt']
money_frequence_one_hour_rate = money_frequence_one_hour_rate[['UID', 'money_frequence_one_hour_rate']]
money_frequence_one_hour_rate = money_frequence_one_hour_rate.groupby('UID')['money_frequence_one_hour_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_one_hour_rate, on='UID', how='left')


# 用户每天的五个小时的和、均值、最大值、最小值、标准差(半夜、早上、下午、晚上)，gap、rate
# for morning
money_frequence_morning = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'trans_amt']]
money_frequence_morning = money_frequence_morning.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_morning = money_frequence_morning[['UID', 'trans_amt']]
money_frequence_morning['trans_amt_sum'] = money_frequence_morning['trans_amt']
money_frequence_morning = money_frequence_morning.groupby('UID')['trans_amt_sum'].agg({'sum', 'mean', 'max', 'min', 'mean', 'std'}).reset_index()
money_frequence_morning.columns = ['UID', 'money_frequence_morning_max', 'money_frequence_morning_std', 'money_frequence_morning_sum', 'money_frequence_morning_min', 'money_frequence_morning_mean']
data = pd.merge(data, money_frequence_morning, on='UID', how='left')

money_frequence_morning_gap = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'trans_amt']]
money_frequence_morning_gap = money_frequence_morning_gap.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_morning_gap = money_frequence_morning_gap[['UID', 'trans_amt']]
money_frequence_morning_gap['trans_amt_before'] = money_frequence_morning_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_morning_gap['trans_amt_gap'] = money_frequence_morning_gap['trans_amt'] - money_frequence_morning_gap['trans_amt_before']
money_frequence_morning_gap = money_frequence_morning_gap[['UID', 'trans_amt_gap']]
money_frequence_morning_gap = money_frequence_morning_gap.groupby('UID')['trans_amt_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_morning_gap.columns = ['UID', 'money_frequence_morning_gap_sum', 'money_frequence_morning_gap_max', 'money_frequence_morning_gap_std', 'money_frequence_morning_gap_min', 'money_frequence_morning_gap_mean']
data = pd.merge(data, money_frequence_morning_gap, on='UID', how='left')

money_frequence_morning_rate = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'trans_amt']]
money_frequence_morning_rate = money_frequence_morning_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_morning_rate = money_frequence_morning_rate[['UID', 'trans_amt']]
money_frequence_morning_rate['trans_amt_before'] = money_frequence_morning_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_morning_rate['trans_amt_rate'] = money_frequence_morning_rate['trans_amt'] / money_frequence_morning_rate['trans_amt_before']
money_frequence_morning_rate = money_frequence_morning_rate[['UID', 'trans_amt_rate']]
money_frequence_morning_rate = money_frequence_morning_rate.groupby('UID')['trans_amt_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_morning_rate, on='UID', how='left')

# for work_time
money_frequence_work_time = transaction_data[(transaction_data.hour >=6)&(transaction_data.hour <= 11)][['UID', 'day', 'trans_amt']]
money_frequence_work_time = money_frequence_work_time.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_work_time = money_frequence_work_time[['UID', 'trans_amt']]
money_frequence_work_time['trans_amt_sum'] = money_frequence_work_time['trans_amt']
money_frequence_work_time = money_frequence_work_time.groupby('UID')['trans_amt_sum'].agg({'sum', 'mean', 'max', 'min', 'mean', 'std'}).reset_index()
money_frequence_work_time.columns = ['UID', 'money_frequence_work_time_max', 'money_frequence_work_time_std', 'money_frequence_work_time_sum', 'money_frequence_work_time_min', 'money_frequence_work_time_mean']
data = pd.merge(data, money_frequence_work_time, on='UID', how='left')

money_frequence_work_time_gap = transaction_data[(transaction_data.hour >=6)&(transaction_data.hour <= 11)][['UID', 'day', 'trans_amt']]
money_frequence_work_time_gap = money_frequence_work_time_gap.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_work_time_gap = money_frequence_work_time_gap[['UID', 'trans_amt']]
money_frequence_work_time_gap['trans_amt_before'] = money_frequence_work_time_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_work_time_gap['trans_amt_gap'] = money_frequence_work_time_gap['trans_amt'] - money_frequence_work_time_gap['trans_amt_before']
money_frequence_work_time_gap = money_frequence_work_time_gap[['UID', 'trans_amt_gap']]
money_frequence_work_time_gap = money_frequence_work_time_gap.groupby('UID')['trans_amt_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_work_time_gap.columns = ['UID', 'money_frequence_work_time_gap_sum', 'money_frequence_work_time_gap_max', 'money_frequence_work_time_gap_std', 'money_frequence_work_time_gap_min', 'money_frequence_work_time_gap_mean']
data = pd.merge(data, money_frequence_work_time_gap, on='UID', how='left')

money_frequence_work_time_rate = transaction_data[(transaction_data.hour >=6)&(transaction_data.hour <= 11)][['UID', 'day', 'trans_amt']]
money_frequence_work_time_rate = money_frequence_work_time_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_work_time_rate = money_frequence_work_time_rate[['UID', 'trans_amt']]
money_frequence_work_time_rate['trans_amt_before'] = money_frequence_work_time_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_work_time_rate['trans_amt_rate'] = money_frequence_work_time_rate['trans_amt'] / money_frequence_work_time_rate['trans_amt_before']
money_frequence_work_time_rate = money_frequence_work_time_rate[['UID', 'trans_amt_rate']]
money_frequence_work_time_rate = money_frequence_work_time_rate.groupby('UID')['trans_amt_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_work_time_rate, on='UID', how='left')

# for afternoon
money_frequence_afternoon = transaction_data[(transaction_data.hour >=12)&(transaction_data.hour <= 17)][['UID', 'day', 'trans_amt']]
money_frequence_afternoon = money_frequence_afternoon.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_afternoon = money_frequence_afternoon[['UID', 'trans_amt']]
money_frequence_afternoon['trans_amt_sum'] = money_frequence_afternoon['trans_amt']
money_frequence_afternoon = money_frequence_afternoon.groupby('UID')['trans_amt_sum'].agg({'sum', 'mean', 'max', 'min', 'mean', 'std'}).reset_index()
money_frequence_afternoon.columns = ['UID', 'money_frequence_afternoon_max', 'money_frequence_afternoon_std', 'money_frequence_afternoon_sum', 'money_frequence_afternoon_min', 'money_frequence_afternoon_mean']
data = pd.merge(data, money_frequence_afternoon, on='UID', how='left')

money_frequence_afternoon_gap = transaction_data[(transaction_data.hour >=12)&(transaction_data.hour <= 17)][['UID', 'day', 'trans_amt']]
money_frequence_afternoon_gap = money_frequence_afternoon_gap.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_afternoon_gap = money_frequence_afternoon_gap[['UID', 'trans_amt']]
money_frequence_afternoon_gap['trans_amt_before'] = money_frequence_afternoon_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_afternoon_gap['trans_amt_gap'] = money_frequence_afternoon_gap['trans_amt'] - money_frequence_afternoon_gap['trans_amt_before']
money_frequence_afternoon_gap = money_frequence_afternoon_gap[['UID', 'trans_amt_gap']]
money_frequence_afternoon_gap = money_frequence_afternoon_gap.groupby('UID')['trans_amt_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_afternoon_gap.columns = ['UID', 'money_frequence_afternoon_gap_sum', 'money_frequence_afternoon_gap_max', 'money_frequence_afternoon_gap_std', 'money_frequence_afternoon_gap_min', 'money_frequence_afternoon_gap_mean']
data = pd.merge(data, money_frequence_afternoon_gap, on='UID', how='left')

money_frequence_afternoon_rate = transaction_data[(transaction_data.hour >=12)&(transaction_data.hour <= 17)][['UID', 'day', 'trans_amt']]
money_frequence_afternoon_rate = money_frequence_afternoon_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_afternoon_rate = money_frequence_afternoon_rate[['UID', 'trans_amt']]
money_frequence_afternoon_rate['trans_amt_before'] = money_frequence_afternoon_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_afternoon_rate['trans_amt_rate'] = money_frequence_afternoon_rate['trans_amt'] / money_frequence_afternoon_rate['trans_amt_before']
money_frequence_afternoon_rate = money_frequence_afternoon_rate[['UID', 'trans_amt_rate']]
money_frequence_afternoon_rate = money_frequence_afternoon_rate.groupby('UID')['trans_amt_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_afternoon_rate, on='UID', how='left')

# for night
money_frequence_night = transaction_data[(transaction_data.hour >=18)&(transaction_data.hour <= 23)][['UID', 'day', 'trans_amt']]
money_frequence_night = money_frequence_night.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_night = money_frequence_night[['UID', 'trans_amt']]
money_frequence_night['trans_amt_sum'] = money_frequence_night['trans_amt']
money_frequence_night = money_frequence_night.groupby('UID')['trans_amt_sum'].agg({'sum', 'mean', 'max', 'min', 'mean', 'std'}).reset_index()
money_frequence_night.columns = ['UID', 'money_frequence_night_max', 'money_frequence_night_std', 'money_frequence_night_sum', 'money_frequence_night_min', 'money_frequence_night_mean']
data = pd.merge(data, money_frequence_night, on='UID', how='left')

money_frequence_night_gap = transaction_data[(transaction_data.hour >=18)&(transaction_data.hour <= 23)][['UID', 'day', 'trans_amt']]
money_frequence_night_gap = money_frequence_night_gap.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_night_gap = money_frequence_night_gap[['UID', 'trans_amt']]
money_frequence_night_gap['trans_amt_before'] = money_frequence_night_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_night_gap['trans_amt_gap'] = money_frequence_night_gap['trans_amt'] - money_frequence_night_gap['trans_amt_before']
money_frequence_night_gap = money_frequence_night_gap[['UID', 'trans_amt_gap']]
money_frequence_night_gap = money_frequence_night_gap.groupby('UID')['trans_amt_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_night_gap.columns = ['UID', 'money_frequence_night_gap_sum', 'money_frequence_night_gap_max', 'money_frequence_night_gap_std', 'money_frequence_night_gap_min', 'money_frequence_night_gap_mean']
data = pd.merge(data, money_frequence_night_gap, on='UID', how='left')

money_frequence_night_rate = transaction_data[(transaction_data.hour >=18)&(transaction_data.hour <= 23)][['UID', 'day', 'trans_amt']]
money_frequence_night_rate = money_frequence_night_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_night_rate = money_frequence_night_rate[['UID', 'trans_amt']]
money_frequence_night_rate['trans_amt_before'] = money_frequence_night_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_night_rate['trans_amt_rate'] = money_frequence_night_rate['trans_amt'] / money_frequence_night_rate['trans_amt_before']
money_frequence_night_rate = money_frequence_night_rate[['UID', 'trans_amt_rate']]
money_frequence_night_rate = money_frequence_night_rate.groupby('UID')['trans_amt_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_night_rate, on='UID', how='left')


# 用户每天的交易金额和、最大值、最小值、均值、极差、gap、rate
money_frequence_day = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_day = money_frequence_day.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_day = money_frequence_day[['UID', 'trans_amt']]
money_frequence_day['trans_amt_day'] = money_frequence_day['trans_amt']
money_frequence_day = money_frequence_day.groupby('UID')['trans_amt_day'].agg({'max', 'min', 'mean', 'std'}).reset_index()
money_frequence_day.columns = ['UID', 'money_frequence_day_max', 'money_frequence_day_std', 'money_frequence_day_min', 'money_frequence_day_mean']
data = pd.merge(data, money_frequence_day, on='UID', how='left')

money_frequence_day_gap = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_day_gap = money_frequence_day_gap.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_day_gap = money_frequence_day_gap[['UID', 'trans_amt']]
money_frequence_day_gap['trans_amt_before'] = money_frequence_day_gap.groupby('UID')['trans_amt'].shift(1)
money_frequence_day_gap['trans_amt_gap'] = money_frequence_day_gap['trans_amt'] - money_frequence_day_gap['trans_amt_before']
money_frequence_day_gap = money_frequence_day_gap[['UID', 'trans_amt_gap']]
money_frequence_day_gap = money_frequence_day_gap.groupby('UID')['trans_amt_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_day_gap.columns = ['UID', 'money_frequence_day_gap_sum', 'money_frequence_day_gap_max', 'money_frequence_day_gap_std', 'money_frequence_day_gap_min', 'money_frequence_day_gap_mean']
data = pd.merge(data, money_frequence_day_gap, on='UID', how='left')

money_frequence_day_rate = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_day_rate = money_frequence_day_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_day_rate = money_frequence_day_rate[['UID', 'trans_amt']]
money_frequence_day_rate['trans_amt_before'] = money_frequence_day_rate.groupby('UID')['trans_amt'].shift(1)
money_frequence_day_rate['trans_amt_rate'] = money_frequence_day_rate['trans_amt'] / money_frequence_day_rate['trans_amt_before']
money_frequence_day_rate = money_frequence_day_rate[['UID', 'trans_amt_rate']]
money_frequence_day_rate = money_frequence_day_rate.groupby('UID')['trans_amt_rate'].agg('sum').reset_index()
data = pd.merge(data, money_frequence_day_rate, on='UID', how='left')


# 每一个用户每三天的交易量的差值的最大值、最小值、均值、标准差
money_frequence_three_day = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_three_day = money_frequence_three_day.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_three_day['trans_amt_before'] = money_frequence_three_day.groupby('UID')['trans_amt'].shift(3)
money_frequence_three_day['trans_amt_three_day_gap'] = money_frequence_three_day['trans_amt'] - money_frequence_three_day['trans_amt_before']
money_frequence_three_day = money_frequence_three_day[['UID', 'trans_amt_three_day_gap']]
money_frequence_three_day = money_frequence_three_day.groupby('UID')['trans_amt_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_three_day.columns = ['UID', 'money_frequence_three_day_sum', 'money_frequence_three_day_max', 'money_frequence_three_day_std', 'money_frequence_three_day_min', 'money_frequence_three_day_mean']
data = pd.merge(data, money_frequence_three_day, on='UID', how='left')

money_frequence_three_day_rate = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_three_day_rate = money_frequence_three_day_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_three_day_rate['trans_amt_before'] = money_frequence_three_day_rate.groupby('UID')['trans_amt'].shift(3)
money_frequence_three_day_rate['trans_amt_three_day_rate'] = money_frequence_three_day_rate['trans_amt'] / money_frequence_three_day_rate['trans_amt_before']
money_frequence_three_day_rate = money_frequence_three_day_rate[['UID', 'trans_amt_three_day_rate']]
money_frequence_three_day_rate = money_frequence_three_day_rate.groupby('UID')['trans_amt_three_day_rate'].agg({'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_three_day_rate.columns = ['UID', 'money_frequence_three_day_rate_max', 'money_frequence_three_day_rate_std', 'money_frequence_three_day_rate_min', 'money_frequence_three_day_rate_mean']
data = pd.merge(data, money_frequence_three_day_rate, on='UID', how='left')

# 每一个用户每七天的交易量的差值的最大值、最小值、均值、标准差
money_frequence_sixteen_day = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_sixteen_day = money_frequence_sixteen_day.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_sixteen_day['trans_amt_before'] = money_frequence_sixteen_day.groupby('UID')['trans_amt'].shift(7)
money_frequence_sixteen_day['trans_amt_sixteen_day_gap'] = money_frequence_sixteen_day['trans_amt'] - money_frequence_sixteen_day['trans_amt_before']
money_frequence_sixteen_day = money_frequence_sixteen_day[['UID', 'trans_amt_sixteen_day_gap']]
money_frequence_sixteen_day = money_frequence_sixteen_day.groupby('UID')['trans_amt_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_sixteen_day.columns = ['UID', 'money_frequence_sixteen_day_sum', 'money_frequence_sixteen_day_max', 'money_frequence_sixteen_day_std', 'money_frequence_sixteen_day_min', 'money_frequence_sixteen_day_mean']
data = pd.merge(data, money_frequence_sixteen_day, on='UID', how='left')

money_frequence_sixteen_day_rate = transaction_data[['UID', 'day', 'trans_amt']]
money_frequence_sixteen_day_rate = money_frequence_sixteen_day_rate.groupby(['UID', 'day'])['trans_amt'].agg('sum').reset_index()
money_frequence_sixteen_day_rate['trans_amt_before'] = money_frequence_sixteen_day_rate.groupby('UID')['trans_amt'].shift(7)
money_frequence_sixteen_day_rate['trans_amt_sixteen_day_rate'] = money_frequence_sixteen_day_rate['trans_amt'] / money_frequence_sixteen_day_rate['trans_amt_before']
money_frequence_sixteen_day_rate = money_frequence_sixteen_day_rate[['UID', 'trans_amt_sixteen_day_rate']]
money_frequence_sixteen_day_rate = money_frequence_sixteen_day_rate.groupby('UID')['trans_amt_sixteen_day_rate'].agg({'mean', 'max', 'min', 'std'}).reset_index()
money_frequence_sixteen_day_rate.columns = ['UID', 'money_frequence_sixteen_day_rate_max', 'money_frequence_sixteen_day_rate_std', 'money_frequence_sixteen_day_rate_min', 'money_frequence_sixteen_day_rate_mean']
data = pd.merge(data, money_frequence_sixteen_day_rate, on='UID', how='left')

In [22]:
"""
用户级：merchant时间频率
"""
# merchant一天之内的总交易额
merchant_day = transaction_data[['day', 'merchant', 'trans_amt']]
merchant_day = merchant_day.groupby(['merchant', 'day'])['trans_amt'].agg('sum').reset_index()

merchant_UID = transaction_data[['UID', 'day', 'merchant']]

merchant_day = pd.merge(merchant_UID, merchant_day, on=['day', 'merchant'], how='left')
merchant_day = merchant_day[['UID', 'trans_amt']]
merchant_day['merchant_day_sum'] = merchant_day['trans_amt']
merchant_day = merchant_day.groupby('UID')['trans_amt'].agg('sum').reset_index()
data = pd.merge(data, merchant_day, on='UID', how='left')

In [23]:
"""
用户 * merchant时间频率
"""
# 用户merchant每天每小时的统计
merchant_every_hour = transaction_data[['UID', 'merchant', 'day', 'hour', 'trans_amt']]
merchant_every_hour = merchant_every_hour.groupby(['UID', 'merchant', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
merchant_every_hour = merchant_every_hour[['UID', 'trans_amt']]
merchant_every_hour = merchant_every_hour.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_hour.columns = ['UID', 'merchant_every_hour_std', 'merchant_every_hour_mean', 'merchant_every_hour_min', 'merchant_every_hour_max']
data = pd.merge(data, merchant_every_hour, on='UID', how='left')

# 用户merchant每天每小时的gap
merchant_every_hour_gap = transaction_data[['UID', 'merchant', 'day', 'hour', 'trans_amt']]
merchant_every_hour_gap = merchant_every_hour_gap.groupby(['UID', 'merchant', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
merchant_every_hour_gap = merchant_every_hour_gap[['UID', 'trans_amt']]
merchant_every_hour_gap['trans_amt_before'] = merchant_every_hour_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_hour_gap['trans_amt_hour_gap'] = merchant_every_hour_gap['trans_amt'] - merchant_every_hour_gap['trans_amt_before']
merchant_every_hour_gap = merchant_every_hour_gap[['UID', 'trans_amt_hour_gap']]
merchant_every_hour_gap = merchant_every_hour_gap.groupby('UID')['trans_amt_hour_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_hour_gap.columns = ['UID', 'merchant_every_hour_gap_sum', 'merchant_every_hour_gap_std', 'merchant_every_hour_gap_mean', 'merchant_every_hour_gap_min', 'merchant_every_hour_gap_max']
data = pd.merge(data, merchant_every_hour_gap, on='UID', how='left')

# 用户merchant每天每小时的比率和
merchant_every_hour_rate = transaction_data[['UID', 'merchant', 'day', 'hour', 'trans_amt']]
merchant_every_hour_rate = merchant_every_hour_rate.groupby(['UID', 'merchant', 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
merchant_every_hour_rate = merchant_every_hour_rate[['UID', 'trans_amt']]
merchant_every_hour_rate['trans_amt_before'] = merchant_every_hour_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_hour_rate['trans_amt_hour_rate'] = merchant_every_hour_rate['trans_amt'] / merchant_every_hour_rate['trans_amt_before']
merchant_every_hour_rate = merchant_every_hour_rate[['UID', 'trans_amt_hour_rate']]
merchant_every_hour_rate = merchant_every_hour_rate.groupby('UID')['trans_amt_hour_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_hour_rate, on='UID', how='left')


# 用户merchant每天的统计
merchant_every_day = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_day = merchant_every_day.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_day = merchant_every_day[['UID', 'trans_amt']]
merchant_every_day = merchant_every_day.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_day.columns = ['UID', 'merchant_every_day_std', 'merchant_every_day_mean', 'merchant_every_day_min', 'merchant_every_day_max']
data = pd.merge(data, merchant_every_day, on='UID', how='left')

# 用户merchant每天的gap
merchant_every_day_gap = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_day_gap = merchant_every_day_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_day_gap = merchant_every_day_gap[['UID', 'trans_amt']]
merchant_every_day_gap['trans_amt_before'] = merchant_every_day_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_day_gap['trans_amt_day_gap'] = merchant_every_day_gap['trans_amt'] - merchant_every_day_gap['trans_amt_before']
merchant_every_day_gap = merchant_every_day_gap[['UID', 'trans_amt_day_gap']]
merchant_every_day_gap = merchant_every_day_gap.groupby('UID')['trans_amt_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_day_gap.columns = ['UID', 'merchant_every_day_gap_sum', 'merchant_every_day_gap_mean', 'merchant_every_day_gap_max', 'merchant_every_day_gap_min', 'merchant_every_day_gap_std']
data = pd.merge(data, merchant_every_day_gap, on='UID', how='left')

# 用户merchant每天的rate
merchant_every_day_rate = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_day_rate = merchant_every_day_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_day_rate = merchant_every_day_rate[['UID', 'trans_amt']]
merchant_every_day_rate['trans_amt_before'] = merchant_every_day_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_day_rate['trans_amt_day_rate'] = merchant_every_day_rate['trans_amt'] / merchant_every_day_rate['trans_amt_before']
merchant_every_day_rate = merchant_every_day_rate[['UID', 'trans_amt_day_rate']]
merchant_every_day_rate = merchant_every_day_rate.groupby('UID')['trans_amt_day_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_day_rate, on='UID', how='left')


# 用户merchant每天半夜的统计量
merchant_every_morning = transaction_data[transaction_data.hour <= 5][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_morning = merchant_every_morning.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_morning = merchant_every_morning[['UID', 'trans_amt']]
merchant_every_morning = merchant_every_morning.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_morning.columns = ['UID', 'merchant_every_morning_std', 'merchant_every_morning_mean', 'merchant_every_morning_min', 'merchant_every_morning_max']
data = pd.merge(data, merchant_every_morning, on='UID', how='left')

# 用户merchant每天半夜的gap
merchant_every_morning_gap = transaction_data[transaction_data.hour <= 5][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_morning_gap = merchant_every_morning_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_morning_gap = merchant_every_morning_gap[['UID', 'trans_amt']]
merchant_every_morning_gap['trans_amt_before'] = merchant_every_morning_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_morning_gap['merchant_every_morning_gap'] = merchant_every_morning_gap['trans_amt'] - merchant_every_morning_gap['trans_amt_before']
merchant_every_morning_gap = merchant_every_morning_gap[['UID', 'merchant_every_morning_gap']]
merchant_every_morning_gap = merchant_every_morning_gap.groupby('UID')['merchant_every_morning_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_morning_gap.columns = ['UID', 'merchant_every_morning_gap_sum', 'merchant_every_morning_gap_std', 'merchant_every_morning_gap_mean', 'merchant_every_morning_gap_min', 'merchant_every_morning_gap_max']
data = pd.merge(data, merchant_every_morning_gap, on='UID', how='left')

# 用户merchant每天半夜的rate
merchant_every_morning_rate = transaction_data[transaction_data.hour <= 5][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_morning_rate = merchant_every_morning_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_morning_rate = merchant_every_morning_rate[['UID', 'trans_amt']]
merchant_every_morning_rate['trans_amt_before'] = merchant_every_morning_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_morning_rate['merchant_every_morning_rate'] = merchant_every_morning_rate['trans_amt'] / merchant_every_morning_rate['trans_amt_before']
merchant_every_morning_rate = merchant_every_morning_rate[['UID', 'merchant_every_morning_rate']]
merchant_every_morning_rate = merchant_every_morning_rate.groupby('UID')['merchant_every_morning_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_morning_rate, on='UID', how='left')


# 用户merchant每天早上的统计量
merchant_every_work_time = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_work_time = merchant_every_work_time.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_work_time = merchant_every_work_time[['UID', 'trans_amt']]
merchant_every_work_time = merchant_every_work_time.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_work_time.columns = ['UID', 'merchant_every_work_time_std', 'merchant_every_work_time_mean', 'merchant_every_work_time_min', 'merchant_every_work_time_max']
data = pd.merge(data, merchant_every_work_time, on='UID', how='left')

# 用户merchant每天早上的gap
merchant_every_work_time_gap = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_work_time_gap = merchant_every_work_time_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_work_time_gap = merchant_every_work_time_gap[['UID', 'trans_amt']]
merchant_every_work_time_gap['trans_amt_before'] = merchant_every_work_time_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_work_time_gap['merchant_every_work_time_gap'] = merchant_every_work_time_gap['trans_amt'] - merchant_every_work_time_gap['trans_amt_before']
merchant_every_work_time_gap = merchant_every_work_time_gap[['UID', 'merchant_every_work_time_gap']]
merchant_every_work_time_gap = merchant_every_work_time_gap.groupby('UID')['merchant_every_work_time_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_work_time_gap.columns = ['UID', 'merchant_every_work_time_gap_sum', 'merchant_every_work_time_gap_std', 'merchant_every_work_time_gap_mean', 'merchant_every_work_time_gap_min', 'merchant_every_work_time_gap_max']
data = pd.merge(data, merchant_every_work_time_gap, on='UID', how='left')

# 用户merchant每天早上的rate
merchant_every_work_time_rate = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_work_time_rate = merchant_every_work_time_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_work_time_rate = merchant_every_work_time_rate[['UID', 'trans_amt']]
merchant_every_work_time_rate['trans_amt_before'] = merchant_every_work_time_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_work_time_rate['merchant_every_work_time_rate'] = merchant_every_work_time_rate['trans_amt'] / merchant_every_work_time_rate['trans_amt_before']
merchant_every_work_time_rate = merchant_every_work_time_rate[['UID', 'merchant_every_work_time_rate']]
merchant_every_work_time_rate = merchant_every_work_time_rate.groupby('UID')['merchant_every_work_time_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_work_time_rate, on='UID', how='left')

# 用户merchant每天下午的统计量
merchant_every_afternoon = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_afternoon = merchant_every_afternoon.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_afternoon = merchant_every_afternoon[['UID', 'trans_amt']]
merchant_every_afternoon = merchant_every_afternoon.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_afternoon.columns = ['UID', 'merchant_every_afternoon_std', 'merchant_every_afternoon_mean', 'merchant_every_afternoon_min', 'merchant_every_afternoon_max']
data = pd.merge(data, merchant_every_afternoon, on='UID', how='left')

# 用户merchant每天下午的gap
merchant_every_afternoon_gap = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_afternoon_gap = merchant_every_afternoon_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_afternoon_gap = merchant_every_afternoon_gap[['UID', 'trans_amt']]
merchant_every_afternoon_gap['trans_amt_before'] = merchant_every_afternoon_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_afternoon_gap['merchant_every_afternoon_gap'] = merchant_every_afternoon_gap['trans_amt'] - merchant_every_afternoon_gap['trans_amt_before']
merchant_every_afternoon_gap = merchant_every_afternoon_gap[['UID', 'merchant_every_afternoon_gap']]
merchant_every_afternoon_gap = merchant_every_afternoon_gap.groupby('UID')['merchant_every_afternoon_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_afternoon_gap.columns = ['UID', 'merchant_every_afternoon_gap_sum', 'merchant_every_afternoon_gap_std', 'merchant_every_afternoon_gap_mean', 'merchant_every_afternoon_gap_min', 'merchant_every_afternoon_gap_max']
data = pd.merge(data, merchant_every_afternoon_gap, on='UID', how='left')

# 用户merchant每天下午的rate
merchant_every_afternoon_rate = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_afternoon_rate = merchant_every_afternoon_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_afternoon_rate = merchant_every_afternoon_rate[['UID', 'trans_amt']]
merchant_every_afternoon_rate['trans_amt_before'] = merchant_every_afternoon_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_afternoon_rate['merchant_every_afternoon_rate'] = merchant_every_afternoon_rate['trans_amt'] / merchant_every_afternoon_rate['trans_amt_before']
merchant_every_afternoon_rate = merchant_every_afternoon_rate[['UID', 'merchant_every_afternoon_rate']]
merchant_every_afternoon_rate = merchant_every_afternoon_rate.groupby('UID')['merchant_every_afternoon_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_afternoon_rate, on='UID', how='left')

# 用户merchant每天晚上的统计量
merchant_every_night = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_night = merchant_every_night.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_night = merchant_every_night[['UID', 'trans_amt']]
merchant_every_night = merchant_every_night.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_night.columns = ['UID', 'merchant_every_night_std', 'merchant_every_night_mean', 'merchant_every_night_min', 'merchant_every_night_max']
data = pd.merge(data, merchant_every_night, on='UID', how='left')

# 用户merchant每天晚上的gap
merchant_every_night_gap = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_night_gap = merchant_every_night_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_night_gap = merchant_every_night_gap[['UID', 'trans_amt']]
merchant_every_night_gap['trans_amt_before'] = merchant_every_night_gap.groupby('UID')['trans_amt'].shift(1)
merchant_every_night_gap['merchant_every_night_gap'] = merchant_every_night_gap['trans_amt'] - merchant_every_night_gap['trans_amt_before']
merchant_every_night_gap = merchant_every_night_gap[['UID', 'merchant_every_night_gap']]
merchant_every_night_gap = merchant_every_night_gap.groupby('UID')['merchant_every_night_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_night_gap.columns = ['UID', 'merchant_every_night_gap_sum', 'merchant_every_night_gap_std', 'merchant_every_night_gap_mean', 'merchant_every_night_gap_min', 'merchant_every_night_gap_max']
data = pd.merge(data, merchant_every_night_gap, on='UID', how='left')

# 用户merchant每天晚上的rate
merchant_every_night_rate = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_night_rate = merchant_every_night_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_night_rate = merchant_every_night_rate[['UID', 'trans_amt']]
merchant_every_night_rate['trans_amt_before'] = merchant_every_night_rate.groupby('UID')['trans_amt'].shift(1)
merchant_every_night_rate['merchant_every_night_rate'] = merchant_every_night_rate['trans_amt'] / merchant_every_night_rate['trans_amt_before']
merchant_every_night_rate = merchant_every_night_rate[['UID', 'merchant_every_night_rate']]
merchant_every_night_rate = merchant_every_night_rate.groupby('UID')['merchant_every_night_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_night_rate, on='UID', how='left')


# 用户merchant每三天的gap
merchant_every_three_day_gap = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_three_day_gap = merchant_every_three_day_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_three_day_gap = merchant_every_three_day_gap[['UID', 'trans_amt']]
merchant_every_three_day_gap['trans_amt_before'] = merchant_every_three_day_gap.groupby('UID')['trans_amt'].shift(3)
merchant_every_three_day_gap['trans_amt_three_day_gap'] = merchant_every_three_day_gap['trans_amt'] - merchant_every_three_day_gap['trans_amt_before']
merchant_every_three_day_gap = merchant_every_three_day_gap[['UID', 'trans_amt_three_day_gap']]
merchant_every_three_day_gap = merchant_every_three_day_gap.groupby('UID')['trans_amt_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_three_day_gap.columns = ['UID', 'merchant_every_three_day_gap_sum', 'merchant_every_three_day_gap_std', 'merchant_every_three_day_gap_mean', 'merchant_every_three_day_gap_min', 'merchant_every_three_day_gap_max']
data = pd.merge(data, merchant_every_three_day_gap, on='UID', how='left')

# 用户merchant每三天的rate
merchant_every_three_day_rate = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_three_day_rate = merchant_every_three_day_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_three_day_rate = merchant_every_three_day_rate[['UID', 'trans_amt']]
merchant_every_three_day_rate['trans_amt_before'] = merchant_every_three_day_rate.groupby('UID')['trans_amt'].shift(3)
merchant_every_three_day_rate['trans_amt_three_day_rate'] = merchant_every_three_day_rate['trans_amt'] / merchant_every_three_day_rate['trans_amt_before']
merchant_every_three_day_rate = merchant_every_three_day_rate[['UID', 'trans_amt_three_day_rate']]
merchant_every_three_day_rate = merchant_every_three_day_rate.groupby('UID')['trans_amt_three_day_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_three_day_rate, on='UID', how='left')


# 用户merchant每七天的gap
merchant_every_sixteen_day_gap = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap[['UID', 'trans_amt']]
merchant_every_sixteen_day_gap['trans_amt_before'] = merchant_every_sixteen_day_gap.groupby('UID')['trans_amt'].shift(7)
merchant_every_sixteen_day_gap['trans_amt_sixteen_day_gap'] = merchant_every_sixteen_day_gap['trans_amt'] - merchant_every_sixteen_day_gap['trans_amt_before']
merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap[['UID', 'trans_amt_sixteen_day_gap']]
merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap.groupby('UID')['trans_amt_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
merchant_every_sixteen_day_gap.columns = ['UID', 'merchant_every_sixteen_day_gap_sum', 'merchant_every_sixteen_day_gap_std', 'merchant_every_sixteen_day_gap_mean', 'merchant_every_sixteen_day_gap_min', 'merchant_every_sixteen_day_gap_max']
data = pd.merge(data, merchant_every_sixteen_day_gap, on='UID', how='left')

# 用户merchant每七天的rate
merchant_every_sixteen_day_rate = transaction_data[['UID', 'merchant', 'day', 'trans_amt']]
merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate.groupby(['UID', 'merchant', 'day'])['trans_amt'].agg('sum').reset_index()
merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate[['UID', 'trans_amt']]
merchant_every_sixteen_day_rate['trans_amt_before'] = merchant_every_sixteen_day_rate.groupby('UID')['trans_amt'].shift(7)
merchant_every_sixteen_day_rate['trans_amt_sixteen_day_rate'] = merchant_every_sixteen_day_rate['trans_amt'] / merchant_every_sixteen_day_rate['trans_amt_before']
merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate[['UID', 'trans_amt_sixteen_day_rate']]
merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate.groupby('UID')['trans_amt_sixteen_day_rate'].agg('sum').reset_index()
data = pd.merge(data, merchant_every_sixteen_day_rate, on='UID', how='left')

In [24]:
def frequence_device_time(data, temp, x, t):
    """
    label和时间交叉count
    """
    # 每秒设备的count
    second = temp[[x, 'day', 'time']]
    second[x + '_second_count'] = 1
    second = second.groupby([x, 'day', 'time']).agg('count').reset_index()
    second = pd.merge(temp[['UID', x, 'day', 'time']], second, on=[x, 'day', 'time'], how='left')
    second = second[['UID', x + '_second_count']]
    second = second.groupby('UID')[x + '_second_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    second.columns = ['UID', x + '_second_std', x + '_second_mean', x + '_second_min', x + '_second_max']
    data = pd.merge(data, second, on='UID', how='left')
    
    # 每分钟设备的count
    temp['minute'] = temp['time'].apply(lambda x: str(int(x.split(':')[0])) + str(int(x.split(':')[1])))
    minute = temp[[x, 'day', 'minute']]
    minute[x + '_minute_count'] = 1
    minute = minute.groupby([x, 'day', 'minute']).agg('count').reset_index()
    minute = pd.merge(temp[['UID', x, 'day', 'minute']], minute, on=[x, 'day', 'minute'], how='left')
    minute = minute[['UID', x + '_minute_count']]
    minute = minute.groupby('UID')[x + '_minute_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    minute.columns = ['UID', x + '_minute_std', x + '_minute_mean', x + '_minute_min', x + '_minute_max']
    data = pd.merge(data, minute, on='UID', how='left')
    
    # 每一天的每一小时的count
    hour = temp[[x, 'day', 'hour']]
    hour[x + '_hour_count'] = 1
    hour = hour.groupby([x, 'day', 'hour']).agg('count').reset_index()
    hour = pd.merge(temp[['UID', x, 'day', 'hour']], hour, on=[x, 'day', 'hour'], how='left')
    hour = hour[['UID', x + '_hour_count']]
    hour = hour.groupby('UID')[x + '_hour_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    hour.columns = ['UID', x + '_hour_std', x + '_hour_mean', x + '_hour_min', x + '_hour_max']
    data = pd.merge(data, hour, on='UID', how='left')
    
    # 每一天的每一小时的count_gap
    hour_gap = temp[[x, 'day', 'hour']]
    hour_gap[x + '_hour_gap_count'] = 1
    hour_gap = hour_gap.groupby([x, 'day', 'hour']).agg('count').reset_index()
    hour_gap['before_hour_count'] = hour_gap.groupby(x)[x + '_hour_gap_count'].shift(1)
    hour_gap[x + 'hour_count_gap'] = hour_gap[x + '_hour_gap_count'] - hour_gap['before_hour_count']
    hour_gap = hour_gap.drop([x + '_hour_gap_count', 'before_hour_count'], axis=1)
    hour_gap = pd.merge(temp[['UID', x, 'day', 'hour']], hour_gap, on=[x, 'day', 'hour'], how='left')
    hour_gap = hour_gap[['UID', x + 'hour_count_gap']]
    hour_gap = hour_gap.groupby('UID')[x + 'hour_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    hour_gap.columns = ['UID', x + '_hour_gap_sum', x + '_hour_gap_std', x + '_hour_gap_mean', x + '_hour_gap_min', x + '_hour_gap_max']
    data = pd.merge(data, hour_gap, on='UID', how='left')
    
    # 每一天的每一小时的count_rate
    hour_rate = temp[[x, 'day', 'hour']]
    hour_rate[x + '_hour_rate_count'] = 1
    hour_rate = hour_rate.groupby([x, 'day', 'hour']).agg('count').reset_index()
    hour_rate['before_hour_count'] = hour_rate.groupby(x)[x + '_hour_rate_count'].shift(1)
    hour_rate[x + 'hour_count_rate'] = hour_rate[x + '_hour_rate_count'] / hour_rate['before_hour_count']
    hour_rate = hour_rate.drop([x + '_hour_rate_count', 'before_hour_count'], axis=1)
    hour_rate = pd.merge(temp[['UID', x, 'day', 'hour']], hour_rate, on=[x, 'day', 'hour'], how='left')
    hour_rate = hour_rate[['UID', x + 'hour_count_rate']]
    hour_rate = hour_rate.groupby('UID')[x + 'hour_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, hour_rate, on='UID', how='left')
    
    # 每一天的count
    day = temp[[x, 'day']]
    day[x + '_day_count'] = 1
    day = day.groupby([x, 'day']).agg('count').reset_index()
    day = pd.merge(temp[['UID', x, 'day']], day, on=[x, 'day'], how='left')
    day = day[['UID', x + '_day_count']]
    day = day.groupby('UID')[x + '_day_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    day.columns = ['UID', x + '_day_std', x + '_day_mean', x + '_day_min', x + '_day_max']
    data = pd.merge(data, day, on='UID', how='left')
    
    # 每一天的count_gap
    day_gap = temp[[x, 'day']]
    day_gap[x + '_day_gap_count'] = 1
    day_gap = day_gap.groupby([x, 'day']).agg('count').reset_index()
    day_gap['before_day_count'] = day_gap.groupby(x)[x + '_day_gap_count'].shift(1)
    day_gap[x + 'day_count_gap'] = day_gap[x + '_day_gap_count'] - day_gap['before_day_count']
    day_gap = day_gap.drop([x + '_day_gap_count', 'before_day_count'], axis=1)
    day_gap = pd.merge(temp[['UID', x, 'day']], day_gap, on=[x, 'day'], how='left')
    day_gap = day_gap[['UID', x + 'day_count_gap']]
    day_gap = day_gap.groupby('UID')[x + 'day_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    day_gap.columns = ['UID', x + '_day_gap_sum', x + '_day_gap_std', x + '_day_gap_mean', x + '_day_gap_min', x + '_day_gap_max']
    data = pd.merge(data, day_gap, on='UID', how='left')
    
    # 每一天的count_rate
    day_rate = temp[[x, 'day']]
    day_rate[x + '_day_rate_count'] = 1
    day_rate = day_rate.groupby([x, 'day']).agg('count').reset_index()
    day_rate['before_day_count'] = day_rate.groupby(x)[x + '_day_rate_count'].shift(1)
    day_rate[x + 'day_count_rate'] = day_rate[x + '_day_rate_count'] / day_rate['before_day_count']
    day_rate = day_rate.drop([x + '_day_rate_count', 'before_day_count'], axis=1)
    day_rate = pd.merge(temp[['UID', x, 'day']], day_rate, on=[x, 'day'], how='left')
    day_rate = day_rate[['UID', x + 'day_count_rate']]
    day_rate = day_rate.groupby('UID')[x + 'day_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, day_rate, on='UID', how='left')
    
    # 每一天半夜的count
    morning = temp[temp.hour <= 5][[x, 'day']]
    morning[x + '_morning_count'] = 1
    morning = morning.groupby([x, 'day']).agg('count').reset_index()
    morning = pd.merge(temp[['UID', x, 'day']], morning, on=[x, 'day'], how='left')
    morning = morning[['UID', x + '_morning_count']]
    morning = morning.groupby('UID')[x + '_morning_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    morning.columns = ['UID', x + '_morning_std', x + '_morning_mean', x + '_morning_min', x + '_morning_max']
    data = pd.merge(data, morning, on='UID', how='left')
    
    # 每一天早上的count
    work_time = temp[(temp.hour >= 6)&(temp.hour <= 11)][[x, 'day']]
    work_time[x + '_work_time_count'] = 1
    work_time = work_time.groupby([x, 'day']).agg('count').reset_index()
    work_time = pd.merge(temp[['UID', x, 'day']], work_time, on=[x, 'day'], how='left')
    work_time = work_time[['UID', x + '_work_time_count']]
    work_time = work_time.groupby('UID')[x + '_work_time_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    work_time.columns = ['UID', x + '_work_time_std', x + '_work_time_mean', x + '_work_time_min', x + '_work_time_max']
    data = pd.merge(data, work_time, on='UID', how='left')
    
    # 每一天下午的count
    afternoon = temp[(temp.hour >= 12)&(temp.hour <= 17)][[x, 'day']]
    afternoon[x + '_afternoon_count'] = 1
    afternoon = afternoon.groupby([x, 'day']).agg('count').reset_index()
    afternoon = pd.merge(temp[['UID', x, 'day']], afternoon, on=[x, 'day'], how='left')
    afternoon = afternoon[['UID', x + '_afternoon_count']]
    afternoon = afternoon.groupby('UID')[x + '_afternoon_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    afternoon.columns = ['UID', x + '_afternoon_std', x + '_afternoon_mean', x + '_afternoon_min', x + '_afternoon_max']
    data = pd.merge(data, afternoon, on='UID', how='left')
    
    # 每一天晚上的count
    night = temp[(temp.hour >= 18)&(temp.hour <= 23)][[x, 'day']]
    night[x + '_night_count'] = 1
    night = night.groupby([x, 'day']).agg('count').reset_index()
    night = pd.merge(temp[['UID', x, 'day']], night, on=[x, 'day'], how='left')
    night = night[['UID', x + '_night_count']]
    night = night.groupby('UID')[x + '_night_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    night.columns = ['UID', x + '_night_std', x + '_night_mean', x + '_night_min', x + '_night_max']
    data = pd.merge(data, night, on='UID', how='left')
    
    # 每一天半夜的gap
    morning_gap = temp[temp.hour <= 5][[x, 'day']]
    morning_gap[x + '_morning_gap_count'] = 1
    morning_gap = morning_gap.groupby([x, 'day']).agg('count').reset_index()
    morning_gap = pd.merge(temp[['UID', x, 'day']], morning_gap, on=[x, 'day'], how='left')
    morning_gap[x + '_morning_gap_count_before'] = morning_gap.groupby('UID')[x + '_morning_gap_count'].shift(1)
    morning_gap[x + '_morning_gap_count_gap'] = morning_gap[x + '_morning_gap_count'] - morning_gap[x + '_morning_gap_count_before']
    morning_gap = morning_gap[['UID', x + '_morning_gap_count_gap']]
    morning_gap = morning_gap.groupby('UID')[x + '_morning_gap_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    morning_gap.columns = ['UID', x + '_morning_gap_sum', x + '_morning_gap_std', x + '_morning_gap_mean', x + '_morning_gap_min', x + '_morning_gap_max']
    data = pd.merge(data, morning_gap, on='UID', how='left')
    
    # 每一天早上的gap
    work_time_gap = temp[(temp.hour >= 6)&(temp.hour <= 11)][[x, 'day']]
    work_time_gap[x + '_work_time_gap_count'] = 1
    work_time_gap = work_time_gap.groupby([x, 'day']).agg('count').reset_index()
    work_time_gap = pd.merge(temp[['UID', x, 'day']], work_time_gap, on=[x, 'day'], how='left')
    work_time_gap[x + '_work_time_gap_count_before'] = work_time_gap.groupby('UID')[x + '_work_time_gap_count'].shift(1)
    work_time_gap[x + '_work_time_gap_count_gap'] = work_time_gap[x + '_work_time_gap_count'] - work_time_gap[x + '_work_time_gap_count_before']
    work_time_gap = work_time_gap[['UID', x + '_work_time_gap_count_gap']]
    work_time_gap = work_time_gap.groupby('UID')[x + '_work_time_gap_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    work_time_gap.columns = ['UID', x + '_work_time_gap_sum', x + '_work_time_gap_std', x + '_work_time_gap_mean', x + '_work_time_gap_min', x + '_work_time_gap_max']
    data = pd.merge(data, work_time_gap, on='UID', how='left')
    
    # 每一天下午的gap
    afternoon_gap = temp[(temp.hour >= 12)&(temp.hour <= 17)][[x, 'day']]
    afternoon_gap[x + '_afternoon_gap_count'] = 1
    afternoon_gap = afternoon_gap.groupby([x, 'day']).agg('count').reset_index()
    afternoon_gap = pd.merge(temp[['UID', x, 'day']], afternoon_gap, on=[x, 'day'], how='left')
    afternoon_gap[x + '_afternoon_gap_count_before'] = afternoon_gap.groupby('UID')[x + '_afternoon_gap_count'].shift(1)
    afternoon_gap[x + '_afternoon_gap_count_gap'] = afternoon_gap[x + '_afternoon_gap_count'] - afternoon_gap[x + '_afternoon_gap_count_before']
    afternoon_gap = afternoon_gap[['UID', x + '_afternoon_gap_count_gap']]
    afternoon_gap = afternoon_gap.groupby('UID')[x + '_afternoon_gap_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    afternoon_gap.columns = ['UID', x + '_afternoon_gap_sum', x + '_afternoon_gap_std', x + '_afternoon_gap_mean', x + '_afternoon_gap_min', x + '_afternoon_gap_max']
    data = pd.merge(data, afternoon_gap, on='UID', how='left')
    
    # 每一天晚上的gap
    night_gap = temp[(temp.hour >= 18)&(temp.hour <= 23)][[x, 'day']]
    night_gap[x + '_night_gap_count'] = 1
    night_gap = night_gap.groupby([x, 'day']).agg('count').reset_index()
    night_gap = pd.merge(temp[['UID', x, 'day']], night_gap, on=[x, 'day'], how='left')
    night_gap[x + '_night_gap_count_before'] = night_gap.groupby('UID')[x + '_night_gap_count'].shift(1)
    night_gap[x + '_night_gap_count_gap'] = night_gap[x + '_night_gap_count'] - night_gap[x + '_night_gap_count_before']
    night_gap = night_gap[['UID', x + '_night_gap_count_gap']]
    night_gap = night_gap.groupby('UID')[x + '_night_gap_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    night_gap.columns = ['UID', x + '_night_gap_sum', x + '_night_gap_std', x + '_night_gap_mean', x + '_night_gap_min', x + '_night_gap_max']
    data = pd.merge(data, night_gap, on='UID', how='left')
    
    
    # 每一天半夜的rate
    morning_rate = temp[temp.hour <= 5][[x, 'day']]
    morning_rate[x + '_morning_rate_count'] = 1
    morning_rate = morning_rate.groupby([x, 'day']).agg('count').reset_index()
    morning_rate = pd.merge(temp[['UID', x, 'day']], morning_rate, on=[x, 'day'], how='left')
    morning_rate[x + '_morning_rate_count_before'] = morning_rate.groupby('UID')[x + '_morning_rate_count'].shift(1)
    morning_rate[x + '_morning_rate_count_rate'] = morning_rate[x + '_morning_rate_count'] / morning_rate[x + '_morning_rate_count_before']
    morning_rate = morning_rate[['UID', x + '_morning_rate_count_rate']]
    morning_rate = morning_rate.groupby('UID')[x + '_morning_rate_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, morning_rate, on='UID', how='left')
    
    # 每一天早上的rate
    work_time_rate = temp[(temp.hour >= 6)&(temp.hour <= 11)][[x, 'day']]
    work_time_rate[x + '_work_time_rate_count'] = 1
    work_time_rate = work_time_rate.groupby([x, 'day']).agg('count').reset_index()
    work_time_rate = pd.merge(temp[['UID', x, 'day']], work_time_rate, on=[x, 'day'], how='left')
    work_time_rate[x + '_work_time_rate_count_before'] = work_time_rate.groupby('UID')[x + '_work_time_rate_count'].shift(1)
    work_time_rate[x + '_work_time_rate_count_rate'] = work_time_rate[x + '_work_time_rate_count'] / work_time_rate[x + '_work_time_rate_count_before']
    work_time_rate = work_time_rate[['UID', x + '_work_time_rate_count_rate']]
    work_time_rate = work_time_rate.groupby('UID')[x + '_work_time_rate_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, work_time_rate, on='UID', how='left')
    
    # 每一天下午的rate
    afternoon_rate = temp[(temp.hour >= 12)&(temp.hour <= 17)][[x, 'day']]
    afternoon_rate[x + '_afternoon_rate_count'] = 1
    afternoon_rate = afternoon_rate.groupby([x, 'day']).agg('count').reset_index()
    afternoon_rate = pd.merge(temp[['UID', x, 'day']], afternoon_rate, on=[x, 'day'], how='left')
    afternoon_rate[x + '_afternoon_rate_count_before'] = afternoon_rate.groupby('UID')[x + '_afternoon_rate_count'].shift(1)
    afternoon_rate[x + '_afternoon_rate_count_rate'] = afternoon_rate[x + '_afternoon_rate_count'] / afternoon_rate[x + '_afternoon_rate_count_before']
    afternoon_rate = afternoon_rate[['UID', x + '_afternoon_rate_count_rate']]
    afternoon_rate = afternoon_rate.groupby('UID')[x + '_afternoon_rate_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, afternoon_rate, on='UID', how='left')
    
    # 每一天晚上的rate
    night_rate = temp[(temp.hour >= 18)&(temp.hour <= 23)][[x, 'day']]
    night_rate[x + '_night_rate_count'] = 1
    night_rate = night_rate.groupby([x, 'day']).agg('count').reset_index()
    night_rate = pd.merge(temp[['UID', x, 'day']], night_rate, on=[x, 'day'], how='left')
    night_rate[x + '_night_rate_count_before'] = night_rate.groupby('UID')[x + '_night_rate_count'].shift(1)
    night_rate[x + '_night_rate_count_rate'] = night_rate[x + '_night_rate_count'] / night_rate[x + '_night_rate_count_before']
    night_rate = night_rate[['UID', x + '_night_rate_count_rate']]
    night_rate = night_rate.groupby('UID')[x + '_night_rate_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, night_rate, on='UID', how='left')
    
    
    # 每三天的gap
    three_day_gap = temp[[x, 'day']]
    three_day_gap[x + '_three_day_gap_count'] = 1
    three_day_gap = three_day_gap.groupby([x, 'day']).agg('count').reset_index()
    three_day_gap['before_three_day_count'] = three_day_gap.groupby(x)[x + '_three_day_gap_count'].shift(3)
    three_day_gap[x + 'three_day_count_gap'] = three_day_gap[x + '_three_day_gap_count'] - three_day_gap['before_three_day_count']
    three_day_gap = three_day_gap.drop([x + '_three_day_gap_count', 'before_three_day_count'], axis=1)
    three_day_gap = pd.merge(temp[['UID', x, 'day']], three_day_gap, on=[x, 'day'], how='left')
    three_day_gap = three_day_gap[['UID', x + 'three_day_count_gap']]
    three_day_gap = three_day_gap.groupby('UID')[x + 'three_day_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    three_day_gap.columns = ['UID', x + '_three_day_gap_sum', x + '_three_day_gap_std', x + '_three_day_gap_mean', x + '_three_day_gap_min', x + '_three_day_gap_max']
    data = pd.merge(data, three_day_gap, on='UID', how='left')
    
    # 每三天的count_rate
    three_day_rate = temp[[x, 'day']]
    three_day_rate[x + '_three_day_rate_count'] = 1
    three_day_rate = three_day_rate.groupby([x, 'day']).agg('count').reset_index()
    three_day_rate['before_three_day_count'] = three_day_rate.groupby(x)[x + '_three_day_rate_count'].shift(3)
    three_day_rate[x + 'three_day_count_rate'] = three_day_rate[x + '_three_day_rate_count'] / three_day_rate['before_three_day_count']
    three_day_rate = three_day_rate.drop([x + '_three_day_rate_count', 'before_three_day_count'], axis=1)
    three_day_rate = pd.merge(temp[['UID', x, 'day']], three_day_rate, on=[x, 'day'], how='left')
    three_day_rate = three_day_rate[['UID', x + 'three_day_count_rate']]
    three_day_rate = three_day_rate.groupby('UID')[x + 'three_day_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, three_day_rate, on='UID', how='left')
    
    
    # 每七天的gap
    sixteen_day_gap = temp[[x, 'day']]
    sixteen_day_gap[x + '_sixteen_day_gap_count'] = 1
    sixteen_day_gap = sixteen_day_gap.groupby([x, 'day']).agg('count').reset_index()
    sixteen_day_gap['before_sixteen_day_count'] = sixteen_day_gap.groupby(x)[x + '_sixteen_day_gap_count'].shift(7)
    sixteen_day_gap[x + 'sixteen_day_count_gap'] = sixteen_day_gap[x + '_sixteen_day_gap_count'] - sixteen_day_gap['before_sixteen_day_count']
    sixteen_day_gap = sixteen_day_gap.drop([x + '_sixteen_day_gap_count', 'before_sixteen_day_count'], axis=1)
    sixteen_day_gap = pd.merge(temp[['UID', x, 'day']], sixteen_day_gap, on=[x, 'day'], how='left')
    sixteen_day_gap = sixteen_day_gap[['UID', x + 'sixteen_day_count_gap']]
    sixteen_day_gap = sixteen_day_gap.groupby('UID')[x + 'sixteen_day_count_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    sixteen_day_gap.columns = ['UID', x + '_sixteen_day_gap_sum', x + '_sixteen_day_gap_std', x + '_sixteen_day_gap_mean', x + '_sixteen_day_gap_min', x + '_sixteen_day_gap_max']
    data = pd.merge(data, sixteen_day_gap, on='UID', how='left')
    
    # 每七天的count_rate
    sixteen_day_rate = temp[[x, 'day']]
    sixteen_day_rate[x + '_sixteen_day_rate_count'] = 1
    sixteen_day_rate = sixteen_day_rate.groupby([x, 'day']).agg('count').reset_index()
    sixteen_day_rate['before_sixteen_day_count'] = sixteen_day_rate.groupby(x)[x + '_sixteen_day_rate_count'].shift(7)
    sixteen_day_rate[x + '_sixteen_day_count_rate'] = sixteen_day_rate[x + '_sixteen_day_rate_count'] / sixteen_day_rate['before_sixteen_day_count']
    sixteen_day_rate = sixteen_day_rate.drop([x + '_sixteen_day_rate_count', 'before_sixteen_day_count'], axis=1)
    sixteen_day_rate = pd.merge(temp[['UID', x, 'day']], sixteen_day_rate, on=[x, 'day'], how='left')
    sixteen_day_rate = sixteen_day_rate[['UID', x + '_sixteen_day_count_rate']]
    sixteen_day_rate = sixteen_day_rate.groupby('UID')[x + '_sixteen_day_count_rate'].agg('sum').reset_index()
    data = pd.merge(data, sixteen_day_rate, on='UID', how='left')
    
    print(x + ' of ' + t, 'is over!')
    return data

#data = frequence_device_time(data, transaction_data, 'merchant', 'transaction_data')
# 原来是mac和ip_sub
for x in ['mac1', 'mac2', 'ip1_sub', 'ip2_sub']:
    data = frequence_device_time(data, operation_data, x, 'operation_data')

for y in ['merchant', 'mac1', 'ip1_sub', 'market_code']:
    data = frequence_device_time(data, transaction_data, y, 'transaction_data')

mac1 of operation_data is over!
mac2 of operation_data is over!
ip1_sub of operation_data is over!
ip2_sub of operation_data is over!
merchant of transaction_data is over!
mac1 of transaction_data is over!
ip1_sub of transaction_data is over!
market_code of transaction_data is over!


In [25]:
def amt_label_frequence(data, temp, x):
    # 每天的每一小时用该x(如ip)进行交易的钱数
    every_day_hour_amt = temp[[x, 'day', 'hour', 'trans_amt']]
    every_day_hour_amt['every_day_' + x + '_amt'] = every_day_hour_amt['trans_amt']
    every_day_hour_amt = every_day_hour_amt.groupby([x, 'day', 'hour'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_hour_amt = pd.merge(temp[['UID', x, 'day', 'hour']], every_day_hour_amt, on=[x, 'day', 'hour'], how='left')
    every_day_hour_amt = every_day_hour_amt[['UID', 'every_day_' + x + '_amt']]
    every_day_hour_amt = every_day_hour_amt.groupby('UID')['every_day_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_day_hour_amt.columns = ['UID', 'every_day_hour_amt_std_' + x, 'every_day_hour_amt_mean_' + x, 'every_day_hour_amt_min_' + x, 'every_day_hour_amt_max_' + x]
    data = pd.merge(data, every_day_hour_amt, on='UID', how='left')
    
    # 每天的每一小时用该x(如ip)进行交易的钱数的gap
    every_day_hour_amt_gap = temp[[x, 'day', 'hour', 'trans_amt']]
    every_day_hour_amt_gap['every_day_' + x + '_amt'] = every_day_hour_amt_gap['trans_amt']
    every_day_hour_amt_gap = every_day_hour_amt_gap.groupby([x, 'day', 'hour'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_hour_amt_gap = pd.merge(temp[['UID', x, 'day', 'hour']], every_day_hour_amt_gap, on=[x, 'day', 'hour'], how='left')
    every_day_hour_amt_gap = every_day_hour_amt_gap[['UID', 'every_day_' + x + '_amt']]
    every_day_hour_amt_gap['before_every_day_hour_amt'] = every_day_hour_amt_gap.groupby('UID')['every_day_' + x + '_amt'].shift(1)
    every_day_hour_amt_gap['every_day_hour_amt_gap_' + 'of_' + x] = every_day_hour_amt_gap['every_day_' + x + '_amt'] - every_day_hour_amt_gap['before_every_day_hour_amt']
    every_day_hour_amt_gap = every_day_hour_amt_gap[['UID', 'every_day_hour_amt_gap_' + 'of_' + x]]
    every_day_hour_amt_gap = every_day_hour_amt_gap.groupby('UID')['every_day_hour_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_day_hour_amt_gap.columns = ['UID', 'every_day_hour_amt_gap_sum_' + x, 'every_day_hour_amt_gap_std_' + x, 'every_day_hour_amt_gap_mean_' + x, 'every_day_hour_amt_gap_min_' + x, 'every_day_hour_amt_gap_max_' + x]
    data = pd.merge(data, every_day_hour_amt_gap, on='UID', how='left')
    
     # 每天的每一小时用该x(如ip)进行交易的钱数的rate
    every_day_hour_amt_rate = temp[[x, 'day', 'hour', 'trans_amt']]
    every_day_hour_amt_rate['every_day_' + x + '_amt'] = every_day_hour_amt_rate['trans_amt']
    every_day_hour_amt_rate = every_day_hour_amt_rate.groupby([x, 'day', 'hour'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_hour_amt_rate = pd.merge(temp[['UID', x, 'day', 'hour']], every_day_hour_amt_rate, on=[x, 'day', 'hour'], how='left')
    every_day_hour_amt_rate = every_day_hour_amt_rate[['UID', 'every_day_' + x + '_amt']]
    every_day_hour_amt_rate['before_every_day_hour_amt'] = every_day_hour_amt_rate.groupby('UID')['every_day_' + x + '_amt'].shift(1)
    every_day_hour_amt_rate['every_day_hour_amt_rate_' + 'of_' + x] = every_day_hour_amt_rate['every_day_' + x + '_amt'] / every_day_hour_amt_rate['before_every_day_hour_amt']
    every_day_hour_amt_rate = every_day_hour_amt_rate[['UID', 'every_day_hour_amt_rate_' + 'of_' + x]]
    every_day_hour_amt_rate = every_day_hour_amt_rate.groupby('UID')['every_day_hour_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_day_hour_amt_rate, on='UID', how='left')
    
    
    
    # 每天用该x(如ip)进行交易的钱数
    every_day_amt = temp[[x, 'day', 'trans_amt']]
    every_day_amt['every_day_' + x + '_amt'] = every_day_amt['trans_amt']
    every_day_amt = every_day_amt.groupby([x, 'day'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_amt = pd.merge(temp[['UID', x, 'day']], every_day_amt, on=[x, 'day'], how='left')
    every_day_amt = every_day_amt[['UID', 'every_day_' + x + '_amt']]
    every_day_amt = every_day_amt.groupby('UID')['every_day_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_day_amt.columns = ['UID', 'every_day_amt_std_' + x, 'every_day_amt_mean_' + x, 'every_day_amt_min_' + x, 'every_day_amt_max_' + x]
    data = pd.merge(data, every_day_amt, on='UID', how='left')

    # 每天用该x(如ip)进行交易的钱数的gap
    every_day_amt_gap = temp[[x, 'day', 'trans_amt']]
    every_day_amt_gap['every_day_' + x + '_amt'] = every_day_amt_gap['trans_amt']
    every_day_amt_gap = every_day_amt_gap.groupby([x, 'day'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_amt_gap = pd.merge(temp[['UID', x, 'day']], every_day_amt_gap, on=[x, 'day'], how='left')
    every_day_amt_gap = every_day_amt_gap[['UID', 'every_day_' + x + '_amt']]
    every_day_amt_gap['before_every_day_amt'] = every_day_amt_gap.groupby('UID')['every_day_' + x + '_amt'].shift(1)
    every_day_amt_gap['every_day_amt_gap_' + 'of_' + x] = every_day_amt_gap['every_day_' + x + '_amt'] - every_day_amt_gap['before_every_day_amt']
    every_day_amt_gap = every_day_amt_gap[['UID', 'every_day_amt_gap_' + 'of_' + x]]
    every_day_amt_gap = every_day_amt_gap.groupby('UID')['every_day_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_day_amt_gap.columns = ['UID', 'every_day_amt_gap_sum_' + x, 'every_day_amt_gap_std_' + x, 'every_day_amt_gap_mean_' + x, 'every_day_amt_gap_min_' + x, 'every_day_amt_gap_max_' + x]
    data = pd.merge(data, every_day_amt_gap, on='UID', how='left')

    # 每天用该x(如ip)进行交易的钱数的rate
    every_day_amt_rate = temp[[x, 'day', 'trans_amt']]
    every_day_amt_rate['every_day_' + x + '_amt'] = every_day_amt_rate['trans_amt']
    every_day_amt_rate = every_day_amt_rate.groupby([x, 'day'])['every_day_' + x + '_amt'].agg('sum').reset_index()
    every_day_amt_rate = pd.merge(temp[['UID', x, 'day']], every_day_amt_rate, on=[x, 'day'], how='left')
    every_day_amt_rate = every_day_amt_rate[['UID', 'every_day_' + x + '_amt']]
    every_day_amt_rate['before_every_day_amt'] = every_day_amt_rate.groupby('UID')['every_day_' + x + '_amt'].shift(1)
    every_day_amt_rate['every_day_amt_rate_' + 'of_' + x] = every_day_amt_rate['every_day_' + x + '_amt'] / every_day_amt_rate['before_every_day_amt']
    every_day_amt_rate = every_day_amt_rate[['UID', 'every_day_amt_rate_' + 'of_' + x]]
    every_day_amt_rate = every_day_amt_rate.groupby('UID')['every_day_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_day_amt_rate, on='UID', how='left')
    
    
    
    # 每天半夜用该x(如ip)进行交易的钱数
    every_morning_amt = temp[temp.hour <= 5][[x, 'day', 'trans_amt']]
    every_morning_amt['every_morning_' + x + '_amt'] = every_morning_amt['trans_amt']
    every_morning_amt = every_morning_amt.groupby([x, 'day'])['every_morning_' + x + '_amt'].agg('sum').reset_index()
    every_morning_amt = pd.merge(temp[['UID', x, 'day']], every_morning_amt, on=[x, 'day'], how='left')
    every_morning_amt = every_morning_amt[['UID', 'every_morning_' + x + '_amt']]
    every_morning_amt = every_morning_amt.groupby('UID')['every_morning_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_morning_amt.columns = ['UID', 'every_morning_amt_std_' + x, 'every_morning_amt_mean_' + x, 'every_morning_amt_min_' + x, 'every_morning_amt_max_' + x]
    data = pd.merge(data, every_morning_amt, on='UID', how='left')

    # 每天半夜用该x(如ip)进行交易的钱数的gap
    every_morning_amt_gap = temp[temp.hour <= 5][[x, 'day', 'trans_amt']]
    every_morning_amt_gap['every_morning_' + x + '_amt'] = every_morning_amt_gap['trans_amt']
    every_morning_amt_gap = every_morning_amt_gap.groupby([x, 'day'])['every_morning_' + x + '_amt'].agg('sum').reset_index()
    every_morning_amt_gap = pd.merge(temp[['UID', x, 'day']], every_morning_amt_gap, on=[x, 'day'], how='left')
    every_morning_amt_gap = every_morning_amt_gap[['UID', 'every_morning_' + x + '_amt']]
    every_morning_amt_gap['before_every_morning_amt'] = every_morning_amt_gap.groupby('UID')['every_morning_' + x + '_amt'].shift(1)
    every_morning_amt_gap['every_morning_amt_gap_' + 'of_' + x] = every_morning_amt_gap['every_morning_' + x + '_amt'] - every_morning_amt_gap['before_every_morning_amt']
    every_morning_amt_gap = every_morning_amt_gap[['UID', 'every_morning_amt_gap_' + 'of_' + x]]
    every_morning_amt_gap = every_morning_amt_gap.groupby('UID')['every_morning_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_morning_amt_gap.columns = ['UID', 'every_morning_amt_gap_sum_' + x, 'every_morning_amt_gap_std_' + x, 'every_morning_amt_gap_mean_' + x, 'every_morning_amt_gap_min_' + x, 'every_morning_amt_gap_max_' + x]
    data = pd.merge(data, every_morning_amt_gap, on='UID', how='left')

     # 每天半夜用该x(如ip)进行交易的钱数的rate
    every_morning_amt_rate = temp[temp.hour <= 5][[x, 'day', 'trans_amt']]
    every_morning_amt_rate['every_morning_' + x + '_amt'] = every_morning_amt_rate['trans_amt']
    every_morning_amt_rate = every_morning_amt_rate.groupby([x, 'day'])['every_morning_' + x + '_amt'].agg('sum').reset_index()
    every_morning_amt_rate = pd.merge(temp[['UID', x, 'day']], every_morning_amt_rate, on=[x, 'day'], how='left')
    every_morning_amt_rate = every_morning_amt_rate[['UID', 'every_morning_' + x + '_amt']]
    every_morning_amt_rate['before_every_morning_amt'] = every_morning_amt_rate.groupby('UID')['every_morning_' + x + '_amt'].shift(1)
    every_morning_amt_rate['every_morning_amt_rate_' + 'of_' + x] = every_morning_amt_rate['every_morning_' + x + '_amt'] / every_morning_amt_rate['before_every_morning_amt']
    every_morning_amt_rate = every_morning_amt_rate[['UID', 'every_morning_amt_rate_' + 'of_' + x]]
    every_morning_amt_rate = every_morning_amt_rate.groupby('UID')['every_morning_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_morning_amt_rate, on='UID', how='left')
    
    
    # 每天早上用该x(如ip)进行交易的钱数
    every_work_time_amt = temp[(temp.hour <= 11)&(temp.hour >= 6)][[x, 'day', 'trans_amt']]
    every_work_time_amt['every_work_time_' + x + '_amt'] = every_work_time_amt['trans_amt']
    every_work_time_amt = every_work_time_amt.groupby([x, 'day'])['every_work_time_' + x + '_amt'].agg('sum').reset_index()
    every_work_time_amt = pd.merge(temp[['UID', x, 'day']], every_work_time_amt, on=[x, 'day'], how='left')
    every_work_time_amt = every_work_time_amt[['UID', 'every_work_time_' + x + '_amt']]
    every_work_time_amt = every_work_time_amt.groupby('UID')['every_work_time_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_work_time_amt.columns = ['UID', 'every_work_time_amt_std_' + x, 'every_work_time_amt_mean_' + x, 'every_work_time_amt_min_' + x, 'every_work_time_amt_max_' + x]
    data = pd.merge(data, every_work_time_amt, on='UID', how='left')

    # 每天早上用该x(如ip)进行交易的钱数的gap
    every_work_time_amt_gap = temp[(temp.hour <= 11)&(temp.hour >= 6)][[x, 'day', 'trans_amt']]
    every_work_time_amt_gap['every_work_time_' + x + '_amt'] = every_work_time_amt_gap['trans_amt']
    every_work_time_amt_gap = every_work_time_amt_gap.groupby([x, 'day'])['every_work_time_' + x + '_amt'].agg('sum').reset_index()
    every_work_time_amt_gap = pd.merge(temp[['UID', x, 'day']], every_work_time_amt_gap, on=[x, 'day'], how='left')
    every_work_time_amt_gap = every_work_time_amt_gap[['UID', 'every_work_time_' + x + '_amt']]
    every_work_time_amt_gap['before_every_work_time_amt'] = every_work_time_amt_gap.groupby('UID')['every_work_time_' + x + '_amt'].shift(1)
    every_work_time_amt_gap['every_work_time_amt_gap_' + 'of_' + x] = every_work_time_amt_gap['every_work_time_' + x + '_amt'] - every_work_time_amt_gap['before_every_work_time_amt']
    every_work_time_amt_gap = every_work_time_amt_gap[['UID', 'every_work_time_amt_gap_' + 'of_' + x]]
    every_work_time_amt_gap = every_work_time_amt_gap.groupby('UID')['every_work_time_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_work_time_amt_gap.columns = ['UID', 'every_work_time_amt_gap_sum_' + x, 'every_work_time_amt_gap_std_' + x, 'every_work_time_amt_gap_mean_' + x, 'every_work_time_amt_gap_min_' + x, 'every_work_time_amt_gap_max_' + x]
    data = pd.merge(data, every_work_time_amt_gap, on='UID', how='left')

     # 每天早上用该x(如ip)进行交易的钱数的rate
    every_work_time_amt_rate = temp[(temp.hour <= 11)&(temp.hour >= 6)][[x, 'day', 'trans_amt']]
    every_work_time_amt_rate['every_work_time_' + x + '_amt'] = every_work_time_amt_rate['trans_amt']
    every_work_time_amt_rate = every_work_time_amt_rate.groupby([x, 'day'])['every_work_time_' + x + '_amt'].agg('sum').reset_index()
    every_work_time_amt_rate = pd.merge(temp[['UID', x, 'day']], every_work_time_amt_rate, on=[x, 'day'], how='left')
    every_work_time_amt_rate = every_work_time_amt_rate[['UID', 'every_work_time_' + x + '_amt']]
    every_work_time_amt_rate['before_every_work_time_amt'] = every_work_time_amt_rate.groupby('UID')['every_work_time_' + x + '_amt'].shift(1)
    every_work_time_amt_rate['every_work_time_amt_rate_' + 'of_' + x] = every_work_time_amt_rate['every_work_time_' + x + '_amt'] / every_work_time_amt_rate['before_every_work_time_amt']
    every_work_time_amt_rate = every_work_time_amt_rate[['UID', 'every_work_time_amt_rate_' + 'of_' + x]]
    every_work_time_amt_rate = every_work_time_amt_rate.groupby('UID')['every_work_time_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_work_time_amt_rate, on='UID', how='left')
    
    # 每天下午用该x(如ip)进行交易的钱数
    every_afternoon_amt = temp[(temp.hour <= 17)&(temp.hour >= 12)][[x, 'day', 'trans_amt']]
    every_afternoon_amt['every_afternoon_' + x + '_amt'] = every_afternoon_amt['trans_amt']
    every_afternoon_amt = every_afternoon_amt.groupby([x, 'day'])['every_afternoon_' + x + '_amt'].agg('sum').reset_index()
    every_afternoon_amt = pd.merge(temp[['UID', x, 'day']], every_afternoon_amt, on=[x, 'day'], how='left')
    every_afternoon_amt = every_afternoon_amt[['UID', 'every_afternoon_' + x + '_amt']]
    every_afternoon_amt = every_afternoon_amt.groupby('UID')['every_afternoon_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_afternoon_amt.columns = ['UID', 'every_afternoon_amt_std_' + x, 'every_afternoon_amt_mean_' + x, 'every_afternoon_amt_min_' + x, 'every_afternoon_amt_max_' + x]
    data = pd.merge(data, every_afternoon_amt, on='UID', how='left')

    # 每天下午用该x(如ip)进行交易的钱数的gap
    every_afternoon_amt_gap = temp[(temp.hour <= 17)&(temp.hour >= 12)][[x, 'day', 'trans_amt']]
    every_afternoon_amt_gap['every_afternoon_' + x + '_amt'] = every_afternoon_amt_gap['trans_amt']
    every_afternoon_amt_gap = every_afternoon_amt_gap.groupby([x, 'day'])['every_afternoon_' + x + '_amt'].agg('sum').reset_index()
    every_afternoon_amt_gap = pd.merge(temp[['UID', x, 'day']], every_afternoon_amt_gap, on=[x, 'day'], how='left')
    every_afternoon_amt_gap = every_afternoon_amt_gap[['UID', 'every_afternoon_' + x + '_amt']]
    every_afternoon_amt_gap['before_every_afternoon_amt'] = every_afternoon_amt_gap.groupby('UID')['every_afternoon_' + x + '_amt'].shift(1)
    every_afternoon_amt_gap['every_afternoon_amt_gap_' + 'of_' + x] = every_afternoon_amt_gap['every_afternoon_' + x + '_amt'] - every_afternoon_amt_gap['before_every_afternoon_amt']
    every_afternoon_amt_gap = every_afternoon_amt_gap[['UID', 'every_afternoon_amt_gap_' + 'of_' + x]]
    every_afternoon_amt_gap = every_afternoon_amt_gap.groupby('UID')['every_afternoon_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_afternoon_amt_gap.columns = ['UID', 'every_afternoon_amt_gap_sum_' + x, 'every_afternoon_amt_gap_std_' + x, 'every_afternoon_amt_gap_mean_' + x, 'every_afternoon_amt_gap_min_' + x, 'every_afternoon_amt_gap_max_' + x]
    data = pd.merge(data, every_afternoon_amt_gap, on='UID', how='left')

     # 每天下午用该x(如ip)进行交易的钱数的rate
    every_afternoon_amt_rate = temp[(temp.hour <= 17)&(temp.hour >= 12)][[x, 'day', 'trans_amt']]
    every_afternoon_amt_rate['every_afternoon_' + x + '_amt'] = every_afternoon_amt_rate['trans_amt']
    every_afternoon_amt_rate = every_afternoon_amt_rate.groupby([x, 'day'])['every_afternoon_' + x + '_amt'].agg('sum').reset_index()
    every_afternoon_amt_rate = pd.merge(temp[['UID', x, 'day']], every_afternoon_amt_rate, on=[x, 'day'], how='left')
    every_afternoon_amt_rate = every_afternoon_amt_rate[['UID', 'every_afternoon_' + x + '_amt']]
    every_afternoon_amt_rate['before_every_afternoon_amt'] = every_afternoon_amt_rate.groupby('UID')['every_afternoon_' + x + '_amt'].shift(1)
    every_afternoon_amt_rate['every_afternoon_amt_rate_' + 'of_' + x] = every_afternoon_amt_rate['every_afternoon_' + x + '_amt'] / every_afternoon_amt_rate['before_every_afternoon_amt']
    every_afternoon_amt_rate = every_afternoon_amt_rate[['UID', 'every_afternoon_amt_rate_' + 'of_' + x]]
    every_afternoon_amt_rate = every_afternoon_amt_rate.groupby('UID')['every_afternoon_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_afternoon_amt_rate, on='UID', how='left')
    
    # 每天晚上用该x(如ip)进行交易的钱数
    every_night_amt = temp[(temp.hour <= 23)&(temp.hour >= 18)][[x, 'day', 'trans_amt']]
    every_night_amt['every_night_' + x + '_amt'] = every_night_amt['trans_amt']
    every_night_amt = every_night_amt.groupby([x, 'day'])['every_night_' + x + '_amt'].agg('sum').reset_index()
    every_night_amt = pd.merge(temp[['UID', x, 'day']], every_night_amt, on=[x, 'day'], how='left')
    every_night_amt = every_night_amt[['UID', 'every_night_' + x + '_amt']]
    every_night_amt = every_night_amt.groupby('UID')['every_night_' + x + '_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    every_night_amt.columns = ['UID', 'every_night_amt_std_' + x, 'every_night_amt_mean_' + x, 'every_night_amt_min_' + x, 'every_night_amt_max_' + x]
    data = pd.merge(data, every_night_amt, on='UID', how='left')

    # 每天晚上用该x(如ip)进行交易的钱数的gap
    every_night_amt_gap = temp[(temp.hour <= 23)&(temp.hour >= 18)][[x, 'day', 'trans_amt']]
    every_night_amt_gap['every_night_' + x + '_amt'] = every_night_amt_gap['trans_amt']
    every_night_amt_gap = every_night_amt_gap.groupby([x, 'day'])['every_night_' + x + '_amt'].agg('sum').reset_index()
    every_night_amt_gap = pd.merge(temp[['UID', x, 'day']], every_night_amt_gap, on=[x, 'day'], how='left')
    every_night_amt_gap = every_night_amt_gap[['UID', 'every_night_' + x + '_amt']]
    every_night_amt_gap['before_every_night_amt'] = every_night_amt_gap.groupby('UID')['every_night_' + x + '_amt'].shift(1)
    every_night_amt_gap['every_night_amt_gap_' + 'of_' + x] = every_night_amt_gap['every_night_' + x + '_amt'] - every_night_amt_gap['before_every_night_amt']
    every_night_amt_gap = every_night_amt_gap[['UID', 'every_night_amt_gap_' + 'of_' + x]]
    every_night_amt_gap = every_night_amt_gap.groupby('UID')['every_night_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_night_amt_gap.columns = ['UID', 'every_night_amt_gap_sum_' + x, 'every_night_amt_gap_std_' + x, 'every_night_amt_gap_mean_' + x, 'every_night_amt_gap_min_' + x, 'every_night_amt_gap_max_' + x]
    data = pd.merge(data, every_night_amt_gap, on='UID', how='left')

     # 每天晚上用该x(如ip)进行交易的钱数的rate
    every_night_amt_rate = temp[(temp.hour <= 23)&(temp.hour >= 18)][[x, 'day', 'trans_amt']]
    every_night_amt_rate['every_night_' + x + '_amt'] = every_night_amt_rate['trans_amt']
    every_night_amt_rate = every_night_amt_rate.groupby([x, 'day'])['every_night_' + x + '_amt'].agg('sum').reset_index()
    every_night_amt_rate = pd.merge(temp[['UID', x, 'day']], every_night_amt_rate, on=[x, 'day'], how='left')
    every_night_amt_rate = every_night_amt_rate[['UID', 'every_night_' + x + '_amt']]
    every_night_amt_rate['before_every_night_amt'] = every_night_amt_rate.groupby('UID')['every_night_' + x + '_amt'].shift(1)
    every_night_amt_rate['every_night_amt_rate_' + 'of_' + x] = every_night_amt_rate['every_night_' + x + '_amt'] / every_night_amt_rate['before_every_night_amt']
    every_night_amt_rate = every_night_amt_rate[['UID', 'every_night_amt_rate_' + 'of_' + x]]
    every_night_amt_rate = every_night_amt_rate.groupby('UID')['every_night_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_night_amt_rate, on='UID', how='left')
    
    # 每三天x与钱的gap
    every_three_day_amt_gap = temp[[x, 'day', 'trans_amt']]
    every_three_day_amt_gap['every_three_day_' + x + '_amt'] = every_three_day_amt_gap['trans_amt']
    every_three_day_amt_gap = every_three_day_amt_gap.groupby([x, 'day'])['every_three_day_' + x + '_amt'].agg('sum').reset_index()
    every_three_day_amt_gap = pd.merge(temp[['UID', x, 'day']], every_three_day_amt_gap, on=[x, 'day'], how='left')
    every_three_day_amt_gap = every_three_day_amt_gap[['UID', 'every_three_day_' + x + '_amt']]
    every_three_day_amt_gap['before_every_three_day_amt'] = every_three_day_amt_gap.groupby('UID')['every_three_day_' + x + '_amt'].shift(3)
    every_three_day_amt_gap['every_three_day_amt_gap_' + 'of_' + x] = every_three_day_amt_gap['every_three_day_' + x + '_amt'] - every_three_day_amt_gap['before_every_three_day_amt']
    every_three_day_amt_gap = every_three_day_amt_gap[['UID', 'every_three_day_amt_gap_' + 'of_' + x]]
    every_three_day_amt_gap = every_three_day_amt_gap.groupby('UID')['every_three_day_amt_gap_' + 'of_' + x].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    every_three_day_amt_gap.columns = ['UID', 'every_three_day_amt_gap_sum_' + x, 'every_three_day_amt_gap_std_' + x, 'every_three_day_amt_gap_mean_' + x, 'every_three_day_amt_gap_min_' + x, 'every_three_day_amt_gap_max_' + x]
    data = pd.merge(data, every_three_day_amt_gap, on='UID', how='left')
    
    every_three_day_amt_rate = temp[[x, 'day', 'trans_amt']]
    every_three_day_amt_rate['every_three_day_' + x + '_amt'] = every_three_day_amt_rate['trans_amt']
    every_three_day_amt_rate = every_three_day_amt_rate.groupby([x, 'day'])['every_three_day_' + x + '_amt'].agg('sum').reset_index()
    every_three_day_amt_rate = pd.merge(temp[['UID', x, 'day']], every_three_day_amt_rate, on=[x, 'day'], how='left')
    every_three_day_amt_rate = every_three_day_amt_rate[['UID', 'every_three_day_' + x + '_amt']]
    every_three_day_amt_rate['before_every_three_day_amt'] = every_three_day_amt_rate.groupby('UID')['every_three_day_' + x + '_amt'].shift(3)
    every_three_day_amt_rate['every_three_day_amt_rate_' + 'of_' + x] = every_three_day_amt_rate['every_three_day_' + x + '_amt'] / every_three_day_amt_rate['before_every_three_day_amt']
    every_three_day_amt_rate = every_three_day_amt_rate[['UID', 'every_three_day_amt_rate_' + 'of_' + x]]
    every_three_day_amt_rate = every_three_day_amt_rate.groupby('UID')['every_three_day_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_three_day_amt_rate, on='UID', how='left')
    
    # 每七天x与钱的gap
    every_sixteen_day_amt_gap = temp[[x, 'day', 'trans_amt']]
    every_sixteen_day_amt_gap['every_sixteen_day_' + x + '_amt'] = every_sixteen_day_amt_gap['trans_amt']
    every_sixteen_day_amt_gap = every_sixteen_day_amt_gap.groupby([x, 'day'])['every_sixteen_day_' + x + '_amt'].agg('sum').reset_index()
    every_sixteen_day_amt_gap = pd.merge(temp[['UID', x, 'day']], every_sixteen_day_amt_gap, on=[x, 'day'], how='left')
    every_sixteen_day_amt_gap = every_sixteen_day_amt_gap[['UID', 'every_sixteen_day_' + x + '_amt']]
    every_sixteen_day_amt_gap['before_every_sixteen_day_amt'] = every_sixteen_day_amt_gap.groupby('UID')['every_sixteen_day_' + x + '_amt'].shift(7)
    every_sixteen_day_amt_gap['every_sixteen_day_amt_gap_' + 'of_' + x] = every_sixteen_day_amt_gap['every_sixteen_day_' + x + '_amt'] - every_sixteen_day_amt_gap['before_every_sixteen_day_amt']
    every_sixteen_day_amt_gap = every_sixteen_day_amt_gap[['UID', 'every_sixteen_day_amt_gap_' + 'of_' + x]]
    every_sixteen_day_amt_gap = every_sixteen_day_amt_gap.groupby('UID')['every_sixteen_day_amt_gap_' + 'of_' + x].agg({'sum' ,'mean', 'max', 'min', 'std'}).reset_index()
    every_sixteen_day_amt_gap.columns = ['UID', 'every_sixteen_day_amt_gap_sum_' + x, 'every_sixteen_day_amt_gap_std_' + x, 'every_sixteen_day_amt_gap_mean_' + x, 'every_sixteen_day_amt_gap_min_' + x, 'every_sixteen_day_amt_gap_max_' + x]
    data = pd.merge(data, every_sixteen_day_amt_gap, on='UID', how='left')

    every_sixteen_day_amt_rate = temp[[x, 'day', 'trans_amt']]
    every_sixteen_day_amt_rate['every_sixteen_day_' + x + '_amt'] = every_sixteen_day_amt_rate['trans_amt']
    every_sixteen_day_amt_rate = every_sixteen_day_amt_rate.groupby([x, 'day'])['every_sixteen_day_' + x + '_amt'].agg('sum').reset_index()
    every_sixteen_day_amt_rate = pd.merge(temp[['UID', x, 'day']], every_sixteen_day_amt_rate, on=[x, 'day'], how='left')
    every_sixteen_day_amt_rate = every_sixteen_day_amt_rate[['UID', 'every_sixteen_day_' + x + '_amt']]
    every_sixteen_day_amt_rate['before_every_sixteen_day_amt'] = every_sixteen_day_amt_rate.groupby('UID')['every_sixteen_day_' + x + '_amt'].shift(7)
    every_sixteen_day_amt_rate['every_sixteen_day_amt_rate_' + 'of_' + x] = every_sixteen_day_amt_rate['every_sixteen_day_' + x + '_amt'] / every_sixteen_day_amt_rate['before_every_sixteen_day_amt']
    every_sixteen_day_amt_rate = every_sixteen_day_amt_rate[['UID', 'every_sixteen_day_amt_rate_' + 'of_' + x]]
    every_sixteen_day_amt_rate = every_sixteen_day_amt_rate.groupby('UID')['every_sixteen_day_amt_rate_' + 'of_' + x].agg('sum').reset_index()
    data = pd.merge(data, every_sixteen_day_amt_rate, on='UID', how='left')
    
    print(x + ' is over!')
    return data

for x in ['merchant', 'mac1', 'ip1_sub', 'market_code']:
    data = amt_label_frequence(data, transaction_data, x)

merchant is over!
mac1 is over!
ip1_sub is over!
market_code is over!


In [26]:
data['UID_'] = data['UID']

In [27]:
# 每一个用户的交易总金额
amt = transaction_data[['UID', 'trans_amt']]
amt['trans_amt_sum'] = amt['trans_amt']
amt = amt.groupby('UID')['trans_amt_sum'].agg('sum').reset_index()
data = pd.merge(data, amt, on='UID', how='left')

In [28]:
def get_label_uid_own(data, operation_or_transaction_data, label, t):
    label_count = operation_or_transaction_data[['UID', label]]
    operation_or_transaction_data = label_count.copy()
    operation_or_transaction_data.drop_duplicates(inplace=True)
    
    label_count[label + '_uid_count_' + t] = label_count['UID']
    label_count = label_count.groupby([label])[label + '_uid_count_' + t].agg('nunique').reset_index()# 好气啊！=_=
    
    operation_or_transaction_data = pd.merge(operation_or_transaction_data, label_count, on=label, how='left')
    operation_or_transaction_data[label + '_count_uid_' + t] = 1
    operation_or_transaction_data = operation_or_transaction_data.groupby('UID')[label + '_uid_count_' + t].agg('sum').reset_index()
    
    data = pd.merge(data, operation_or_transaction_data, on='UID', how='left')
    
    return data

# 每一个用户uid的每一种label种类数
for x in ['day', 'success', 'device_code1', 'device_code2', 'mac1', 'ip1', 'ip2', 'device_code3', 'mac2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub']:
    data = get_label_uid_own(data, operation_data, x, 'operation')
for x in ['day', 'merchant', 'code1', 'code2', 'acc_id1', 'device_code1',  'device_code2', 'device_code3', 'mac1', 'ip1', 'acc_id2', 'acc_id3', 'geo_code', 'ip1_sub', 'trans_amt']:
    data = get_label_uid_own(data, transaction_data, x, 'transaction')

In [29]:
def get_same_label(data, operation_data, transaction_data):
    not_need = ['time', 'UID', 'hour']
    operation = [x for x in operation_data if x not in not_need]
    transaction = [x for x in transaction_data if x not in not_need]
    
    same_label = list(set(operation).intersection(set(transaction)))
    return same_label
same_label = get_same_label(data, operation_data, transaction_data)
same_label.remove('minute')
same_label

['device2',
 'ip1_sub',
 'device_code2',
 'ip1',
 'geo_code',
 'device_code3',
 'day',
 'device_code1',
 'mac1',
 'device1']

In [30]:
def get_uid_operation_transaction_different(data, operation_data, transaction_data, same_label):
    label = same_label.copy()
    label.append('UID')
    
    operation = operation_data[label]
    transaction = transaction_data[label]

    temp = pd.concat([operation, transaction], axis=0, ignore_index=True)# 先堆在一起节约运行时间
    
    for x in same_label:
        t = temp[['UID', x]]
        t['same'] = t[x]
        t = t.groupby('UID')['same'].agg('nunique').reset_index()
        
        t1 = operation[['UID', x]]
        t1['operation'] = t1[x]
        t1 = t1.groupby('UID')['operation'].agg('nunique').reset_index()

        t2 = transaction[['UID', x]]
        t2['transaction'] = t2[x]
        t2 = t2.groupby('UID')['transaction'].agg('nunique').reset_index()

        t = pd.merge(t, t1, on='UID', how='left')
        t = pd.merge(t, t2, on='UID', how='left')
        t = t.fillna(0)
        t['uid_operation_transaction_different_of_' + x] = t['operation'] + t['transaction'] - t['same']
        t = t.drop(['operation', 'transaction', 'same'], axis=1)
        data = pd.merge(data, t, on='UID', how='left')
        print(x + ' over!')
        
    return data

# 统计每一个用户uid操作和交易的每一种label不相同的数量
data = get_uid_operation_transaction_different(data, operation_data, transaction_data, same_label)

device2 over!
ip1_sub over!
device_code2 over!
ip1 over!
geo_code over!
device_code3 over!
day over!
device_code1 over!
mac1 over!
device1 over!


In [31]:
def get_is_null_ratio(data, t_data, x, t):
    """
    对每一个UID的每一个设备类型，计算nan值的count
    """
    temp = t_data[t_data[x].isnull()][['UID']]
    temp[x + '_null_' + t] = 1
    temp = temp.groupby('UID').agg('count').reset_index()
    temp = temp[['UID', x + '_null_' + t]]
    data = pd.merge(data, temp, on='UID', how='left')
    return data

o = ['success', 'version', 'device1', 'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1', 'mac2', 'ip1', 'ip2', 'geo_code', 'ip1_sub', 'ip2_sub']
t = ['code1', 'code2', 'acc_id1', 'device_code1', 'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1', 'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2', 'market_code', 'market_type', 'ip1_sub']
for x in o:
    data = get_is_null_ratio(data, operation_data, x, 'operation')
for y in t:
    data = get_is_null_ratio(data, transaction_data, y, 'transaction')

In [32]:
"""
支付方式
trans_type1是真正的方式，trans_type2是线上线下这种
"""
# 每一天的每一小时用户支付方式的支付次数
trans_type1_count = transaction_data[['UID', 'day', 'hour', 'trans_type1']]
trans_type1_count['every_hour_trans_type1_count'] = 1
trans_type1_count = trans_type1_count.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
trans_type1_count = trans_type1_count[['UID', 'every_hour_trans_type1_count']]
trans_type1_count = trans_type1_count.groupby('UID')['every_hour_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_count.columns = ['UID', 'trans_type1_count_mean', 'trans_type1_count_min', 'trans_type1_count_std', 'trans_type1_count_max']
data = pd.merge(data, trans_type1_count, on='UID', how='left')

# 每一天每一小时用户在该ip用该支付方式支付次数
trans_type1_ip1_count = transaction_data[['UID', 'day', 'hour', 'ip1_sub', 'trans_type1']]
trans_type1_ip1_count['every_hour_ip1_sub_trans_type1_count'] = 1
trans_type1_ip1_count = trans_type1_ip1_count.groupby(['UID', 'day', 'hour', 'ip1_sub', 'trans_type1']).agg('count').reset_index()
trans_type1_ip1_count = trans_type1_ip1_count[['UID', 'every_hour_ip1_sub_trans_type1_count']]
trans_type1_ip1_count = trans_type1_ip1_count.groupby('UID')['every_hour_ip1_sub_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_ip1_count.columns = ['UID', 'trans_type1_ip1_count_mean', 'trans_type1_ip1_count_min', 'trans_type1_ip1_count_std', 'trans_type1_ip1_count_max']
data = pd.merge(data, trans_type1_ip1_count, on='UID', how='left')

# 每一天每一小时用户在该ip用该支付方式支付次数
trans_type1_mac1_count = transaction_data[['UID', 'day', 'hour', 'mac1', 'trans_type1']]
trans_type1_mac1_count['every_hour_mac1_trans_type1_count'] = 1
trans_type1_mac1_count = trans_type1_mac1_count.groupby(['UID', 'day', 'hour', 'mac1', 'trans_type1']).agg('count').reset_index()
trans_type1_mac1_count = trans_type1_mac1_count[['UID', 'every_hour_mac1_trans_type1_count']]
trans_type1_mac1_count = trans_type1_mac1_count.groupby('UID')['every_hour_mac1_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_mac1_count.columns = ['UID', 'trans_type1_mac1_count_mean', 'trans_type1_mac1_count_min', 'trans_type1_mac1_count_std', 'trans_type1_mac1_count_max']
data = pd.merge(data, trans_type1_mac1_count, on='UID', how='left')

# 每一天每一小时用户用device1用该支付方式支付次数
trans_type_device_code1_count = transaction_data[['UID', 'day', 'hour', 'device_code1', 'trans_type1']]
trans_type_device_code1_count['every_hour_device_code1_trans_type1_count'] = 1
trans_type_device_code1_count = trans_type_device_code1_count.groupby(['UID', 'day', 'hour', 'device_code1', 'trans_type1']).agg('count').reset_index()
trans_type_device_code1_count = trans_type_device_code1_count[['UID', 'every_hour_device_code1_trans_type1_count']]
trans_type_device_code1_count = trans_type_device_code1_count.groupby('UID')['every_hour_device_code1_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code1_count.columns = ['UID', 'trans_type_device_code1_count_mean', 'trans_type_device_code1_count_min', 'trans_type_device_code1_count_std', 'trans_type_device_code1_count_max']
data = pd.merge(data, trans_type_device_code1_count, on='UID', how='left')

# 每一天每一小时用户用device2用该支付方式支付次数
trans_type_device_code2_count = transaction_data[['UID', 'day', 'hour', 'device_code2', 'trans_type1']]
trans_type_device_code2_count['every_hour_device_code2_trans_type1_count'] = 1
trans_type_device_code2_count = trans_type_device_code2_count.groupby(['UID', 'day', 'hour', 'device_code2', 'trans_type1']).agg('count').reset_index()
trans_type_device_code2_count = trans_type_device_code2_count[['UID', 'every_hour_device_code2_trans_type1_count']]
trans_type_device_code2_count = trans_type_device_code2_count.groupby('UID')['every_hour_device_code2_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code2_count.columns = ['UID', 'trans_type_device_code2_count_mean', 'trans_type_device_code2_count_min', 'trans_type_device_code2_count_std', 'trans_type_device_code2_count_max']
data = pd.merge(data, trans_type_device_code2_count, on='UID', how='left')

# 每一天每一小时用户用device3用该支付方式支付次数
trans_type_device_code3_count = transaction_data[['UID', 'day', 'hour', 'device_code3', 'trans_type1']]
trans_type_device_code3_count['every_hour_device_code3_trans_type1_count'] = 1
trans_type_device_code3_count = trans_type_device_code3_count.groupby(['UID', 'day', 'hour', 'device_code3', 'trans_type1']).agg('count').reset_index()
trans_type_device_code3_count = trans_type_device_code3_count[['UID', 'every_hour_device_code3_trans_type1_count']]
trans_type_device_code3_count = trans_type_device_code3_count.groupby('UID')['every_hour_device_code3_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code3_count.columns = ['UID', 'trans_type_device_code3_count_mean', 'trans_type_device_code3_count_min', 'trans_type_device_code3_count_std', 'trans_type_device_code3_count_max']
data = pd.merge(data, trans_type_device_code3_count, on='UID', how='left')

# 每一天每一个月用户该支付方式的gap
trans_type_hour_gap = transaction_data[['UID', 'day', 'hour', 'trans_type1']]
trans_type_hour_gap['trans_type_hour_gap'] = 1
trans_type_hour_gap = trans_type_hour_gap.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
trans_type_hour_gap['before_hour_gap'] = trans_type_hour_gap.groupby('UID')['trans_type_hour_gap'].shift(1)
trans_type_hour_gap['trans_type_every_hour_gap'] = trans_type_hour_gap['trans_type_hour_gap'] - trans_type_hour_gap['before_hour_gap']
trans_type_hour_gap = trans_type_hour_gap[['UID', 'trans_type_every_hour_gap']]
trans_type_hour_gap = trans_type_hour_gap.groupby('UID')['trans_type_every_hour_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
trans_type_hour_gap.columns = ['UID', 'trans_type_hour_gap_sum', 'trans_type_hour_gap_mean', 'trans_type_hour_gap_min', 'trans_type_hour_gap_std', 'trans_type_hour_gap_max']
data = pd.merge(data, trans_type_hour_gap, on='UID', how='left')

# 每一天每一个月用户该支付方式的rate
trans_type_hour_rate = transaction_data[['UID', 'day', 'hour', 'trans_type1']]
trans_type_hour_rate['trans_type_hour_rate'] = 1
trans_type_hour_rate = trans_type_hour_rate.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
trans_type_hour_rate['before_hour_rate'] = trans_type_hour_rate.groupby('UID')['trans_type_hour_rate'].shift(1)
trans_type_hour_rate['trans_type_every_hour_rate'] = trans_type_hour_rate['trans_type_hour_rate'] / trans_type_hour_rate['before_hour_rate']
trans_type_hour_rate = trans_type_hour_rate[['UID', 'trans_type_every_hour_rate']]
trans_type_hour_rate = trans_type_hour_rate.groupby('UID')['trans_type_every_hour_rate'].agg('sum').reset_index()
data = pd.merge(data, trans_type_hour_rate, on='UID', how='left')


# 每一天的用户支付方式的支付次数
trans_type1_count = transaction_data[['UID', 'day', 'trans_type1']]
trans_type1_count['every_day_trans_type1_count'] = 1
trans_type1_count = trans_type1_count.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
trans_type1_count = trans_type1_count[['UID', 'every_day_trans_type1_count']]
trans_type1_count = trans_type1_count.groupby('UID')['every_day_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_count.columns = ['UID', 'trans_type1_count_mean_day', 'trans_type1_count_min_day', 'trans_type1_count_std_day', 'trans_type1_count_max_day']
data = pd.merge(data, trans_type1_count, on='UID', how='left')

# 每一天用户在该ip用该支付方式支付次数
trans_type1_ip1_count = transaction_data[['UID', 'day', 'ip1_sub', 'trans_type1']]
trans_type1_ip1_count['every_day_ip1_sub_trans_type1_count'] = 1
trans_type1_ip1_count = trans_type1_ip1_count.groupby(['UID', 'day', 'ip1_sub', 'trans_type1']).agg('count').reset_index()
trans_type1_ip1_count = trans_type1_ip1_count[['UID', 'every_day_ip1_sub_trans_type1_count']]
trans_type1_ip1_count = trans_type1_ip1_count.groupby('UID')['every_day_ip1_sub_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_ip1_count.columns = ['UID', 'trans_type1_ip1_count_mean_day', 'trans_type1_ip1_count_min_day', 'trans_type1_ip1_count_std_day', 'trans_type1_ip1_count_max_day']
data = pd.merge(data, trans_type1_ip1_count, on='UID', how='left')

# 每一天用户在该mac用该支付方式支付次数
trans_type1_mac1_count = transaction_data[['UID', 'day', 'mac1', 'trans_type1']]
trans_type1_mac1_count['every_day_mac1_trans_type1_count'] = 1
trans_type1_mac1_count = trans_type1_mac1_count.groupby(['UID', 'day', 'mac1', 'trans_type1']).agg('count').reset_index()
trans_type1_mac1_count = trans_type1_mac1_count[['UID', 'every_day_mac1_trans_type1_count']]
trans_type1_mac1_count = trans_type1_mac1_count.groupby('UID')['every_day_mac1_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type1_mac1_count.columns = ['UID', 'trans_type1_mac1_count_mean_day', 'trans_type1_mac1_count_min_day', 'trans_type1_mac1_count_std_day', 'trans_type1_mac1_count_max_day']
data = pd.merge(data, trans_type1_mac1_count, on='UID', how='left')

# 每一天用户用device1用该支付方式支付次数
trans_type_device_code1_count = transaction_data[['UID', 'day', 'device_code1', 'trans_type1']]
trans_type_device_code1_count['every_day_device_code1_trans_type1_count'] = 1
trans_type_device_code1_count = trans_type_device_code1_count.groupby(['UID', 'day', 'device_code1', 'trans_type1']).agg('count').reset_index()
trans_type_device_code1_count = trans_type_device_code1_count[['UID', 'every_day_device_code1_trans_type1_count']]
trans_type_device_code1_count = trans_type_device_code1_count.groupby('UID')['every_day_device_code1_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code1_count.columns = ['UID', 'trans_type_device_code1_count_mean_day', 'trans_type_device_code1_count_min_day', 'trans_type_device_code1_count_std_day', 'trans_type_device_code1_count_max_day']
data = pd.merge(data, trans_type_device_code1_count, on='UID', how='left')

# 每一天用户用device2用该支付方式支付次数
trans_type_device_code2_count = transaction_data[['UID', 'day', 'device_code2', 'trans_type1']]
trans_type_device_code2_count['every_day_device_code2_trans_type1_count'] = 1
trans_type_device_code2_count = trans_type_device_code2_count.groupby(['UID', 'day', 'device_code2', 'trans_type1']).agg('count').reset_index()
trans_type_device_code2_count = trans_type_device_code2_count[['UID', 'every_day_device_code2_trans_type1_count']]
trans_type_device_code2_count = trans_type_device_code2_count.groupby('UID')['every_day_device_code2_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code2_count.columns = ['UID', 'trans_type_device_code2_count_mean_day', 'trans_type_device_code2_count_min_day', 'trans_type_device_code2_count_std_day', 'trans_type_device_code2_count_max_day']
data = pd.merge(data, trans_type_device_code2_count, on='UID', how='left')

# 每一天用户用device3用该支付方式支付次数
trans_type_device_code3_count = transaction_data[['UID', 'day', 'device_code3', 'trans_type1']]
trans_type_device_code3_count['every_day_device_code3_trans_type1_count'] = 1
trans_type_device_code3_count = trans_type_device_code3_count.groupby(['UID', 'day', 'device_code3', 'trans_type1']).agg('count').reset_index()
trans_type_device_code3_count = trans_type_device_code3_count[['UID', 'every_day_device_code3_trans_type1_count']]
trans_type_device_code3_count = trans_type_device_code3_count.groupby('UID')['every_day_device_code3_trans_type1_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code3_count.columns = ['UID', 'trans_type_device_code3_count_mean_day', 'trans_type_device_code3_count_min_day', 'trans_type_device_code3_count_std_day', 'trans_type_device_code3_count_max_day']
data = pd.merge(data, trans_type_device_code3_count, on='UID', how='left')

# 每一天每一个月用户该支付方式的gap
trans_type_day_gap = transaction_data[['UID', 'day', 'trans_type1']]
trans_type_day_gap['trans_type_day_gap'] = 1
trans_type_day_gap = trans_type_day_gap.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
trans_type_day_gap['before_day_gap'] = trans_type_day_gap.groupby('UID')['trans_type_day_gap'].shift(1)
trans_type_day_gap['trans_type_every_day_gap'] = trans_type_day_gap['trans_type_day_gap'] - trans_type_day_gap['before_day_gap']
trans_type_day_gap = trans_type_day_gap[['UID', 'trans_type_every_day_gap']]
trans_type_day_gap = trans_type_day_gap.groupby('UID')['trans_type_every_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
trans_type_day_gap.columns = ['UID', 'trans_type_day_gap_sum_day', 'trans_type_day_gap_mean_day', 'trans_type_day_gap_min_day', 'trans_type_day_gap_std_day', 'trans_type_day_gap_max_day']
data = pd.merge(data, trans_type_day_gap, on='UID', how='left')

# 每一天每一个月用户该支付方式的rate
trans_type_day_rate = transaction_data[['UID', 'day', 'trans_type1']]
trans_type_day_rate['trans_type_day_rate'] = 1
trans_type_day_rate = trans_type_day_rate.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
trans_type_day_rate['before_day_rate'] = trans_type_day_rate.groupby('UID')['trans_type_day_rate'].shift(1)
trans_type_day_rate['trans_type_every_day_rate'] = trans_type_day_rate['trans_type_day_rate'] / trans_type_day_rate['before_day_rate']
trans_type_day_rate = trans_type_day_rate[['UID', 'trans_type_every_day_rate']]
trans_type_day_rate = trans_type_day_rate.groupby('UID')['trans_type_every_day_rate'].agg('sum').reset_index()
data = pd.merge(data, trans_type_day_rate, on='UID', how='left')


# 每天的半夜用户该支付方式的count
every_morning_trans_type_count = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'trans_type1']]
every_morning_trans_type_count['every_morning_trans_type_count'] = 1
every_morning_trans_type_count = every_morning_trans_type_count.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_morning_trans_type_count = every_morning_trans_type_count[['UID', 'every_morning_trans_type_count']]
every_morning_trans_type_count = every_morning_trans_type_count.groupby('UID')['every_morning_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_morning_trans_type_count.columns = ['UID', 'every_morning_trans_type_count_mean', 'every_morning_trans_type_count_min', 'every_morning_trans_type_count_std', 'every_morning_trans_type_count_max']
data = pd.merge(data, every_morning_trans_type_count, on='UID', how='left')

# 每天的半夜用户该支付方式的gap
every_morning_trans_type_gap = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'trans_type1']]
every_morning_trans_type_gap['every_morning_trans_type_gap'] = 1
every_morning_trans_type_gap = every_morning_trans_type_gap.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_morning_trans_type_gap = every_morning_trans_type_gap[['UID', 'every_morning_trans_type_gap']]
every_morning_trans_type_gap['before_every_morning'] = every_morning_trans_type_gap.groupby('UID')['every_morning_trans_type_gap'].shift(1)
every_morning_trans_type_gap['morning_trans_type_gap'] = every_morning_trans_type_gap['every_morning_trans_type_gap'] - every_morning_trans_type_gap['before_every_morning']
every_morning_trans_type_gap = every_morning_trans_type_gap[['UID', 'morning_trans_type_gap']]
every_morning_trans_type_gap = every_morning_trans_type_gap.groupby('UID')['morning_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_morning_trans_type_gap.columns = ['UID', 'every_morning_trans_type_gap_sum', 'every_morning_trans_type_gap_mean', 'every_morning_trans_type_gap_min', 'every_morning_trans_type_gap_std', 'every_morning_trans_type_gap_max']
data = pd.merge(data, every_morning_trans_type_gap, on='UID', how='left')

# 每天的半夜用户该支付方式的rate
every_morning_trans_type_rate = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'trans_type1']]
every_morning_trans_type_rate['every_morning_trans_type_rate'] = 1
every_morning_trans_type_rate = every_morning_trans_type_rate.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_morning_trans_type_rate = every_morning_trans_type_rate[['UID', 'every_morning_trans_type_rate']]
every_morning_trans_type_rate['before_every_morning'] = every_morning_trans_type_rate.groupby('UID')['every_morning_trans_type_rate'].shift(1)
every_morning_trans_type_rate['morning_trans_type_rate'] = every_morning_trans_type_rate['every_morning_trans_type_rate'] / every_morning_trans_type_rate['before_every_morning']
every_morning_trans_type_rate = every_morning_trans_type_rate[['UID', 'morning_trans_type_rate']]
every_morning_trans_type_rate = every_morning_trans_type_rate.groupby('UID')['morning_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_morning_trans_type_rate, on='UID', how='left')

# 每天的早上用户该支付方式的count
every_work_time_trans_type_count = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'trans_type1']]
every_work_time_trans_type_count['every_work_time_trans_type_count'] = 1
every_work_time_trans_type_count = every_work_time_trans_type_count.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_work_time_trans_type_count = every_work_time_trans_type_count[['UID', 'every_work_time_trans_type_count']]
every_work_time_trans_type_count = every_work_time_trans_type_count.groupby('UID')['every_work_time_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_work_time_trans_type_count.columns = ['UID', 'every_work_time_trans_type_count_mean', 'every_work_time_trans_type_count_min', 'every_work_time_trans_type_count_std', 'every_work_time_trans_type_count_max']
data = pd.merge(data, every_work_time_trans_type_count, on='UID', how='left')

# 每天的早上用户该支付方式的gap
every_work_time_trans_type_gap = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'trans_type1']]
every_work_time_trans_type_gap['every_work_time_trans_type_gap'] = 1
every_work_time_trans_type_gap = every_work_time_trans_type_gap.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_work_time_trans_type_gap = every_work_time_trans_type_gap[['UID', 'every_work_time_trans_type_gap']]
every_work_time_trans_type_gap['before_every_work_time'] = every_work_time_trans_type_gap.groupby('UID')['every_work_time_trans_type_gap'].shift(1)
every_work_time_trans_type_gap['work_time_trans_type_gap'] = every_work_time_trans_type_gap['every_work_time_trans_type_gap'] - every_work_time_trans_type_gap['before_every_work_time']
every_work_time_trans_type_gap = every_work_time_trans_type_gap[['UID', 'work_time_trans_type_gap']]
every_work_time_trans_type_gap = every_work_time_trans_type_gap.groupby('UID')['work_time_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_work_time_trans_type_gap.columns = ['UID', 'every_work_time_trans_type_gap_sum', 'every_work_time_trans_type_gap_mean', 'every_work_time_trans_type_gap_min', 'every_work_time_trans_type_gap_std', 'every_work_time_trans_type_gap_max']
data = pd.merge(data, every_work_time_trans_type_gap, on='UID', how='left')

# 每天的早上用户该支付方式的rate
every_work_time_trans_type_rate = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'trans_type1']]
every_work_time_trans_type_rate['every_work_time_trans_type_rate'] = 1
every_work_time_trans_type_rate = every_work_time_trans_type_rate.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_work_time_trans_type_rate = every_work_time_trans_type_rate[['UID', 'every_work_time_trans_type_rate']]
every_work_time_trans_type_rate['before_every_work_time'] = every_work_time_trans_type_rate.groupby('UID')['every_work_time_trans_type_rate'].shift(1)
every_work_time_trans_type_rate['work_time_trans_type_rate'] = every_work_time_trans_type_rate['every_work_time_trans_type_rate'] / every_work_time_trans_type_rate['before_every_work_time']
every_work_time_trans_type_rate = every_work_time_trans_type_rate[['UID', 'work_time_trans_type_rate']]
every_work_time_trans_type_rate = every_work_time_trans_type_rate.groupby('UID')['work_time_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_work_time_trans_type_rate, on='UID', how='left')

# 每天的下午用户该支付方式的count
every_afternoon_trans_type_count = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'trans_type1']]
every_afternoon_trans_type_count['every_afternoon_trans_type_count'] = 1
every_afternoon_trans_type_count = every_afternoon_trans_type_count.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_afternoon_trans_type_count = every_afternoon_trans_type_count[['UID', 'every_afternoon_trans_type_count']]
every_afternoon_trans_type_count = every_afternoon_trans_type_count.groupby('UID')['every_afternoon_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_afternoon_trans_type_count.columns = ['UID', 'every_afternoon_trans_type_count_mean', 'every_afternoon_trans_type_count_min', 'every_afternoon_trans_type_count_std', 'every_afternoon_trans_type_count_max']
data = pd.merge(data, every_afternoon_trans_type_count, on='UID', how='left')

# 每天的下午用户该支付方式的gap
every_afternoon_trans_type_gap = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'trans_type1']]
every_afternoon_trans_type_gap['every_afternoon_trans_type_gap'] = 1
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap[['UID', 'every_afternoon_trans_type_gap']]
every_afternoon_trans_type_gap['before_every_afternoon'] = every_afternoon_trans_type_gap.groupby('UID')['every_afternoon_trans_type_gap'].shift(1)
every_afternoon_trans_type_gap['afternoon_trans_type_gap'] = every_afternoon_trans_type_gap['every_afternoon_trans_type_gap'] - every_afternoon_trans_type_gap['before_every_afternoon']
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap[['UID', 'afternoon_trans_type_gap']]
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap.groupby('UID')['afternoon_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_afternoon_trans_type_gap.columns = ['UID', 'every_afternoon_trans_type_gap_sum', 'every_afternoon_trans_type_gap_mean', 'every_afternoon_trans_type_gap_min', 'every_afternoon_trans_type_gap_std', 'every_afternoon_trans_type_gap_max']
data = pd.merge(data, every_afternoon_trans_type_gap, on='UID', how='left')

# 每天的下午用户该支付方式的rate
every_afternoon_trans_type_rate = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'trans_type1']]
every_afternoon_trans_type_rate['every_afternoon_trans_type_rate'] = 1
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate[['UID', 'every_afternoon_trans_type_rate']]
every_afternoon_trans_type_rate['before_every_afternoon'] = every_afternoon_trans_type_rate.groupby('UID')['every_afternoon_trans_type_rate'].shift(1)
every_afternoon_trans_type_rate['afternoon_trans_type_rate'] = every_afternoon_trans_type_rate['every_afternoon_trans_type_rate'] / every_afternoon_trans_type_rate['before_every_afternoon']
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate[['UID', 'afternoon_trans_type_rate']]
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate.groupby('UID')['afternoon_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_afternoon_trans_type_rate, on='UID', how='left')

# 每天的晚上用户该支付方式的count
every_night_trans_type_count = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'trans_type1']]
every_night_trans_type_count['every_night_trans_type_count'] = 1
every_night_trans_type_count = every_night_trans_type_count.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_night_trans_type_count = every_night_trans_type_count[['UID', 'every_night_trans_type_count']]
every_night_trans_type_count = every_night_trans_type_count.groupby('UID')['every_night_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_night_trans_type_count.columns = ['UID', 'every_night_trans_type_count_mean', 'every_night_trans_type_count_min', 'every_night_trans_type_count_std', 'every_night_trans_type_count_max']
data = pd.merge(data, every_night_trans_type_count, on='UID', how='left')

# 每天的晚上用户该支付方式的gap
every_night_trans_type_gap = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'trans_type1']]
every_night_trans_type_gap['every_night_trans_type_gap'] = 1
every_night_trans_type_gap = every_night_trans_type_gap.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_night_trans_type_gap = every_night_trans_type_gap[['UID', 'every_night_trans_type_gap']]
every_night_trans_type_gap['before_every_night'] = every_night_trans_type_gap.groupby('UID')['every_night_trans_type_gap'].shift(1)
every_night_trans_type_gap['night_trans_type_gap'] = every_night_trans_type_gap['every_night_trans_type_gap'] - every_night_trans_type_gap['before_every_night']
every_night_trans_type_gap = every_night_trans_type_gap[['UID', 'night_trans_type_gap']]
every_night_trans_type_gap = every_night_trans_type_gap.groupby('UID')['night_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_night_trans_type_gap.columns = ['UID', 'every_night_trans_type_gap_sum', 'every_night_trans_type_gap_mean', 'every_night_trans_type_gap_min', 'every_night_trans_type_gap_std', 'every_night_trans_type_gap_max']
data = pd.merge(data, every_night_trans_type_gap, on='UID', how='left')

# 每天的晚上用户该支付方式的rate
every_night_trans_type_rate = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'trans_type1']]
every_night_trans_type_rate['every_night_trans_type_rate'] = 1
every_night_trans_type_rate = every_night_trans_type_rate.groupby(['UID', 'day', 'hour', 'trans_type1']).agg('count').reset_index()
every_night_trans_type_rate = every_night_trans_type_rate[['UID', 'every_night_trans_type_rate']]
every_night_trans_type_rate['before_every_night'] = every_night_trans_type_rate.groupby('UID')['every_night_trans_type_rate'].shift(1)
every_night_trans_type_rate['night_trans_type_rate'] = every_night_trans_type_rate['every_night_trans_type_rate'] / every_night_trans_type_rate['before_every_night']
every_night_trans_type_rate = every_night_trans_type_rate[['UID', 'night_trans_type_rate']]
every_night_trans_type_rate = every_night_trans_type_rate.groupby('UID')['night_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_night_trans_type_rate, on='UID', how='left')

# 每三天用户的gap
every_three_day_gap = transaction_data[['UID', 'day', 'trans_type1']]
every_three_day_gap['every_day_count'] = 1
every_three_day_gap = every_three_day_gap.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
every_three_day_gap['before_day_count'] = every_three_day_gap.groupby('UID')['every_day_count'].shift(3)
every_three_day_gap['every_three_day_gap'] = every_three_day_gap['every_day_count'] - every_three_day_gap['before_day_count']
every_three_day_gap = every_three_day_gap[['UID', 'every_three_day_gap']]
every_three_day_gap = every_three_day_gap.groupby('UID')['every_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_three_day_gap.columns = ['UID', 'every_three_day_gap_sum', 'every_three_day_gap_mean', 'every_three_day_gap_min', 'every_three_day_gap_std', 'every_three_day_gap_max']
data = pd.merge(data, every_three_day_gap, on='UID', how='left')

# 每三天用户的rate
every_three_day_rate = transaction_data[['UID', 'day', 'trans_type1']]
every_three_day_rate['every_day_count'] = 1
every_three_day_rate = every_three_day_rate.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
every_three_day_rate['before_day_count'] = every_three_day_rate.groupby('UID')['every_day_count'].shift(3)
every_three_day_rate['every_three_day_rate'] = every_three_day_rate['every_day_count'] / every_three_day_rate['before_day_count']
every_three_day_rate = every_three_day_rate[['UID', 'every_three_day_rate']]
every_three_day_rate = every_three_day_rate.groupby('UID')['every_three_day_rate'].agg('sum').reset_index()
data = pd.merge(data, every_three_day_rate, on='UID', how='left')

# 每七天用户的gap
every_sixteen_day_gap = transaction_data[['UID', 'day', 'trans_type1']]
every_sixteen_day_gap['every_day_count'] = 1
every_sixteen_day_gap = every_sixteen_day_gap.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
every_sixteen_day_gap['before_day_count'] = every_sixteen_day_gap.groupby('UID')['every_day_count'].shift(7)
every_sixteen_day_gap['every_sixteen_day_gap'] = every_sixteen_day_gap['every_day_count'] - every_sixteen_day_gap['before_day_count']
every_sixteen_day_gap = every_sixteen_day_gap[['UID', 'every_sixteen_day_gap']]
every_sixteen_day_gap = every_sixteen_day_gap.groupby('UID')['every_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_sixteen_day_gap.columns = ['UID', 'every_sixteen_day_gap_sum', 'every_sixteen_day_gap_mean', 'every_sixteen_day_gap_min', 'every_sixteen_day_gap_std', 'every_sixteen_day_gap_max']
data = pd.merge(data, every_sixteen_day_gap, on='UID', how='left')

# 每七天用户的rate
every_sixteen_day_rate = transaction_data[['UID', 'day', 'trans_type1']]
every_sixteen_day_rate['every_day_count'] = 1
every_sixteen_day_rate = every_sixteen_day_rate.groupby(['UID', 'day', 'trans_type1']).agg('count').reset_index()
every_sixteen_day_rate['before_day_count'] = every_sixteen_day_rate.groupby('UID')['every_day_count'].shift(7)
every_sixteen_day_rate['every_sixteen_day_rate'] = every_sixteen_day_rate['every_day_count'] / every_sixteen_day_rate['before_day_count']
every_sixteen_day_rate = every_sixteen_day_rate[['UID', 'every_sixteen_day_rate']]
every_sixteen_day_rate = every_sixteen_day_rate.groupby('UID')['every_sixteen_day_rate'].agg('sum').reset_index()
data = pd.merge(data, every_sixteen_day_rate, on='UID', how='left')

In [33]:
# 每一天的每一小时用户支付方式的支付次数
market_code_count = transaction_data[['UID', 'day', 'hour', 'market_code']]
market_code_count['every_hour_market_code_count'] = 1
market_code_count = market_code_count.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
market_code_count = market_code_count[['UID', 'every_hour_market_code_count']]
market_code_count = market_code_count.groupby('UID')['every_hour_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_count.columns = ['UID', 'market_code_count_mean', 'market_code_count_min', 'market_code_count_std', 'market_code_count_max']
data = pd.merge(data, market_code_count, on='UID', how='left')

# 每一天每一小时用户在该ip用该支付方式支付次数
market_code_ip1_count = transaction_data[['UID', 'day', 'hour', 'ip1_sub', 'market_code']]
market_code_ip1_count['every_hour_ip1_sub_market_code_count'] = 1
market_code_ip1_count = market_code_ip1_count.groupby(['UID', 'day', 'hour', 'ip1_sub', 'market_code']).agg('count').reset_index()
market_code_ip1_count = market_code_ip1_count[['UID', 'every_hour_ip1_sub_market_code_count']]
market_code_ip1_count = market_code_ip1_count.groupby('UID')['every_hour_ip1_sub_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_ip1_count.columns = ['UID', 'market_code_ip1_count_mean', 'market_code_ip1_count_min', 'market_code_ip1_count_std', 'market_code_ip1_count_max']
data = pd.merge(data, market_code_ip1_count, on='UID', how='left')

# 每一天每一小时用户在该ip用该支付方式支付次数
market_code_mac1_count = transaction_data[['UID', 'day', 'hour', 'mac1', 'market_code']]
market_code_mac1_count['every_hour_mac1_market_code_count'] = 1
market_code_mac1_count = market_code_mac1_count.groupby(['UID', 'day', 'hour', 'mac1', 'market_code']).agg('count').reset_index()
market_code_mac1_count = market_code_mac1_count[['UID', 'every_hour_mac1_market_code_count']]
market_code_mac1_count = market_code_mac1_count.groupby('UID')['every_hour_mac1_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_mac1_count.columns = ['UID', 'market_code_mac1_count_mean', 'market_code_mac1_count_min', 'market_code_mac1_count_std', 'market_code_mac1_count_max']
data = pd.merge(data, market_code_mac1_count, on='UID', how='left')

# 每一天每一小时用户用device1用该支付方式支付次数
trans_type_device_code1_count = transaction_data[['UID', 'day', 'hour', 'device_code1', 'market_code']]
trans_type_device_code1_count['every_hour_device_code1_market_code_count'] = 1
trans_type_device_code1_count = trans_type_device_code1_count.groupby(['UID', 'day', 'hour', 'device_code1', 'market_code']).agg('count').reset_index()
trans_type_device_code1_count = trans_type_device_code1_count[['UID', 'every_hour_device_code1_market_code_count']]
trans_type_device_code1_count = trans_type_device_code1_count.groupby('UID')['every_hour_device_code1_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code1_count.columns = ['UID', 'trans_type_device_code1_count_mean', 'trans_type_device_code1_count_min', 'trans_type_device_code1_count_std', 'trans_type_device_code1_count_max']
data = pd.merge(data, trans_type_device_code1_count, on='UID', how='left')

# 每一天每一小时用户用device2用该支付方式支付次数
trans_type_device_code2_count = transaction_data[['UID', 'day', 'hour', 'device_code2', 'market_code']]
trans_type_device_code2_count['every_hour_device_code2_market_code_count'] = 1
trans_type_device_code2_count = trans_type_device_code2_count.groupby(['UID', 'day', 'hour', 'device_code2', 'market_code']).agg('count').reset_index()
trans_type_device_code2_count = trans_type_device_code2_count[['UID', 'every_hour_device_code2_market_code_count']]
trans_type_device_code2_count = trans_type_device_code2_count.groupby('UID')['every_hour_device_code2_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code2_count.columns = ['UID', 'trans_type_device_code2_count_mean', 'trans_type_device_code2_count_min', 'trans_type_device_code2_count_std', 'trans_type_device_code2_count_max']
data = pd.merge(data, trans_type_device_code2_count, on='UID', how='left')

# 每一天每一小时用户用device3用该支付方式支付次数
trans_type_device_code3_count = transaction_data[['UID', 'day', 'hour', 'device_code3', 'market_code']]
trans_type_device_code3_count['every_hour_device_code3_market_code_count'] = 1
trans_type_device_code3_count = trans_type_device_code3_count.groupby(['UID', 'day', 'hour', 'device_code3', 'market_code']).agg('count').reset_index()
trans_type_device_code3_count = trans_type_device_code3_count[['UID', 'every_hour_device_code3_market_code_count']]
trans_type_device_code3_count = trans_type_device_code3_count.groupby('UID')['every_hour_device_code3_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code3_count.columns = ['UID', 'trans_type_device_code3_count_mean', 'trans_type_device_code3_count_min', 'trans_type_device_code3_count_std', 'trans_type_device_code3_count_max']
data = pd.merge(data, trans_type_device_code3_count, on='UID', how='left')

# 每一天每一个月用户该支付方式的gap
trans_type_hour_gap = transaction_data[['UID', 'day', 'hour', 'market_code']]
trans_type_hour_gap['trans_type_hour_gap'] = 1
trans_type_hour_gap = trans_type_hour_gap.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
trans_type_hour_gap['before_hour_gap'] = trans_type_hour_gap.groupby('UID')['trans_type_hour_gap'].shift(1)
trans_type_hour_gap['trans_type_every_hour_gap'] = trans_type_hour_gap['trans_type_hour_gap'] - trans_type_hour_gap['before_hour_gap']
trans_type_hour_gap = trans_type_hour_gap[['UID', 'trans_type_every_hour_gap']]
trans_type_hour_gap = trans_type_hour_gap.groupby('UID')['trans_type_every_hour_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
trans_type_hour_gap.columns = ['UID', 'trans_type_hour_gap_sum', 'trans_type_hour_gap_mean', 'trans_type_hour_gap_min', 'trans_type_hour_gap_std', 'trans_type_hour_gap_max']
data = pd.merge(data, trans_type_hour_gap, on='UID', how='left')

# 每一天每一个月用户该支付方式的rate
trans_type_hour_rate = transaction_data[['UID', 'day', 'hour', 'market_code']]
trans_type_hour_rate['trans_type_hour_rate'] = 1
trans_type_hour_rate = trans_type_hour_rate.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
trans_type_hour_rate['before_hour_rate'] = trans_type_hour_rate.groupby('UID')['trans_type_hour_rate'].shift(1)
trans_type_hour_rate['trans_type_every_hour_rate'] = trans_type_hour_rate['trans_type_hour_rate'] / trans_type_hour_rate['before_hour_rate']
trans_type_hour_rate = trans_type_hour_rate[['UID', 'trans_type_every_hour_rate']]
trans_type_hour_rate = trans_type_hour_rate.groupby('UID')['trans_type_every_hour_rate'].agg('sum').reset_index()
data = pd.merge(data, trans_type_hour_rate, on='UID', how='left')


# 每一天的用户支付方式的支付次数
market_code_count = transaction_data[['UID', 'day', 'market_code']]
market_code_count['every_day_market_code_count'] = 1
market_code_count = market_code_count.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
market_code_count = market_code_count[['UID', 'every_day_market_code_count']]
market_code_count = market_code_count.groupby('UID')['every_day_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_count.columns = ['UID', 'market_code_count_mean_day', 'market_code_count_min_day', 'market_code_count_std_day', 'market_code_count_max_day']
data = pd.merge(data, market_code_count, on='UID', how='left')

# 每一天用户在该ip用该支付方式支付次数
market_code_ip1_count = transaction_data[['UID', 'day', 'ip1_sub', 'market_code']]
market_code_ip1_count['every_day_ip1_sub_market_code_count'] = 1
market_code_ip1_count = market_code_ip1_count.groupby(['UID', 'day', 'ip1_sub', 'market_code']).agg('count').reset_index()
market_code_ip1_count = market_code_ip1_count[['UID', 'every_day_ip1_sub_market_code_count']]
market_code_ip1_count = market_code_ip1_count.groupby('UID')['every_day_ip1_sub_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_ip1_count.columns = ['UID', 'market_code_ip1_count_mean_day', 'market_code_ip1_count_min_day', 'market_code_ip1_count_std_day', 'market_code_ip1_count_max_day']
data = pd.merge(data, market_code_ip1_count, on='UID', how='left')

# 每一天用户在该mac用该支付方式支付次数
market_code_mac1_count = transaction_data[['UID', 'day', 'mac1', 'market_code']]
market_code_mac1_count['every_day_mac1_market_code_count'] = 1
market_code_mac1_count = market_code_mac1_count.groupby(['UID', 'day', 'mac1', 'market_code']).agg('count').reset_index()
market_code_mac1_count = market_code_mac1_count[['UID', 'every_day_mac1_market_code_count']]
market_code_mac1_count = market_code_mac1_count.groupby('UID')['every_day_mac1_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
market_code_mac1_count.columns = ['UID', 'market_code_mac1_count_mean_day', 'market_code_mac1_count_min_day', 'market_code_mac1_count_std_day', 'market_code_mac1_count_max_day']
data = pd.merge(data, market_code_mac1_count, on='UID', how='left')

# 每一天用户用device1用该支付方式支付次数
trans_type_device_code1_count = transaction_data[['UID', 'day', 'device_code1', 'market_code']]
trans_type_device_code1_count['every_day_device_code1_market_code_count'] = 1
trans_type_device_code1_count = trans_type_device_code1_count.groupby(['UID', 'day', 'device_code1', 'market_code']).agg('count').reset_index()
trans_type_device_code1_count = trans_type_device_code1_count[['UID', 'every_day_device_code1_market_code_count']]
trans_type_device_code1_count = trans_type_device_code1_count.groupby('UID')['every_day_device_code1_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code1_count.columns = ['UID', 'trans_type_device_code1_count_mean_day', 'trans_type_device_code1_count_min_day', 'trans_type_device_code1_count_std_day', 'trans_type_device_code1_count_max_day']
data = pd.merge(data, trans_type_device_code1_count, on='UID', how='left')

# 每一天用户用device2用该支付方式支付次数
trans_type_device_code2_count = transaction_data[['UID', 'day', 'device_code2', 'market_code']]
trans_type_device_code2_count['every_day_device_code2_market_code_count'] = 1
trans_type_device_code2_count = trans_type_device_code2_count.groupby(['UID', 'day', 'device_code2', 'market_code']).agg('count').reset_index()
trans_type_device_code2_count = trans_type_device_code2_count[['UID', 'every_day_device_code2_market_code_count']]
trans_type_device_code2_count = trans_type_device_code2_count.groupby('UID')['every_day_device_code2_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code2_count.columns = ['UID', 'trans_type_device_code2_count_mean_day', 'trans_type_device_code2_count_min_day', 'trans_type_device_code2_count_std_day', 'trans_type_device_code2_count_max_day']
data = pd.merge(data, trans_type_device_code2_count, on='UID', how='left')

# 每一天用户用device3用该支付方式支付次数
trans_type_device_code3_count = transaction_data[['UID', 'day', 'device_code3', 'market_code']]
trans_type_device_code3_count['every_day_device_code3_market_code_count'] = 1
trans_type_device_code3_count = trans_type_device_code3_count.groupby(['UID', 'day', 'device_code3', 'market_code']).agg('count').reset_index()
trans_type_device_code3_count = trans_type_device_code3_count[['UID', 'every_day_device_code3_market_code_count']]
trans_type_device_code3_count = trans_type_device_code3_count.groupby('UID')['every_day_device_code3_market_code_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
trans_type_device_code3_count.columns = ['UID', 'trans_type_device_code3_count_mean_day', 'trans_type_device_code3_count_min_day', 'trans_type_device_code3_count_std_day', 'trans_type_device_code3_count_max_day']
data = pd.merge(data, trans_type_device_code3_count, on='UID', how='left')

# 每一天每一个月用户该支付方式的gap
trans_type_day_gap = transaction_data[['UID', 'day', 'market_code']]
trans_type_day_gap['trans_type_day_gap'] = 1
trans_type_day_gap = trans_type_day_gap.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
trans_type_day_gap['before_day_gap'] = trans_type_day_gap.groupby('UID')['trans_type_day_gap'].shift(1)
trans_type_day_gap['trans_type_every_day_gap'] = trans_type_day_gap['trans_type_day_gap'] - trans_type_day_gap['before_day_gap']
trans_type_day_gap = trans_type_day_gap[['UID', 'trans_type_every_day_gap']]
trans_type_day_gap = trans_type_day_gap.groupby('UID')['trans_type_every_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
trans_type_day_gap.columns = ['UID', 'trans_type_day_gap_sum_day', 'trans_type_day_gap_mean_day', 'trans_type_day_gap_min_day', 'trans_type_day_gap_std_day', 'trans_type_day_gap_max_day']
data = pd.merge(data, trans_type_day_gap, on='UID', how='left')

# 每一天每一个月用户该支付方式的rate
trans_type_day_rate = transaction_data[['UID', 'day', 'market_code']]
trans_type_day_rate['trans_type_day_rate'] = 1
trans_type_day_rate = trans_type_day_rate.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
trans_type_day_rate['before_day_rate'] = trans_type_day_rate.groupby('UID')['trans_type_day_rate'].shift(1)
trans_type_day_rate['trans_type_every_day_rate'] = trans_type_day_rate['trans_type_day_rate'] / trans_type_day_rate['before_day_rate']
trans_type_day_rate = trans_type_day_rate[['UID', 'trans_type_every_day_rate']]
trans_type_day_rate = trans_type_day_rate.groupby('UID')['trans_type_every_day_rate'].agg('sum').reset_index()
data = pd.merge(data, trans_type_day_rate, on='UID', how='left')


# 每天的半夜用户该支付方式的count
every_morning_trans_type_count = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'market_code']]
every_morning_trans_type_count['every_morning_trans_type_count'] = 1
every_morning_trans_type_count = every_morning_trans_type_count.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_morning_trans_type_count = every_morning_trans_type_count[['UID', 'every_morning_trans_type_count']]
every_morning_trans_type_count = every_morning_trans_type_count.groupby('UID')['every_morning_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_morning_trans_type_count.columns = ['UID', 'every_morning_trans_type_count_mean', 'every_morning_trans_type_count_min', 'every_morning_trans_type_count_std', 'every_morning_trans_type_count_max']
data = pd.merge(data, every_morning_trans_type_count, on='UID', how='left')

# 每天的半夜用户该支付方式的gap
every_morning_trans_type_gap = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'market_code']]
every_morning_trans_type_gap['every_morning_trans_type_gap'] = 1
every_morning_trans_type_gap = every_morning_trans_type_gap.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_morning_trans_type_gap = every_morning_trans_type_gap[['UID', 'every_morning_trans_type_gap']]
every_morning_trans_type_gap['before_every_morning'] = every_morning_trans_type_gap.groupby('UID')['every_morning_trans_type_gap'].shift(1)
every_morning_trans_type_gap['morning_trans_type_gap'] = every_morning_trans_type_gap['every_morning_trans_type_gap'] - every_morning_trans_type_gap['before_every_morning']
every_morning_trans_type_gap = every_morning_trans_type_gap[['UID', 'morning_trans_type_gap']]
every_morning_trans_type_gap = every_morning_trans_type_gap.groupby('UID')['morning_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_morning_trans_type_gap.columns = ['UID', 'every_morning_trans_type_gap_sum', 'every_morning_trans_type_gap_mean', 'every_morning_trans_type_gap_min', 'every_morning_trans_type_gap_std', 'every_morning_trans_type_gap_max']
data = pd.merge(data, every_morning_trans_type_gap, on='UID', how='left')

# 每天的半夜用户该支付方式的rate
every_morning_trans_type_rate = transaction_data[transaction_data.hour <= 5][['UID', 'day', 'hour', 'market_code']]
every_morning_trans_type_rate['every_morning_trans_type_rate'] = 1
every_morning_trans_type_rate = every_morning_trans_type_rate.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_morning_trans_type_rate = every_morning_trans_type_rate[['UID', 'every_morning_trans_type_rate']]
every_morning_trans_type_rate['before_every_morning'] = every_morning_trans_type_rate.groupby('UID')['every_morning_trans_type_rate'].shift(1)
every_morning_trans_type_rate['morning_trans_type_rate'] = every_morning_trans_type_rate['every_morning_trans_type_rate'] / every_morning_trans_type_rate['before_every_morning']
every_morning_trans_type_rate = every_morning_trans_type_rate[['UID', 'morning_trans_type_rate']]
every_morning_trans_type_rate = every_morning_trans_type_rate.groupby('UID')['morning_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_morning_trans_type_rate, on='UID', how='left')

# 每天的早上用户该支付方式的count
every_work_time_trans_type_count = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'market_code']]
every_work_time_trans_type_count['every_work_time_trans_type_count'] = 1
every_work_time_trans_type_count = every_work_time_trans_type_count.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_work_time_trans_type_count = every_work_time_trans_type_count[['UID', 'every_work_time_trans_type_count']]
every_work_time_trans_type_count = every_work_time_trans_type_count.groupby('UID')['every_work_time_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_work_time_trans_type_count.columns = ['UID', 'every_work_time_trans_type_count_mean', 'every_work_time_trans_type_count_min', 'every_work_time_trans_type_count_std', 'every_work_time_trans_type_count_max']
data = pd.merge(data, every_work_time_trans_type_count, on='UID', how='left')

# 每天的早上用户该支付方式的gap
every_work_time_trans_type_gap = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'market_code']]
every_work_time_trans_type_gap['every_work_time_trans_type_gap'] = 1
every_work_time_trans_type_gap = every_work_time_trans_type_gap.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_work_time_trans_type_gap = every_work_time_trans_type_gap[['UID', 'every_work_time_trans_type_gap']]
every_work_time_trans_type_gap['before_every_work_time'] = every_work_time_trans_type_gap.groupby('UID')['every_work_time_trans_type_gap'].shift(1)
every_work_time_trans_type_gap['work_time_trans_type_gap'] = every_work_time_trans_type_gap['every_work_time_trans_type_gap'] - every_work_time_trans_type_gap['before_every_work_time']
every_work_time_trans_type_gap = every_work_time_trans_type_gap[['UID', 'work_time_trans_type_gap']]
every_work_time_trans_type_gap = every_work_time_trans_type_gap.groupby('UID')['work_time_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_work_time_trans_type_gap.columns = ['UID', 'every_work_time_trans_type_gap_sum', 'every_work_time_trans_type_gap_mean', 'every_work_time_trans_type_gap_min', 'every_work_time_trans_type_gap_std', 'every_work_time_trans_type_gap_max']
data = pd.merge(data, every_work_time_trans_type_gap, on='UID', how='left')

# 每天的早上用户该支付方式的rate
every_work_time_trans_type_rate = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', 'day', 'hour', 'market_code']]
every_work_time_trans_type_rate['every_work_time_trans_type_rate'] = 1
every_work_time_trans_type_rate = every_work_time_trans_type_rate.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_work_time_trans_type_rate = every_work_time_trans_type_rate[['UID', 'every_work_time_trans_type_rate']]
every_work_time_trans_type_rate['before_every_work_time'] = every_work_time_trans_type_rate.groupby('UID')['every_work_time_trans_type_rate'].shift(1)
every_work_time_trans_type_rate['work_time_trans_type_rate'] = every_work_time_trans_type_rate['every_work_time_trans_type_rate'] / every_work_time_trans_type_rate['before_every_work_time']
every_work_time_trans_type_rate = every_work_time_trans_type_rate[['UID', 'work_time_trans_type_rate']]
every_work_time_trans_type_rate = every_work_time_trans_type_rate.groupby('UID')['work_time_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_work_time_trans_type_rate, on='UID', how='left')

# 每天的下午用户该支付方式的count
every_afternoon_trans_type_count = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'market_code']]
every_afternoon_trans_type_count['every_afternoon_trans_type_count'] = 1
every_afternoon_trans_type_count = every_afternoon_trans_type_count.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_afternoon_trans_type_count = every_afternoon_trans_type_count[['UID', 'every_afternoon_trans_type_count']]
every_afternoon_trans_type_count = every_afternoon_trans_type_count.groupby('UID')['every_afternoon_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_afternoon_trans_type_count.columns = ['UID', 'every_afternoon_trans_type_count_mean', 'every_afternoon_trans_type_count_min', 'every_afternoon_trans_type_count_std', 'every_afternoon_trans_type_count_max']
data = pd.merge(data, every_afternoon_trans_type_count, on='UID', how='left')

# 每天的下午用户该支付方式的gap
every_afternoon_trans_type_gap = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'market_code']]
every_afternoon_trans_type_gap['every_afternoon_trans_type_gap'] = 1
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap[['UID', 'every_afternoon_trans_type_gap']]
every_afternoon_trans_type_gap['before_every_afternoon'] = every_afternoon_trans_type_gap.groupby('UID')['every_afternoon_trans_type_gap'].shift(1)
every_afternoon_trans_type_gap['afternoon_trans_type_gap'] = every_afternoon_trans_type_gap['every_afternoon_trans_type_gap'] - every_afternoon_trans_type_gap['before_every_afternoon']
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap[['UID', 'afternoon_trans_type_gap']]
every_afternoon_trans_type_gap = every_afternoon_trans_type_gap.groupby('UID')['afternoon_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_afternoon_trans_type_gap.columns = ['UID', 'every_afternoon_trans_type_gap_sum', 'every_afternoon_trans_type_gap_mean', 'every_afternoon_trans_type_gap_min', 'every_afternoon_trans_type_gap_std', 'every_afternoon_trans_type_gap_max']
data = pd.merge(data, every_afternoon_trans_type_gap, on='UID', how='left')

# 每天的下午用户该支付方式的rate
every_afternoon_trans_type_rate = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', 'day', 'hour', 'market_code']]
every_afternoon_trans_type_rate['every_afternoon_trans_type_rate'] = 1
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate[['UID', 'every_afternoon_trans_type_rate']]
every_afternoon_trans_type_rate['before_every_afternoon'] = every_afternoon_trans_type_rate.groupby('UID')['every_afternoon_trans_type_rate'].shift(1)
every_afternoon_trans_type_rate['afternoon_trans_type_rate'] = every_afternoon_trans_type_rate['every_afternoon_trans_type_rate'] / every_afternoon_trans_type_rate['before_every_afternoon']
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate[['UID', 'afternoon_trans_type_rate']]
every_afternoon_trans_type_rate = every_afternoon_trans_type_rate.groupby('UID')['afternoon_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_afternoon_trans_type_rate, on='UID', how='left')

# 每天的晚上用户该支付方式的count
every_night_trans_type_count = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'market_code']]
every_night_trans_type_count['every_night_trans_type_count'] = 1
every_night_trans_type_count = every_night_trans_type_count.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_night_trans_type_count = every_night_trans_type_count[['UID', 'every_night_trans_type_count']]
every_night_trans_type_count = every_night_trans_type_count.groupby('UID')['every_night_trans_type_count'].agg({'mean', 'max', 'min', 'std'}).reset_index()
every_night_trans_type_count.columns = ['UID', 'every_night_trans_type_count_mean', 'every_night_trans_type_count_min', 'every_night_trans_type_count_std', 'every_night_trans_type_count_max']
data = pd.merge(data, every_night_trans_type_count, on='UID', how='left')

# 每天的晚上用户该支付方式的gap
every_night_trans_type_gap = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'market_code']]
every_night_trans_type_gap['every_night_trans_type_gap'] = 1
every_night_trans_type_gap = every_night_trans_type_gap.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_night_trans_type_gap = every_night_trans_type_gap[['UID', 'every_night_trans_type_gap']]
every_night_trans_type_gap['before_every_night'] = every_night_trans_type_gap.groupby('UID')['every_night_trans_type_gap'].shift(1)
every_night_trans_type_gap['night_trans_type_gap'] = every_night_trans_type_gap['every_night_trans_type_gap'] - every_night_trans_type_gap['before_every_night']
every_night_trans_type_gap = every_night_trans_type_gap[['UID', 'night_trans_type_gap']]
every_night_trans_type_gap = every_night_trans_type_gap.groupby('UID')['night_trans_type_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_night_trans_type_gap.columns = ['UID', 'every_night_trans_type_gap_sum', 'every_night_trans_type_gap_mean', 'every_night_trans_type_gap_min', 'every_night_trans_type_gap_std', 'every_night_trans_type_gap_max']
data = pd.merge(data, every_night_trans_type_gap, on='UID', how='left')

# 每天的晚上用户该支付方式的rate
every_night_trans_type_rate = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', 'day', 'hour', 'market_code']]
every_night_trans_type_rate['every_night_trans_type_rate'] = 1
every_night_trans_type_rate = every_night_trans_type_rate.groupby(['UID', 'day', 'hour', 'market_code']).agg('count').reset_index()
every_night_trans_type_rate = every_night_trans_type_rate[['UID', 'every_night_trans_type_rate']]
every_night_trans_type_rate['before_every_night'] = every_night_trans_type_rate.groupby('UID')['every_night_trans_type_rate'].shift(1)
every_night_trans_type_rate['night_trans_type_rate'] = every_night_trans_type_rate['every_night_trans_type_rate'] / every_night_trans_type_rate['before_every_night']
every_night_trans_type_rate = every_night_trans_type_rate[['UID', 'night_trans_type_rate']]
every_night_trans_type_rate = every_night_trans_type_rate.groupby('UID')['night_trans_type_rate'].agg('sum').reset_index()
data = pd.merge(data, every_night_trans_type_rate, on='UID', how='left')

# 每三天用户的gap
every_three_day_gap = transaction_data[['UID', 'day', 'market_code']]
every_three_day_gap['every_day_count'] = 1
every_three_day_gap = every_three_day_gap.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
every_three_day_gap['before_day_count'] = every_three_day_gap.groupby('UID')['every_day_count'].shift(3)
every_three_day_gap['every_three_day_gap'] = every_three_day_gap['every_day_count'] - every_three_day_gap['before_day_count']
every_three_day_gap = every_three_day_gap[['UID', 'every_three_day_gap']]
every_three_day_gap = every_three_day_gap.groupby('UID')['every_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_three_day_gap.columns = ['UID', 'every_three_day_gap_sum', 'every_three_day_gap_mean', 'every_three_day_gap_min', 'every_three_day_gap_std', 'every_three_day_gap_max']
data = pd.merge(data, every_three_day_gap, on='UID', how='left')

# 每三天用户的rate
every_three_day_rate = transaction_data[['UID', 'day', 'market_code']]
every_three_day_rate['every_day_count'] = 1
every_three_day_rate = every_three_day_rate.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
every_three_day_rate['before_day_count'] = every_three_day_rate.groupby('UID')['every_day_count'].shift(3)
every_three_day_rate['every_three_day_rate'] = every_three_day_rate['every_day_count'] / every_three_day_rate['before_day_count']
every_three_day_rate = every_three_day_rate[['UID', 'every_three_day_rate']]
every_three_day_rate = every_three_day_rate.groupby('UID')['every_three_day_rate'].agg('sum').reset_index()
data = pd.merge(data, every_three_day_rate, on='UID', how='left')

# 每七天用户的gap
every_sixteen_day_gap = transaction_data[['UID', 'day', 'market_code']]
every_sixteen_day_gap['every_day_count'] = 1
every_sixteen_day_gap = every_sixteen_day_gap.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
every_sixteen_day_gap['before_day_count'] = every_sixteen_day_gap.groupby('UID')['every_day_count'].shift(7)
every_sixteen_day_gap['every_sixteen_day_gap'] = every_sixteen_day_gap['every_day_count'] - every_sixteen_day_gap['before_day_count']
every_sixteen_day_gap = every_sixteen_day_gap[['UID', 'every_sixteen_day_gap']]
every_sixteen_day_gap = every_sixteen_day_gap.groupby('UID')['every_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_sixteen_day_gap.columns = ['UID', 'every_sixteen_day_gap_sum', 'every_sixteen_day_gap_mean', 'every_sixteen_day_gap_min', 'every_sixteen_day_gap_std', 'every_sixteen_day_gap_max']
data = pd.merge(data, every_sixteen_day_gap, on='UID', how='left')

# 每七天用户的rate
every_sixteen_day_rate = transaction_data[['UID', 'day', 'market_code']]
every_sixteen_day_rate['every_day_count'] = 1
every_sixteen_day_rate = every_sixteen_day_rate.groupby(['UID', 'day', 'market_code']).agg('count').reset_index()
every_sixteen_day_rate['before_day_count'] = every_sixteen_day_rate.groupby('UID')['every_day_count'].shift(7)
every_sixteen_day_rate['every_sixteen_day_rate'] = every_sixteen_day_rate['every_day_count'] / every_sixteen_day_rate['before_day_count']
every_sixteen_day_rate = every_sixteen_day_rate[['UID', 'every_sixteen_day_rate']]
every_sixteen_day_rate = every_sixteen_day_rate.groupby('UID')['every_sixteen_day_rate'].agg('sum').reset_index()
data = pd.merge(data, every_sixteen_day_rate, on='UID', how='left')

In [34]:
"""
时间差特征
"""
# 每一天用户操作的时间差
every_day_time_gap = operation_data[['UID', 'day']]
every_day_time_gap = every_day_time_gap.sort_values(['UID', 'day'])
every_day_time_gap['last_day'] = every_day_time_gap.groupby('UID')['day'].shift(1)
every_day_time_gap['last_day_gap'] = every_day_time_gap['day'] - every_day_time_gap['last_day']
every_day_time_gap = every_day_time_gap[['UID', 'last_day_gap']]
every_day_time_gap = every_day_time_gap.groupby('UID')['last_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_day_time_gap.columns = ['UID', 'every_day_time_gap_sum', 'every_day_time_gap_std', 'every_day_time_gap_min', 'every_day_time_gap_mean', 'every_day_time_gap_max']
data = pd.merge(data, every_day_time_gap, on='UID', how='left')

# 每一天每一小时用户操作的时间差
every_hour_time_gap = operation_data[['UID', 'day', 'hour']]
every_hour_time_gap['hour_'] = every_hour_time_gap[['day', 'hour']].apply(lambda x: 24 * (x['day'] - 1) + x['hour'], axis=1)
every_hour_time_gap = every_hour_time_gap[['UID', 'hour_']]
every_hour_time_gap = every_hour_time_gap.sort_values(['UID', 'hour_'])
every_hour_time_gap['last_day_hour'] = every_hour_time_gap.groupby('UID')['hour_'].shift(1)
every_hour_time_gap['every_hour_time_gap'] = every_hour_time_gap['hour_'] - every_hour_time_gap['last_day_hour']
every_hour_time_gap = every_hour_time_gap[['UID', 'every_hour_time_gap']]
every_hour_time_gap = every_hour_time_gap.groupby('UID')['every_hour_time_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_hour_time_gap.columns = ['UID', 'every_hour_time_gap_sum', 'every_hour_time_gap_std', 'every_hour_time_gap_mean', 'every_hour_time_gap_min', 'every_hour_time_gap_max']
data = pd.merge(data, every_hour_time_gap, on='UID', how='left')

# 每一天每一分钟用户操作的时间差
every_minute_time_gap = operation_data[['UID', 'day', 'time']]
every_minute_time_gap['minute'] = every_minute_time_gap[['day', 'time']].apply(lambda x: 1440 * (x['day'] - 1) + 60 * int(x['time'].split(':')[0]) + int(x['time'].split(':')[1]), axis=1)
every_minute_time_gap = every_minute_time_gap[['UID', 'minute']]
every_minute_time_gap = every_minute_time_gap.sort_values(['UID', 'minute'])
every_minute_time_gap['last_minute'] = every_minute_time_gap.groupby('UID')['minute'].shift(1)
every_minute_time_gap['last_minute_gap'] = every_minute_time_gap['minute'] - every_minute_time_gap['last_minute']
every_minute_time_gap = every_minute_time_gap[['UID', 'last_minute_gap']]
every_minute_time_gap = every_minute_time_gap.groupby('UID')['last_minute_gap'].agg({'sum', 'mean', 'std', 'max', 'min'}).reset_index()
data = pd.merge(data, every_minute_time_gap, on='UID', how='left')

# 每三天用户操作的时间差
every_three_day_time_gap = operation_data[['UID', 'day']]
every_three_day_time_gap = every_three_day_time_gap.sort_values(['UID', 'day'])
every_three_day_time_gap['last_three_day'] = every_three_day_time_gap.groupby('UID')['day'].shift(3)
every_three_day_time_gap['last_three_day_gap'] = every_three_day_time_gap['day'] - every_three_day_time_gap['last_three_day']
every_three_day_time_gap = every_three_day_time_gap[['UID', 'last_three_day_gap']]
every_three_day_time_gap = every_three_day_time_gap.groupby('UID')['last_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_three_day_time_gap.columns = ['UID', 'every_three_day_time_gap_sum', 'every_three_day_time_gap_std', 'every_three_day_time_gap_min', 'every_three_day_time_gap_mean', 'every_three_day_time_gap_max']
data = pd.merge(data, every_three_day_time_gap, on='UID', how='left')

# 每七天用户操作的时间差
every_sixteen_day_time_gap = operation_data[['UID', 'day']]
every_sixteen_day_time_gap = every_sixteen_day_time_gap.sort_values(['UID', 'day'])
every_sixteen_day_time_gap['last_sixteen_day'] = every_sixteen_day_time_gap.groupby('UID')['day'].shift(7)
every_sixteen_day_time_gap['last_sixteen_day_gap'] = every_sixteen_day_time_gap['day'] - every_sixteen_day_time_gap['last_sixteen_day']
every_sixteen_day_time_gap = every_sixteen_day_time_gap[['UID', 'last_sixteen_day_gap']]
every_sixteen_day_time_gap = every_sixteen_day_time_gap.groupby('UID')['last_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_sixteen_day_time_gap.columns = ['UID', 'every_sixteen_day_time_gap_sum', 'every_sixteen_day_time_gap_std', 'every_sixteen_day_time_gap_min', 'every_sixteen_day_time_gap_mean', 'every_sixteen_day_time_gap_max']
data = pd.merge(data, every_sixteen_day_time_gap, on='UID', how='left')

In [35]:
"""
时间差特征
"""
# 每一天用户操作的时间差
every_day_time2_gap = transaction_data[['UID', 'day']]
every_day_time2_gap = every_day_time2_gap.sort_values(['UID', 'day'])
every_day_time2_gap['last_day'] = every_day_time2_gap.groupby('UID')['day'].shift(1)
every_day_time2_gap['last_day_gap'] = every_day_time2_gap['day'] - every_day_time2_gap['last_day']
every_day_time2_gap = every_day_time2_gap[['UID', 'last_day_gap']]
every_day_time2_gap = every_day_time2_gap.groupby('UID')['last_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_day_time2_gap.columns = ['UID', 'every_day_time2_gap_sum', 'every_day_time2_gap_std', 'every_day_time2_gap_min', 'every_day_time2_gap_mean', 'every_day_time2_gap_max']
data = pd.merge(data, every_day_time2_gap, on='UID', how='left')

# 每一天每一小时用户操作的时间差
every_hour_time2_gap = transaction_data[['UID', 'day', 'hour']]
every_hour_time2_gap['hour_'] = every_hour_time2_gap[['day', 'hour']].apply(lambda x: 24 * (x['day'] - 1) + x['hour'], axis=1)
every_hour_time2_gap = every_hour_time2_gap[['UID', 'hour_']]
every_hour_time2_gap = every_hour_time2_gap.sort_values(['UID', 'hour_'])
every_hour_time2_gap['last_day_hour'] = every_hour_time2_gap.groupby('UID')['hour_'].shift(1)
every_hour_time2_gap['every_hour_time2_gap'] = every_hour_time2_gap['hour_'] - every_hour_time2_gap['last_day_hour']
every_hour_time2_gap = every_hour_time2_gap[['UID', 'every_hour_time2_gap']]
every_hour_time2_gap = every_hour_time2_gap.groupby('UID')['every_hour_time2_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_hour_time2_gap.columns = ['UID', 'every_hour_time2_gap_sum', 'every_hour_time2_gap_std', 'every_hour_time2_gap_mean', 'every_hour_time2_gap_min', 'every_hour_time2_gap_max']
data = pd.merge(data, every_hour_time2_gap, on='UID', how='left')

# 每一天每一分钟用户操作的时间差
every_minute_time2_gap = transaction_data[['UID', 'day', 'time']]
every_minute_time2_gap['minute'] = every_minute_time2_gap[['day', 'time']].apply(lambda x: 1440 * (x['day'] - 1) + 60 * int(x['time'].split(':')[0]) + int(x['time'].split(':')[1]), axis=1)
every_minute_time2_gap = every_minute_time2_gap[['UID', 'minute']]
every_minute_time2_gap = every_minute_time2_gap.sort_values(['UID', 'minute'])
every_minute_time2_gap['last_minute'] = every_minute_time2_gap.groupby('UID')['minute'].shift(1)
every_minute_time2_gap['last_minute_gap'] = every_minute_time2_gap['minute'] - every_minute_time2_gap['last_minute']
every_minute_time2_gap = every_minute_time2_gap[['UID', 'last_minute_gap']]
every_minute_time2_gap = every_minute_time2_gap.groupby('UID')['last_minute_gap'].agg({'sum', 'mean', 'std', 'max', 'min'}).reset_index()
data = pd.merge(data, every_minute_time2_gap, on='UID', how='left')

# 每三天用户操作的时间差
every_three_day_time2_gap = transaction_data[['UID', 'day']]
every_three_day_time2_gap = every_three_day_time2_gap.sort_values(['UID', 'day'])
every_three_day_time2_gap['last_three_day'] = every_three_day_time2_gap.groupby('UID')['day'].shift(3)
every_three_day_time2_gap['last_three_day_gap'] = every_three_day_time2_gap['day'] - every_three_day_time2_gap['last_three_day']
every_three_day_time2_gap = every_three_day_time2_gap[['UID', 'last_three_day_gap']]
every_three_day_time2_gap = every_three_day_time2_gap.groupby('UID')['last_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_three_day_time2_gap.columns = ['UID', 'every_three_day_time2_gap_sum', 'every_three_day_time2_gap_std', 'every_three_day_time2_gap_min', 'every_three_day_time2_gap_mean', 'every_three_day_time2_gap_max']
data = pd.merge(data, every_three_day_time2_gap, on='UID', how='left')

# 每七天用户操作的时间差
every_sixteen_day_time2_gap = transaction_data[['UID', 'day']]
every_sixteen_day_time2_gap = every_sixteen_day_time2_gap.sort_values(['UID', 'day'])
every_sixteen_day_time2_gap['last_sixteen_day'] = every_sixteen_day_time2_gap.groupby('UID')['day'].shift(7)
every_sixteen_day_time2_gap['last_sixteen_day_gap'] = every_sixteen_day_time2_gap['day'] - every_sixteen_day_time2_gap['last_sixteen_day']
every_sixteen_day_time2_gap = every_sixteen_day_time2_gap[['UID', 'last_sixteen_day_gap']]
every_sixteen_day_time2_gap = every_sixteen_day_time2_gap.groupby('UID')['last_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
every_sixteen_day_time2_gap.columns = ['UID', 'every_sixteen_day_time2_gap_sum', 'every_sixteen_day_time2_gap_std', 'every_sixteen_day_time2_gap_min', 'every_sixteen_day_time2_gap_mean', 'every_sixteen_day_time2_gap_max']
data = pd.merge(data, every_sixteen_day_time2_gap, on='UID', how='left')

In [36]:
def get_UID_time_label(data, x, transaction_data):
    """
    UID与x和trans_amt交叉的时间频率
    """
    # 用户merchant每天每小时的统计
    merchant_every_hour = transaction_data[['UID', x, 'day', 'hour', 'trans_amt']]
    merchant_every_hour = merchant_every_hour.groupby(['UID', x, 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
    merchant_every_hour = merchant_every_hour[['UID', 'trans_amt']]
    merchant_every_hour = merchant_every_hour.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_hour.columns = ['UID', x + '_merchant_every_hour_std', x + '_merchant_every_hour_mean', x + '_merchant_every_hour_min', x + '_merchant_every_hour_max']
    data = pd.merge(data, merchant_every_hour, on='UID', how='left')

    # 用户merchant每天每小时的gap
    merchant_every_hour_gap = transaction_data[['UID', x, 'day', 'hour', 'trans_amt']]
    merchant_every_hour_gap = merchant_every_hour_gap.groupby(['UID', x, 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
    merchant_every_hour_gap = merchant_every_hour_gap[['UID', 'trans_amt']]
    merchant_every_hour_gap['trans_amt_before'] = merchant_every_hour_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_hour_gap['trans_amt_hour_gap'] = merchant_every_hour_gap['trans_amt'] - merchant_every_hour_gap['trans_amt_before']
    merchant_every_hour_gap = merchant_every_hour_gap[['UID', 'trans_amt_hour_gap']]
    merchant_every_hour_gap = merchant_every_hour_gap.groupby('UID')['trans_amt_hour_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_hour_gap.columns = ['UID', x + '_merchant_every_hour_gap_sum', x + '_merchant_every_hour_gap_std', x + '_merchant_every_hour_gap_mean', x + '_merchant_every_hour_gap_min', x + '_merchant_every_hour_gap_max']
    data = pd.merge(data, merchant_every_hour_gap, on='UID', how='left')

    # 用户merchant每天每小时的比率和
    merchant_every_hour_rate = transaction_data[['UID', x, 'day', 'hour', 'trans_amt']]
    merchant_every_hour_rate = merchant_every_hour_rate.groupby(['UID', x, 'day', 'hour'])['trans_amt'].agg('sum').reset_index()
    merchant_every_hour_rate = merchant_every_hour_rate[['UID', 'trans_amt']]
    merchant_every_hour_rate['trans_amt_before'] = merchant_every_hour_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_hour_rate['trans_amt_hour_rate'] = merchant_every_hour_rate['trans_amt'] / merchant_every_hour_rate['trans_amt_before']
    merchant_every_hour_rate = merchant_every_hour_rate[['UID', 'trans_amt_hour_rate']]
    merchant_every_hour_rate = merchant_every_hour_rate.groupby('UID')['trans_amt_hour_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_hour_rate, on='UID', how='left')


    # 用户merchant每天的统计
    merchant_every_day = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_day = merchant_every_day.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_day = merchant_every_day[['UID', 'trans_amt']]
    merchant_every_day = merchant_every_day.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_day.columns = ['UID', x + '_merchant_every_day_std', x + '_merchant_every_day_mean', x + '_merchant_every_day_min', x + '_merchant_every_day_max']
    data = pd.merge(data, merchant_every_day, on='UID', how='left')

    # 用户merchant每天的gap
    merchant_every_day_gap = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_day_gap = merchant_every_day_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_day_gap = merchant_every_day_gap[['UID', 'trans_amt']]
    merchant_every_day_gap['trans_amt_before'] = merchant_every_day_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_day_gap['trans_amt_day_gap'] = merchant_every_day_gap['trans_amt'] - merchant_every_day_gap['trans_amt_before']
    merchant_every_day_gap = merchant_every_day_gap[['UID', 'trans_amt_day_gap']]
    merchant_every_day_gap = merchant_every_day_gap.groupby('UID')['trans_amt_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_day_gap.columns = ['UID', x + '_merchant_every_day_gap_sum', x + '_merchant_every_day_gap_mean', x + '_merchant_every_day_gap_max', x + '_merchant_every_day_gap_min', x + '_merchant_every_day_gap_std']
    data = pd.merge(data, merchant_every_day_gap, on='UID', how='left')

    # 用户merchant每天的rate
    merchant_every_day_rate = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_day_rate = merchant_every_day_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_day_rate = merchant_every_day_rate[['UID', 'trans_amt']]
    merchant_every_day_rate['trans_amt_before'] = merchant_every_day_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_day_rate['trans_amt_day_rate'] = merchant_every_day_rate['trans_amt'] / merchant_every_day_rate['trans_amt_before']
    merchant_every_day_rate = merchant_every_day_rate[['UID', 'trans_amt_day_rate']]
    merchant_every_day_rate = merchant_every_day_rate.groupby('UID')['trans_amt_day_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_day_rate, on='UID', how='left')


    # 用户merchant每天半夜的统计量
    merchant_every_morning = transaction_data[transaction_data.hour <= 5][['UID', x, 'day', 'trans_amt']]
    merchant_every_morning = merchant_every_morning.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_morning = merchant_every_morning[['UID', 'trans_amt']]
    merchant_every_morning = merchant_every_morning.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_morning.columns = ['UID', x + '_merchant_every_morning_std', x + '_merchant_every_morning_mean', x + '_merchant_every_morning_min', x + '_merchant_every_morning_max']
    data = pd.merge(data, merchant_every_morning, on='UID', how='left')

    # 用户merchant每天半夜的gap
    merchant_every_morning_gap = transaction_data[transaction_data.hour <= 5][['UID', x, 'day', 'trans_amt']]
    merchant_every_morning_gap = merchant_every_morning_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_morning_gap = merchant_every_morning_gap[['UID', 'trans_amt']]
    merchant_every_morning_gap['trans_amt_before'] = merchant_every_morning_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_morning_gap[x + '_merchant_every_morning_gap'] = merchant_every_morning_gap['trans_amt'] - merchant_every_morning_gap['trans_amt_before']
    merchant_every_morning_gap = merchant_every_morning_gap[['UID', x + '_merchant_every_morning_gap']]
    merchant_every_morning_gap = merchant_every_morning_gap.groupby('UID')[x + '_merchant_every_morning_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_morning_gap.columns = ['UID', x + '_merchant_every_morning_gap_sum', x + '_merchant_every_morning_gap_std', x + '_merchant_every_morning_gap_mean', x + '_merchant_every_morning_gap_min', x + '_merchant_every_morning_gap_max']
    data = pd.merge(data, merchant_every_morning_gap, on='UID', how='left')

    # 用户merchant每天半夜的rate
    merchant_every_morning_rate = transaction_data[transaction_data.hour <= 5][['UID', x, 'day', 'trans_amt']]
    merchant_every_morning_rate = merchant_every_morning_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_morning_rate = merchant_every_morning_rate[['UID', 'trans_amt']]
    merchant_every_morning_rate['trans_amt_before'] = merchant_every_morning_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_morning_rate[x + '_merchant_every_morning_rate'] = merchant_every_morning_rate['trans_amt'] / merchant_every_morning_rate['trans_amt_before']
    merchant_every_morning_rate = merchant_every_morning_rate[['UID', x + '_merchant_every_morning_rate']]
    merchant_every_morning_rate = merchant_every_morning_rate.groupby('UID')[x + '_merchant_every_morning_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_morning_rate, on='UID', how='left')


    # 用户merchant每天早上的统计量
    merchant_every_work_time = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', x, 'day', 'trans_amt']]
    merchant_every_work_time = merchant_every_work_time.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_work_time = merchant_every_work_time[['UID', 'trans_amt']]
    merchant_every_work_time = merchant_every_work_time.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_work_time.columns = ['UID', x + '_merchant_every_work_time_std', x + '_merchant_every_work_time_mean', x + '_merchant_every_work_time_min', x + '_merchant_every_work_time_max']
    data = pd.merge(data, merchant_every_work_time, on='UID', how='left')

    # 用户merchant每天早上的gap
    merchant_every_work_time_gap = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', x, 'day', 'trans_amt']]
    merchant_every_work_time_gap = merchant_every_work_time_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_work_time_gap = merchant_every_work_time_gap[['UID', 'trans_amt']]
    merchant_every_work_time_gap['trans_amt_before'] = merchant_every_work_time_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_work_time_gap[x + '_merchant_every_work_time_gap'] = merchant_every_work_time_gap['trans_amt'] - merchant_every_work_time_gap['trans_amt_before']
    merchant_every_work_time_gap = merchant_every_work_time_gap[['UID', x + '_merchant_every_work_time_gap']]
    merchant_every_work_time_gap = merchant_every_work_time_gap.groupby('UID')[x + '_merchant_every_work_time_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_work_time_gap.columns = ['UID', x + '_merchant_every_work_time_gap_sum', x + '_merchant_every_work_time_gap_std', x + '_merchant_every_work_time_gap_mean', x + '_merchant_every_work_time_gap_min', x + '_merchant_every_work_time_gap_max']
    data = pd.merge(data, merchant_every_work_time_gap, on='UID', how='left')

    # 用户merchant每天早上的rate
    merchant_every_work_time_rate = transaction_data[(transaction_data.hour >= 6)&(transaction_data.hour <= 11)][['UID', x, 'day', 'trans_amt']]
    merchant_every_work_time_rate = merchant_every_work_time_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_work_time_rate = merchant_every_work_time_rate[['UID', 'trans_amt']]
    merchant_every_work_time_rate['trans_amt_before'] = merchant_every_work_time_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_work_time_rate[x + '_merchant_every_work_time_rate'] = merchant_every_work_time_rate['trans_amt'] / merchant_every_work_time_rate['trans_amt_before']
    merchant_every_work_time_rate = merchant_every_work_time_rate[['UID', x + '_merchant_every_work_time_rate']]
    merchant_every_work_time_rate = merchant_every_work_time_rate.groupby('UID')[x + '_merchant_every_work_time_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_work_time_rate, on='UID', how='left')

    # 用户merchant每天下午的统计量
    merchant_every_afternoon = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', x, 'day', 'trans_amt']]
    merchant_every_afternoon = merchant_every_afternoon.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_afternoon = merchant_every_afternoon[['UID', 'trans_amt']]
    merchant_every_afternoon = merchant_every_afternoon.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_afternoon.columns = ['UID', x + '_merchant_every_afternoon_std', x + '_merchant_every_afternoon_mean', x + '_merchant_every_afternoon_min', x + '_merchant_every_afternoon_max']
    data = pd.merge(data, merchant_every_afternoon, on='UID', how='left')

    # 用户merchant每天下午的gap
    merchant_every_afternoon_gap = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', x, 'day', 'trans_amt']]
    merchant_every_afternoon_gap = merchant_every_afternoon_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_afternoon_gap = merchant_every_afternoon_gap[['UID', 'trans_amt']]
    merchant_every_afternoon_gap['trans_amt_before'] = merchant_every_afternoon_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_afternoon_gap[x + '_merchant_every_afternoon_gap'] = merchant_every_afternoon_gap['trans_amt'] - merchant_every_afternoon_gap['trans_amt_before']
    merchant_every_afternoon_gap = merchant_every_afternoon_gap[['UID', x + '_merchant_every_afternoon_gap']]
    merchant_every_afternoon_gap = merchant_every_afternoon_gap.groupby('UID')[x + '_merchant_every_afternoon_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_afternoon_gap.columns = ['UID', x + '_merchant_every_afternoon_gap_sum', x + '_merchant_every_afternoon_gap_std', x + '_merchant_every_afternoon_gap_mean', x + '_merchant_every_afternoon_gap_min', x + '_merchant_every_afternoon_gap_max']
    data = pd.merge(data, merchant_every_afternoon_gap, on='UID', how='left')

    # 用户merchant每天下午的rate
    merchant_every_afternoon_rate = transaction_data[(transaction_data.hour >= 12)&(transaction_data.hour <= 17)][['UID', x, 'day', 'trans_amt']]
    merchant_every_afternoon_rate = merchant_every_afternoon_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_afternoon_rate = merchant_every_afternoon_rate[['UID', 'trans_amt']]
    merchant_every_afternoon_rate['trans_amt_before'] = merchant_every_afternoon_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_afternoon_rate[x + '_merchant_every_afternoon_rate'] = merchant_every_afternoon_rate['trans_amt'] / merchant_every_afternoon_rate['trans_amt_before']
    merchant_every_afternoon_rate = merchant_every_afternoon_rate[['UID', x + '_merchant_every_afternoon_rate']]
    merchant_every_afternoon_rate = merchant_every_afternoon_rate.groupby('UID')[x + '_merchant_every_afternoon_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_afternoon_rate, on='UID', how='left')

    # 用户merchant每天晚上的统计量
    merchant_every_night = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', x, 'day', 'trans_amt']]
    merchant_every_night = merchant_every_night.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_night = merchant_every_night[['UID', 'trans_amt']]
    merchant_every_night = merchant_every_night.groupby('UID')['trans_amt'].agg({'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_night.columns = ['UID', x + '_merchant_every_night_std', x + '_merchant_every_night_mean', x + '_merchant_every_night_min', x + '_merchant_every_night_max']
    data = pd.merge(data, merchant_every_night, on='UID', how='left')

    # 用户merchant每天晚上的gap
    merchant_every_night_gap = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', x, 'day', 'trans_amt']]
    merchant_every_night_gap = merchant_every_night_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_night_gap = merchant_every_night_gap[['UID', 'trans_amt']]
    merchant_every_night_gap['trans_amt_before'] = merchant_every_night_gap.groupby('UID')['trans_amt'].shift(1)
    merchant_every_night_gap[x + '_merchant_every_night_gap'] = merchant_every_night_gap['trans_amt'] - merchant_every_night_gap['trans_amt_before']
    merchant_every_night_gap = merchant_every_night_gap[['UID', x + '_merchant_every_night_gap']]
    merchant_every_night_gap = merchant_every_night_gap.groupby('UID')[x + '_merchant_every_night_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_night_gap.columns = ['UID', x + '_merchant_every_night_gap_sum', x + '_merchant_every_night_gap_std', x + '_merchant_every_night_gap_mean', x + '_merchant_every_night_gap_min', x + '_merchant_every_night_gap_max']
    data = pd.merge(data, merchant_every_night_gap, on='UID', how='left')

    # 用户merchant每天晚上的rate
    merchant_every_night_rate = transaction_data[(transaction_data.hour >= 18)&(transaction_data.hour <= 23)][['UID', x, 'day', 'trans_amt']]
    merchant_every_night_rate = merchant_every_night_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_night_rate = merchant_every_night_rate[['UID', 'trans_amt']]
    merchant_every_night_rate['trans_amt_before'] = merchant_every_night_rate.groupby('UID')['trans_amt'].shift(1)
    merchant_every_night_rate[x + '_merchant_every_night_rate'] = merchant_every_night_rate['trans_amt'] / merchant_every_night_rate['trans_amt_before']
    merchant_every_night_rate = merchant_every_night_rate[['UID', x + '_merchant_every_night_rate']]
    merchant_every_night_rate = merchant_every_night_rate.groupby('UID')[x + '_merchant_every_night_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_night_rate, on='UID', how='left')


    # 用户merchant每三天的gap
    merchant_every_three_day_gap = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_three_day_gap = merchant_every_three_day_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_three_day_gap = merchant_every_three_day_gap[['UID', 'trans_amt']]
    merchant_every_three_day_gap['trans_amt_before'] = merchant_every_three_day_gap.groupby('UID')['trans_amt'].shift(3)
    merchant_every_three_day_gap['trans_amt_three_day_gap'] = merchant_every_three_day_gap['trans_amt'] - merchant_every_three_day_gap['trans_amt_before']
    merchant_every_three_day_gap = merchant_every_three_day_gap[['UID', 'trans_amt_three_day_gap']]
    merchant_every_three_day_gap = merchant_every_three_day_gap.groupby('UID')['trans_amt_three_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_three_day_gap.columns = ['UID', x + '_merchant_every_three_day_gap_sum', x + '_merchant_every_three_day_gap_std', x + '_merchant_every_three_day_gap_mean', x + '_merchant_every_three_day_gap_min', x + '_merchant_every_three_day_gap_max']
    data = pd.merge(data, merchant_every_three_day_gap, on='UID', how='left')

    # 用户merchant每三天的rate
    merchant_every_three_day_rate = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_three_day_rate = merchant_every_three_day_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_three_day_rate = merchant_every_three_day_rate[['UID', 'trans_amt']]
    merchant_every_three_day_rate['trans_amt_before'] = merchant_every_three_day_rate.groupby('UID')['trans_amt'].shift(3)
    merchant_every_three_day_rate['trans_amt_three_day_rate'] = merchant_every_three_day_rate['trans_amt'] / merchant_every_three_day_rate['trans_amt_before']
    merchant_every_three_day_rate = merchant_every_three_day_rate[['UID', 'trans_amt_three_day_rate']]
    merchant_every_three_day_rate = merchant_every_three_day_rate.groupby('UID')['trans_amt_three_day_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_three_day_rate, on='UID', how='left')


    # 用户merchant每七天的gap
    merchant_every_sixteen_day_gap = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap[['UID', 'trans_amt']]
    merchant_every_sixteen_day_gap['trans_amt_before'] = merchant_every_sixteen_day_gap.groupby('UID')['trans_amt'].shift(7)
    merchant_every_sixteen_day_gap['trans_amt_sixteen_day_gap'] = merchant_every_sixteen_day_gap['trans_amt'] - merchant_every_sixteen_day_gap['trans_amt_before']
    merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap[['UID', 'trans_amt_sixteen_day_gap']]
    merchant_every_sixteen_day_gap = merchant_every_sixteen_day_gap.groupby('UID')['trans_amt_sixteen_day_gap'].agg({'sum', 'mean', 'max', 'min', 'std'}).reset_index()
    merchant_every_sixteen_day_gap.columns = ['UID', x + '_merchant_every_sixteen_day_gap_sum', x + '_merchant_every_sixteen_day_gap_std', x + '_merchant_every_sixteen_day_gap_mean', x + '_merchant_every_sixteen_day_gap_min', x + '_merchant_every_sixteen_day_gap_max']
    data = pd.merge(data, merchant_every_sixteen_day_gap, on='UID', how='left')

    # 用户merchant每七天的rate
    merchant_every_sixteen_day_rate = transaction_data[['UID', x, 'day', 'trans_amt']]
    merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate.groupby(['UID', x, 'day'])['trans_amt'].agg('sum').reset_index()
    merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate[['UID', 'trans_amt']]
    merchant_every_sixteen_day_rate['trans_amt_before'] = merchant_every_sixteen_day_rate.groupby('UID')['trans_amt'].shift(7)
    merchant_every_sixteen_day_rate['trans_amt_sixteen_day_rate'] = merchant_every_sixteen_day_rate['trans_amt'] / merchant_every_sixteen_day_rate['trans_amt_before']
    merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate[['UID', 'trans_amt_sixteen_day_rate']]
    merchant_every_sixteen_day_rate = merchant_every_sixteen_day_rate.groupby('UID')['trans_amt_sixteen_day_rate'].agg('sum').reset_index()
    data = pd.merge(data, merchant_every_sixteen_day_rate, on='UID', how='left')
    
    print(x + ' is over!')
    return data

for x in ['mac1', 'ip1_sub', 'trans_type1', 'market_code']:
    data = get_UID_time_label(data, x, transaction_data)

mac1 is over!


KeyboardInterrupt: 

In [ ]:
temp = data.copy()

In [ ]:
def get_two_count(data, t_data, x1, x2, t):
    """
    获得x1和x2的交叉count
    """
    temp = t_data[[x1, x2, 'UID']]
    
    operation_or_transaction_data = temp.copy()
    operation_or_transaction_data.drop_duplicates(inplace=True)
    
    temp[x1 + '_' + x2 + '_count'] = temp['UID']
    temp = temp.groupby([x1, x2])[x1 + '_' + x2 + '_count'].agg('nunique').reset_index()
    
    operation_or_transaction_data = pd.merge(operation_or_transaction_data, temp, on=[x1, x2], how='left')
    operation_or_transaction_data[x1 + '_' + x2 + '_count_uid_' + t] = 1
    operation_or_transaction_data = operation_or_transaction_data.groupby('UID')[x1 + '_' + x2 + '_count_uid_' + t].agg('sum').reset_index()
    
    data = pd.merge(data, operation_or_transaction_data, on='UID', how='left')
    return data

data = get_two_count(data, operation_data, 'ip1', 'geo_code', 'operation_data')
data = get_two_count(data, operation_data, 'ip1', 'ip1_sub', 'operation_data')
data = get_two_count(data, operation_data, 'ip1', 'wifi', 'operation_data')
data = get_two_count(data, operation_data, 'geo_code', 'ip1_sub', 'operation_data')
data = get_two_count(data, operation_data, 'geo_code', 'wifi', 'operation_data')
data = get_two_count(data, operation_data, 'ip1_sub', 'wifi', 'operation_data')

data = get_two_count(data, transaction_data, 'merchant', 'geo_code', 'transaction_data')
data = get_two_count(data, transaction_data, 'merchant', 'ip1_sub', 'transaction_data')
data = get_two_count(data, transaction_data, 'merchant', 'code1', 'transaction_data')
data = get_two_count(data, transaction_data, 'geo_code', 'ip1_sub', 'transaction_data')
data = get_two_count(data, transaction_data, 'geo_code', 'code1', 'transaction_data')
data = get_two_count(data, transaction_data, 'ip1_sub', 'code1', 'transaction_data')

In [ ]:
def get_two_shuxing_count(data, t_data, x1, x2, t):
    """
    groupby(x1)[x2].agg('nunique').reset_index()
    """
    UID = t_data[['UID', x1]].drop_duplicates()
    temp = t_data[[x1, x2]]
    temp[x2 + '_of_nunique_' + x1] = temp[x2]
    temp = temp.groupby(x1)[x2 + '_of_nunique_' + x1].agg('nunique').reset_index()
    
    UID = pd.merge(UID, temp, on=x1, how='left')
    UID = UID.drop(x1, axis=1)
    UID = UID.groupby('UID')[x2 + '_of_nunique_' + x1].agg('sum').reset_index()
    
    data = pd.merge(data, UID, on='UID', how='left')
    return data
    
for x1 in ['ip1', 'geo_code', 'ip1_sub', 'wifi']:
    for x2 in ['ip1', 'geo_code', 'ip1_sub', 'wifi']:
        if x1 != x2:
            data = get_two_shuxing_count(data, operation_data, x1, x2, 'operation_data')
for x1 in ['device_code1', 'device_code2', 'device_code3']:
    for x2 in ['ip1', 'geo_code', 'ip1_sub', 'wifi']:
        data = get_two_shuxing_count(data, operation_data, x1, x2, 'operation_data')
print('operation_data is over!')
for x1 in ['merchant', 'geo_code', 'ip1_sub', 'code1']:
    for x2 in ['merchant', 'geo_code', 'ip1_sub', 'code1']:
        if x1 != x2:
            data = get_two_shuxing_count(data, transaction_data, x1, x2, 'transaction_data')
print('transaction_data is over!')

In [ ]:
def get_three_count(data, t_data, x1, x2, x3, t):
    temp = t_data[[x1, x2, x3, 'UID']]
    
    operation_or_transaction_data = temp.copy()
    operation_or_transaction_data.drop_duplicates(inplace=True)
    
    temp[x1 + '_' + x2 + '_' + x3 + '_count'] = temp['UID']
    temp = temp.groupby([x1, x2, x3])[x1 + '_' + x2 + '_' + x3 + '_count'].agg('nunique').reset_index()
    
    operation_or_transaction_data = pd.merge(operation_or_transaction_data, temp, on=[x1, x2, x3], how='left')
    operation_or_transaction_data[x1 + '_' + x2 + '_' + x3 + '_count'] = 1
    operation_or_transaction_data = operation_or_transaction_data.groupby('UID')[x1 + '_' + x2 + '_' + x3 + '_count'].agg('sum').reset_index()
    
    print(operation_or_transaction_data)
    data = pd.merge(data, operation_or_transaction_data, on='UID', how='left')
    return data

data = get_three_count(data, transaction_data, 'merchant', 'day', 'device1', 'transaction')
data = get_three_count(data, transaction_data, 'merchant', 'day', 'device2', 'transaction')

In [37]:
data = data.fillna(-1)
data.head(5)

,Tag,UID,has_operation,has_transaction,operation_transaction_count_gap,success_rate,mode_success_max,mode_success_std,mode_success_sum,mode_success_min,...,mac1_merchant_every_three_day_gap_mean,mac1_merchant_every_three_day_gap_min,mac1_merchant_every_three_day_gap_max,trans_amt_three_day_rate_y,mac1_merchant_every_sixteen_day_gap_sum,mac1_merchant_every_sixteen_day_gap_std,mac1_merchant_every_sixteen_day_gap_mean,mac1_merchant_every_sixteen_day_gap_min,mac1_merchant_every_sixteen_day_gap_max,trans_amt_sixteen_day_rate_y
0,1.0,10000,1,1,-7.0,1.000000,1.0,1.000000,3.000000,0.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,10002,1,1,-8.0,1.000000,1.0,1.000000,7.000000,0.000000,...,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0
2,0.0,10003,1,1,-11.0,1.000000,1.0,1.000000,2.000000,0.000000,...,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0
3,0.0,10004,1,1,-27.0,0.941176,1.0,0.333333,8.333333,0.222222,...,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0
4,0.0,10005,0,1,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,...,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0


In [38]:
drop = ['UID', 'Tag']

train = data[data.Tag != -1]
test = data[data.Tag == -1]

y_train = train.loc[:,'Tag']
res = test.loc[:, ['UID']]

train.drop(drop, axis=1, inplace=True)
print('train:',train.shape)
print('train.columns:',train.columns)

test.drop(drop, axis=1, inplace=True)
print('test:',test.shape)

all_feature = [x for x in train.columns]# 所有的特征

X_loc_train = train.values
y_loc_train = y_train.values
X_loc_test = test.values

train: (57389, 1661)
train.columns: Index(['has_operation', 'has_transaction', 'operation_transaction_count_gap',
       'success_rate', 'mode_success_max', 'mode_success_std',
       'mode_success_sum', 'mode_success_min', 'mode_success_mean',
       'day_nunique_of_operation',
       ...
       'mac1_merchant_every_three_day_gap_mean',
       'mac1_merchant_every_three_day_gap_min',
       'mac1_merchant_every_three_day_gap_max', 'trans_amt_three_day_rate_y',
       'mac1_merchant_every_sixteen_day_gap_sum',
       'mac1_merchant_every_sixteen_day_gap_std',
       'mac1_merchant_every_sixteen_day_gap_mean',
       'mac1_merchant_every_sixteen_day_gap_min',
       'mac1_merchant_every_sixteen_day_gap_max',
       'trans_amt_sixteen_day_rate_y'],
      dtype='object', length=1661)
test: (31588, 1661)


In [ ]:
# 模型部分
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=30, max_depth=-1, learning_rate=0.02, n_estimators=3000,
                           max_bin=500, subsample_for_bin=100000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=2018, n_jobs=10, silent=True)

# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
base_auc = []
loss = 0
oof_preds = np.zeros(train.shape[0])
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='auc',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]),
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    base_auc.append(lgb_model.best_score_['valid']['auc'])
    loss += lgb_model.best_score_['valid']['auc']
    oof_preds[test_index] = lgb_model.predict_proba(train.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]
    
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('roc_auc_score:', base_auc, loss/5)
# 加权平均
res['Tag'] = 0
for i in range(5):
    res['Tag'] += res['prob_%s' % str(i)]
res['Tag'] = res['Tag']/5

# 提交结果
mean = res['Tag'].mean()
print('mean:',mean)

endtime = datetime.datetime.now()

Fold 0
[1]	train's auc: 0.952989	valid's auc: 0.94214
Training until validation scores don't improve for 100 rounds.
[2]	train's auc: 0.958286	valid's auc: 0.948767
[3]	train's auc: 0.961965	valid's auc: 0.951978
[4]	train's auc: 0.964586	valid's auc: 0.956656
[5]	train's auc: 0.964669	valid's auc: 0.956738
[6]	train's auc: 0.965246	valid's auc: 0.957904
[7]	train's auc: 0.966045	valid's auc: 0.958898
[8]	train's auc: 0.967934	valid's auc: 0.961831
[9]	train's auc: 0.968299	valid's auc: 0.962125
[10]	train's auc: 0.968438	valid's auc: 0.9627
[11]	train's auc: 0.968973	valid's auc: 0.963046
[12]	train's auc: 0.969297	valid's auc: 0.96309
[13]	train's auc: 0.972041	valid's auc: 0.966445
[14]	train's auc: 0.972717	valid's auc: 0.967588
[15]	train's auc: 0.972602	valid's auc: 0.967342
[16]	train's auc: 0.972945	valid's auc: 0.96782
[17]	train's auc: 0.972709	valid's auc: 0.967515
[18]	train's auc: 0.972856	valid's auc: 0.967663
[19]	train's auc: 0.973759	valid's auc: 0.968432
[20]	train's 

[167]	train's auc: 0.990658	valid's auc: 0.984554
[168]	train's auc: 0.990684	valid's auc: 0.984564
[169]	train's auc: 0.990729	valid's auc: 0.984556
[170]	train's auc: 0.990762	valid's auc: 0.984537
[171]	train's auc: 0.990788	valid's auc: 0.984534
[172]	train's auc: 0.990834	valid's auc: 0.984554
[173]	train's auc: 0.990861	valid's auc: 0.984555
[174]	train's auc: 0.990887	valid's auc: 0.984551
[175]	train's auc: 0.990921	valid's auc: 0.984593
[176]	train's auc: 0.990959	valid's auc: 0.984612
[177]	train's auc: 0.990997	valid's auc: 0.98463
[178]	train's auc: 0.991036	valid's auc: 0.984639
[179]	train's auc: 0.991077	valid's auc: 0.984668
[180]	train's auc: 0.991109	valid's auc: 0.984677
[181]	train's auc: 0.991147	valid's auc: 0.984706
[182]	train's auc: 0.991188	valid's auc: 0.984697
[183]	train's auc: 0.991212	valid's auc: 0.9847
[184]	train's auc: 0.991246	valid's auc: 0.984718
[185]	train's auc: 0.991273	valid's auc: 0.984702
[186]	train's auc: 0.991297	valid's auc: 0.984797
[18

[332]	train's auc: 0.99556	valid's auc: 0.987001
[333]	train's auc: 0.995577	valid's auc: 0.987017
[334]	train's auc: 0.995595	valid's auc: 0.987012
[335]	train's auc: 0.995615	valid's auc: 0.987036
[336]	train's auc: 0.995638	valid's auc: 0.987056
[337]	train's auc: 0.995658	valid's auc: 0.98706
[338]	train's auc: 0.99568	valid's auc: 0.987071
[339]	train's auc: 0.995695	valid's auc: 0.987077
[340]	train's auc: 0.995712	valid's auc: 0.987071
[341]	train's auc: 0.995735	valid's auc: 0.987101
[342]	train's auc: 0.995761	valid's auc: 0.987117
[343]	train's auc: 0.995779	valid's auc: 0.987117
[344]	train's auc: 0.995801	valid's auc: 0.987117
[345]	train's auc: 0.995811	valid's auc: 0.987131
[346]	train's auc: 0.995832	valid's auc: 0.987149
[347]	train's auc: 0.995855	valid's auc: 0.987149
[348]	train's auc: 0.99588	valid's auc: 0.987179
[349]	train's auc: 0.9959	valid's auc: 0.987195
[350]	train's auc: 0.995922	valid's auc: 0.987203
[351]	train's auc: 0.995941	valid's auc: 0.987199
[352]	

[497]	train's auc: 0.997996	valid's auc: 0.987995
[498]	train's auc: 0.998003	valid's auc: 0.987994
[499]	train's auc: 0.998017	valid's auc: 0.987998
[500]	train's auc: 0.998029	valid's auc: 0.987995
[501]	train's auc: 0.998037	valid's auc: 0.988001
[502]	train's auc: 0.998045	valid's auc: 0.988008
[503]	train's auc: 0.998053	valid's auc: 0.988005
[504]	train's auc: 0.998064	valid's auc: 0.988012
[505]	train's auc: 0.998074	valid's auc: 0.988018
[506]	train's auc: 0.998082	valid's auc: 0.988026
[507]	train's auc: 0.998089	valid's auc: 0.988036
[508]	train's auc: 0.9981	valid's auc: 0.988026
[509]	train's auc: 0.998109	valid's auc: 0.988038
[510]	train's auc: 0.998117	valid's auc: 0.988036
[511]	train's auc: 0.998129	valid's auc: 0.988047
[512]	train's auc: 0.998138	valid's auc: 0.98804
[513]	train's auc: 0.998149	valid's auc: 0.988051
[514]	train's auc: 0.998157	valid's auc: 0.988057
[515]	train's auc: 0.998163	valid's auc: 0.988061
[516]	train's auc: 0.998174	valid's auc: 0.98807
[517

In [ ]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

m = tpr_weight_funtion(y_predict=oof_preds,y_true=y_loc_train)
print(m[1])

In [ ]:
# 特征重要性
not_need = ['UID', 'Tag'] 
feature_importance = pd.DataFrame({'features':all_feature, 'imp':lgb_model.feature_importances_}).sort_values('imp', ascending=False)
feature_importance[feature_importance.imp >= 100]

In [171]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[res.UID >= 100000][['UID', 'Tag']].to_csv("./submit/lgb_baseline_%s.csv" % now, index=False)

In [ ]:
0.9529450261780104
0.7628879366316064 - 0.389138 = 0.3737499366